In [2]:
import pandas as pd
from datetime import datetime
import tempfile
import os
import urllib.request

# Import necessary classes from DSSATTools
#
from DSSATTools import (
    crop,
    WeatherStation,
    SoilProfile,
    filex,
    DSSAT
)

# --- 1. Download Data Files ---
# This part fetches the required weather and soil files from the DSSAT GitHub repository
# so the script can run anywhere.

# Weather Files
for year in range(80, 82):
    url = f'https://github.com/DSSAT/dssat-csm-data/blob/develop/Weather/ITHY{year}01.WTH?raw=true'
    response = urllib.request.urlopen(url)
    content = ''.join([l.decode('utf-8') for l in response])
    with open(f'ITHY{year}01.WTH', 'w') as f:
        f.write(content)

# Soil File
url = 'https://github.com/DSSAT/dssat-csm-data/blob/develop/Soil/SOIL.SOL?raw=true'
response = urllib.request.urlopen(url)
content = ''.join([l.decode('utf-8') for l in response])
with open('SOIL.SOL', 'w') as f:
    f.write(content)

print("--- Data files downloaded successfully ---\n")

# --- 2. Define Simulation Components ---

# Weather ☀️
# Create a WeatherStation object from the downloaded files
weather_station = WeatherStation.from_files(['ITHY8001.WTH', 'ITHY8101.WTH'])

# Soil 🌱
# Create a SoilProfile object from the downloaded SOIL.SOL file
soil = SoilProfile.from_file('IBSG910085', 'SOIL.SOL')

# Cultivar 🧬
# Define the sorghum cultivar
cultivar = crop.Sorghum('IB0026')

# Management (FileX Sections) 🚜
# Define the field, linking the weather and soil
field = filex.Field(
    id_field='ITHY0001', wsta=weather_station, id_soil=soil
)

# Define initial conditions
initial_conditions = filex.InitialConditions(
    pcr='SG', icdat=datetime(1980, 7, 3), icrt=500, icres=1300,
    table=pd.DataFrame([
        (10, .06, 2.5, 1.8), (22, .06, 2.5, 1.8), (52, .195, 3., 4.5),
        (82, .21, 3.5, 5.0), (112, 0.2, 2., 2.0), (142, 0.2, 1., 0.7),
        (172, 0.2, 1., 0.6),
    ], columns=['icbl', 'sh2o', 'snh4', 'sno3'])
)

# Define planting details
planting = filex.Planting(
    pdate=datetime(1980, 6, 17), ppop=18, ppoe=18, plme='S',
    plds='R', plrs=45, pldp=5
)

# Define simulation controls, ensuring daily reports are enabled
simulation_controls = filex.SimulationControls(
    general=filex.SCGeneral(sdate=datetime(1980, 6, 17)),
    options=filex.SCOptions(water='Y', nitro='Y'),
    management=filex.SCManagement(irrig='N', ferti='R', harvs='M'),
    outputs=filex.SCOutputs(
        grout='Y',  # Plant Growth daily report (PlantGro.OUT)
        waout='Y',  # Soil Water daily report (SoilWat.OUT)
        niout='Y'   # Soil Nitrogen daily report (SoilNi.OUT)
    )
)

# --- 3. Run the Simulation --- 🚀

# Create the simulation environment in a temporary folder
tmp_dir = tempfile.gettempdir()
dssat = DSSAT(os.path.join(tmp_dir, 'dssat_test_daily'))

# Run the simulation with all the defined components
dssat.run_treatment(
    field=field, cultivar=cultivar, planting=planting,
    initial_conditions=initial_conditions, simulation_controls=simulation_controls
)

# --- 4. Display Daily Reports --- 📊

# Access the parsed output tables and print the first few rows of each daily report.
#
print("\n--- Daily Plant Growth Report (PlantGro.OUT) ---")
daily_growth = dssat.output_tables.get('PlantGro')
if daily_growth is not None:
    print(daily_growth[['DAS', 'GSTD', 'LAID', 'CWAD', 'GWAD']].head())
else:
    print("PlantGro output not available.")

print("\n--- Daily Soil Water Report (SoilWat.OUT) ---")
daily_water = dssat.output_tables.get('SoilWat')
if daily_water is not None:
    print(daily_water[['DAS', 'SWXD', 'SW1D', 'SW2D', 'SW3D']].head())
else:
    print("SoilWat output not available.")

print("\n--- Daily Soil Nitrogen Report (SoilNi.OUT) ---")
daily_nitrogen = dssat.output_tables.get('SoilNi')
if daily_nitrogen is not None:
    print(daily_nitrogen[['DAS', 'NIAD', 'NI1D', 'NI2D', 'NI3D']].head())
else:
    print("SoilNi output not available.")

d:\Final-Year-Project\.venv\Lib\site-packages\DSSATTools\__init__.py:105: DeprecationWarning: DSSATTools version 3.0.0 is a major upgrade and will not be backwards compatible with previous versions. If you are running code that was developed  using a previous DSSATTools version, then install DSSATTools version 2.1.6
  warnings.warn(


--- Data files downloaded successfully ---

C:\Users\gurup\AppData\Local\Temp\dssat_test_daily created.

RUN    TRT FLO MAT TOPWT HARWT  RAIN  TIRR   CET  PESW  TNUP  TNLF   TSON TSOC
           dap dap kg/ha kg/ha    mm    mm    mm    mm kg/ha kg/ha  kg/ha t/ha
  1 SG   1  67 109 12639  5284   592     0   348    99   110    28  13363   95

--- Daily Plant Growth Report (PlantGro.OUT) ---
            DAS  GSTD  LAID  CWAD  GWAD
1980-06-17    1     0   0.0     0     0
1980-06-18    2     0   0.0     0     0
1980-06-19    3     0   0.0     0     0
1980-06-20    4     0   0.0     0     0
1980-06-21    5     0   0.0     0     0

--- Daily Soil Water Report (SoilWat.OUT) ---
            DAS  SWXD   SW1D   SW2D   SW3D
1980-06-16    0    58  0.060  0.085  0.132
1980-06-17    1    58  0.064  0.085  0.132
1980-06-18    2    58  0.053  0.085  0.133
1980-06-19    3    58  0.053  0.086  0.133
1980-06-20    4    58  0.053  0.086  0.133

--- Daily Soil Nitrogen Report (SoilNi.OUT) ---
            DA

In [8]:
import os
import urllib.request
import pandas as pd
from datetime import datetime
from DSSATTools import (
    crop,
    WeatherStation,
    SoilProfile,
    filex,
    DSSAT
)

def download_file(url, target_path):
    """Downloads a file from a URL and saves it locally."""
    try:
        with urllib.request.urlopen(url) as response, open(target_path, 'wb') as out_file:
            data = response.read()  # a `bytes` object
            out_file.write(data)
        print(f"Successfully downloaded {os.path.basename(target_path)}")
    except Exception as e:
        print(f"Error downloading {url}: {e}")
        raise

def setup_simulation_environment():
    """
    Creates a robust, self-contained environment for a DSSAT simulation.

    This function handles the entire workflow:
    1.  Sets up a local workspace directory.
    2.  Downloads all necessary weather and soil data files.
    3.  Parses and processes data into DSSATTools objects.
    4.  Configures a detailed DSSAT experiment for Sorghum.
    5.  Runs the simulation and retrieves daily outputs.
    """
    # 1. Setup Workspace
    # -------------------
    # Create the simulation workspace in a local subdirectory.
    workspace = os.path.join(os.getcwd(), 'dssat_simulation_workspace')
    os.makedirs(workspace, exist_ok=True)
    print(f"DSSAT workspace created at: {workspace}")

    # 2. Data Acquisition and Processing
    # ----------------------------------
    # Download soil data
    soil_url = 'https://raw.githubusercontent.com/DSSAT/dssat-csm-data/develop/Soil/SOIL.SOL'
    soil_path = os.path.join(workspace, 'SOIL.SOL')
    download_file(soil_url, soil_path)

    # Download weather data for 1980 and 1981
    weather_files_to_download = {
        'ITHY8001.WTH': 'https://raw.githubusercontent.com/DSSAT/dssat-csm-data/develop/Weather/ITHY8001.WTH',
        'ITHY8101.WTH': 'https://raw.githubusercontent.com/DSSAT/dssat-csm-data/develop/Weather/ITHY8101.WTH'
    }
    weather_dfs = []
    weather_col_names = ['date', 'srad', 'tmax', 'tmin', 'rain']
    weather_col_specs = [(0, 5), (6, 11), (12, 17), (18, 23), (24, 29)]

    for filename, url in weather_files_to_download.items():
        path = os.path.join(workspace, filename)
        download_file(url, path)
        # Use pandas.read_fwf for robust parsing of fixed-width files
        df = pd.read_fwf(path, colspecs=weather_col_specs, names=weather_col_names, skiprows=4)
        weather_dfs.append(df)

    # Combine, clean, and process weather data
    df_weather = pd.concat(weather_dfs, ignore_index=True)
    df_weather.dropna(subset=['date'], inplace=True) # Remove potential empty lines
    df_weather['date'] = pd.to_datetime(df_weather['date'], format='%y%j')
    
    # Critical step: Remove duplicate dates to prevent AssertionError
    df_weather.drop_duplicates(subset='date', keep='first', inplace=True)
    df_weather.sort_values(by='date', inplace=True)

    # 3. Create DSSATTools Objects
    # ----------------------------
    # Create a WeatherStation object
    weather_station = WeatherStation(
        insi='ITHY', lat=17.53, long=78.27, elev=0,
        tav=25.8, amp=11.8, refht=2.0, wndht=3.0,
        table=df_weather
    )

    # Create a SoilProfile object from the downloaded file
    soil = SoilProfile.from_file('IBSG910085', soil_path)

    # Define the Sorghum cultivar
    cultivar = crop.Sorghum('IB0026')

    # Define the experimental conditions (FileX sections)
    field = filex.Field(
        id_field='ITHY0001', wsta=weather_station, id_soil=soil
    )

    initial_conditions = filex.InitialConditions(
        pcr='SG', icdat=datetime(1980, 7, 3), icrt=500, icrn=1, icre=1,
        icres=1300, icren=0.5, icrip=100, icrid=10,
        table=pd.DataFrame([
            (10, .06, 2.5, 1.8), (22, .06, 2.5, 1.8),
            (52, .195, 3., 4.5), (82, .21, 3.5, 5.0),
            (112, 0.2, 2., 2.0), (142, 0.2, 1., 0.7),
            (172, 0.2, 1., 0.6),
        ], columns=['icbl', 'sh2o', 'snh4', 'sno3'])
    )

    planting = filex.Planting(
        pdate=datetime(1980, 6, 17), ppop=18, ppoe=18, plme='S',
        plds='R', plrs=45, pldp=5
    )
    
    # Configure detailed daily outputs
    outputs_config = filex.SCOutputs(
        fname='N',        # Standard file name
        ovvew='Y',        # Overview.OUT
        sumry='Y',        # Summary.OUT
        grout='Y',        # PlantGro.OUT (Daily Plant Growth)
        waout='Y',        # SoilWat.OUT (Daily Soil Water)
        niout='Y',        # SoilNi.OUT (Daily Soil Nitrogen)
        vbose='Y'         # Verbose output
    )

    simulation_controls = filex.SimulationControls(
        general=filex.SCGeneral(sdate=datetime(1980, 6, 17)),
        options=filex.SCOptions(water='Y', nitro='Y', symbi='N'),
        methods=filex.SCMethods(infil='S'),
        management=filex.SCManagement(irrig='N', ferti='R', resid='N', harvs='M'),
        outputs=outputs_config
    )

    # 4. Run the Simulation
    # ---------------------
    dssat_env = DSSAT(workspace)
    
    print("\nRunning DSSAT simulation for the control treatment...")
    # The run_treatment function returns the content of Summary.OUT
    summary_results = dssat_env.run_treatment(
        field=field,
        cultivar=cultivar,
        planting=planting,
        initial_conditions=initial_conditions,
        simulation_controls=simulation_controls
    )

    # 5. Retrieve and Display Results
    # -------------------------------
    print("\nSimulation complete. Retrieving daily output data...")
    
    # Access the parsed output tables
    plant_gro_df = dssat_env.output_tables.get('PlantGro')
    soil_wat_df = dssat_env.output_tables.get('SoilWat')
    soil_ni_df = dssat_env.output_tables.get('SoilNi')

    print("\n--- Daily Plant Growth (PlantGro.OUT) ---")
    if plant_gro_df is not None and not plant_gro_df.empty:
        print(plant_gro_df.head())
    else:
        print("PlantGro.OUT was not generated or is empty.")

    print("\n--- Daily Soil Water (SoilWat.OUT) ---")
    if soil_wat_df is not None and not soil_wat_df.empty:
        print(soil_wat_df.head())
    else:
        print("SoilWat.OUT was not generated or is empty.")
        
    print("\n--- Daily Soil Nitrogen (SoilNi.OUT) ---")
    if soil_ni_df is not None and not soil_ni_df.empty:
        print(soil_ni_df.head())
    else:
        print("SoilNi.OUT was not generated or is empty.")

if __name__ == "__main__":
    setup_simulation_environment()

DSSAT workspace created at: d:\Final-Year-Project\dssat_simulation_workspace
Successfully downloaded SOIL.SOL
Successfully downloaded ITHY8001.WTH
Successfully downloaded ITHY8101.WTH
d:\Final-Year-Project\dssat_simulation_workspace created.

Running DSSAT simulation for the control treatment...

RUN    TRT FLO MAT TOPWT HARWT  RAIN  TIRR   CET  PESW  TNUP  TNLF   TSON TSOC
           dap dap kg/ha kg/ha    mm    mm    mm    mm kg/ha kg/ha  kg/ha t/ha
  1 SG   1  67 109 12726  5446   592     0   351    93   112    29  13372   95

Simulation complete. Retrieving daily output data...

--- Daily Plant Growth (PlantGro.OUT) ---
           @YEAR  DOY  DAS  DAP  L#SD  GSTD  LAID  LWAD  SWAD  GWAD  ...  \
1980-06-17  1980  169    1    0   0.0     0   0.0     0     0     0  ...   
1980-06-18  1980  170    2    1   0.0     0   0.0     0     0     0  ...   
1980-06-19  1980  171    3    2   0.0     0   0.0     0     0     0  ...   
1980-06-20  1980  172    4    3   0.0     0   0.0     0     0   

In [10]:
import os
import urllib.request
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import subprocess
import sys

def install_package(package):
    """Installs a package using pip, trying 'python3' and then 'python'."""
    try:
        print("Attempting to install package using 'python3'...")
        subprocess.check_call(["python3", "-m", "pip", "install", package])
        print(f"Successfully installed {package} with 'python3'")
    except FileNotFoundError:
        print("'python3' not found. Trying 'python' instead...")
        try:
            subprocess.check_call(["python", "-m", "pip", "install", package])
            print(f"Successfully installed {package} with 'python'")
        except FileNotFoundError:
            print("Error: Neither 'python3' nor 'python' command was found in the environment's PATH.")
            raise
        except subprocess.CalledProcessError as e:
            print(f"Error installing {package} with 'python': {e}")
            raise
    except subprocess.CalledProcessError as e:
        print(f"Error installing {package} with 'python3': {e}")
        raise


def download_file(url, target_path):
    """Downloads a file from a URL and saves it locally."""
    try:
        with urllib.request.urlopen(url) as response, open(target_path, 'wb') as out_file:
            data = response.read()
            out_file.write(data)
        print(f"Successfully downloaded {os.path.basename(target_path)}")
    except Exception as e:
        print(f"Error downloading {url}: {e}")
        raise

def setup_and_visualize_simulation():
    """
    Creates a robust, self-contained environment for a DSSAT simulation,
    runs the simulation, and generates visualizations of the results.
    """
    # First, ensure DSSATTools is installed
    install_package('DSSATTools')
    
    # Now import from the newly installed package
    from DSSATTools import (
        crop,
        WeatherStation,
        SoilProfile,
        filex,
        DSSAT
    )
    from DSSATTools.filex import IrrigationEvent

    # 1. Setup Workspace
    workspace = os.path.join(os.getcwd(), 'dssat_simulation_workspace')
    os.makedirs(workspace, exist_ok=True)
    print(f"DSSAT workspace created at: {workspace}")

    # 2. Data Acquisition and Processing
    soil_url = 'https://raw.githubusercontent.com/DSSAT/dssat-csm-data/develop/Soil/SOIL.SOL'
    soil_path = os.path.join(workspace, 'SOIL.SOL')
    download_file(soil_url, soil_path)

    weather_files_to_download = {
        'ITHY8001.WTH': 'https://raw.githubusercontent.com/DSSAT/dssat-csm-data/develop/Weather/ITHY8001.WTH',
        'ITHY8101.WTH': 'https://raw.githubusercontent.com/DSSAT/dssat-csm-data/develop/Weather/ITHY8101.WTH'
    }
    weather_dfs = []
    weather_col_names = ['date', 'srad', 'tmax', 'tmin', 'rain']
    weather_col_specs = [(0, 5), (6, 11), (12, 17), (18, 23), (24, 29)]

    for filename, url in weather_files_to_download.items():
        path = os.path.join(workspace, filename)
        download_file(url, path)
        df = pd.read_fwf(path, colspecs=weather_col_specs, names=weather_col_names, skiprows=4)
        weather_dfs.append(df)

    df_weather = pd.concat(weather_dfs, ignore_index=True)
    df_weather.dropna(subset=['date'], inplace=True)
    df_weather['date'] = pd.to_datetime(df_weather['date'], format='%y%j')
    df_weather.drop_duplicates(subset='date', keep='first', inplace=True)
    df_weather.sort_values(by='date', inplace=True)

    # 3. Create DSSATTools Objects
    weather_station = WeatherStation(
        insi='ITHY', lat=17.53, long=78.27, elev=0,
        tav=25.8, amp=11.8, refht=2.0, wndht=3.0,
        table=df_weather
    )

    soil = SoilProfile.from_file('IBSG910085', soil_path)
    cultivar = crop.Sorghum('IB0026')

    field = filex.Field(
        id_field='ITHY0001', wsta=weather_station, id_soil=soil
    )

    initial_conditions = filex.InitialConditions(
        pcr='SG', icdat=datetime(1980, 7, 3), icrt=500, icrn=1, icre=1,
        icres=1300, icren=0.5, icrip=100, icrid=10,
        table=pd.DataFrame([
            (10, .06, 2.5, 1.8), (22, .06, 2.5, 1.8),
            (52, .195, 3., 4.5), (82, .21, 3.5, 5.0),
            (112, 0.2, 2., 2.0), (142, 0.2, 1., 0.7),
            (172, 0.2, 1., 0.6),
        ], columns=['icbl', 'sh2o', 'snh4', 'sno3'])
    )

    planting = filex.Planting(
        pdate=datetime(1980, 6, 17), ppop=18, ppoe=18, plme='S',
        plds='R', plrs=45, pldp=5
    )
    
    irrigation = filex.Irrigation(
        table=[
            IrrigationEvent(idate=datetime(1980, 7, 15), irop='IR001', irval=20),
            IrrigationEvent(idate=datetime(1980, 7, 30), irop='IR001', irval=20),
            IrrigationEvent(idate=datetime(1980, 8, 15), irop='IR001', irval=25),
        ]
    )

    outputs_config = filex.SCOutputs(   
        fname='N', ovvew='Y', sumry='Y',
        grout='Y', waout='Y', niout='Y',
        vbose='Y'
    )

    simulation_controls = filex.SimulationControls(
        general=filex.SCGeneral(sdate=datetime(1980, 6, 17)),
        options=filex.SCOptions(water='Y', nitro='Y', symbi='N'),
        methods=filex.SCMethods(infil='S'),
        management=filex.SCManagement(irrig='R', ferti='R', resid='N', harvs='M'),
        outputs=outputs_config
    )

    # 4. Run the Simulation
    dssat_env = DSSAT(workspace)
    print("\nRunning DSSAT simulation with irrigation...")
    dssat_env.run_treatment(
        field=field, cultivar=cultivar, planting=planting,
        initial_conditions=initial_conditions,
        simulation_controls=simulation_controls,
        irrigation=irrigation
    )

    # 5. Retrieve and Visualize Results
    print("\nSimulation complete. Generating visualizations...")
    
    plant_gro_df = dssat_env.output_tables.get('PlantGro')
    soil_wat_df = dssat_env.output_tables.get('SoilWat')
    
    if plant_gro_df is not None and not plant_gro_df.empty:
        plt.figure(figsize=(10, 6))
        plt.plot(plant_gro_df.index, plant_gro_df['GWAD'])
        plt.title('Daily Grain Growth')
        plt.xlabel('Date')
        plt.ylabel('Grain Weight (kg/ha)')
        plt.grid(True)
        plt.tight_layout()
        grain_growth_path = os.path.join(workspace, 'grain_growth.png')
        plt.savefig(grain_growth_path)
        print(f"Saved grain growth plot to: {grain_growth_path}")
        plt.close()

    if plant_gro_df is not None and not plant_gro_df.empty:
        plt.figure(figsize=(10, 6))
        plt.plot(plant_gro_df.index, plant_gro_df['LAID'])
        plt.title('Daily Leaf Area Index (LAI)')
        plt.xlabel('Date')
        plt.ylabel('Leaf Area Index')
        plt.grid(True)
        plt.tight_layout()
        lai_growth_path = os.path.join(workspace, 'lai_growth.png')
        plt.savefig(lai_growth_path)
        print(f"Saved LAI plot to: {lai_growth_path}")
        plt.close()
        
    if soil_wat_df is not None and not soil_wat_df.empty and 'IRR' in soil_wat_df.columns:
        irrigation_events = soil_wat_df[soil_wat_df['IRR'] > 0]
        plt.figure(figsize=(10, 6))
        plt.bar(irrigation_events.index, irrigation_events['IRR'], width=1.5, label='Irrigation')
        plt.title('Daily Irrigation Events')
        plt.xlabel('Date')
        plt.ylabel('Irrigation (mm)')
        plt.grid(axis='y')
        plt.tight_layout()
        irrigation_path = os.path.join(workspace, 'irrigation.png')
        plt.savefig(irrigation_path)
        print(f"Saved irrigation plot to: {irrigation_path}")
        plt.close()

if __name__ == "__main__":
    setup_and_visualize_simulation()

Attempting to install package using 'python3'...
DSSAT workspace created at: d:\Final-Year-Project\dssat_simulation_workspace
Successfully downloaded SOIL.SOL
Successfully downloaded ITHY8001.WTH
Successfully downloaded ITHY8101.WTH
d:\Final-Year-Project\dssat_simulation_workspace created.

Running DSSAT simulation with irrigation...

RUN    TRT FLO MAT TOPWT HARWT  RAIN  TIRR   CET  PESW  TNUP  TNLF   TSON TSOC
           dap dap kg/ha kg/ha    mm    mm    mm    mm kg/ha kg/ha  kg/ha t/ha
  1 SG   1  67 109 12690  5098   592    65   361    94   108    25  13371   95

Simulation complete. Generating visualizations...
Saved grain growth plot to: d:\Final-Year-Project\dssat_simulation_workspace\grain_growth.png
Saved LAI plot to: d:\Final-Year-Project\dssat_simulation_workspace\lai_growth.png


In [14]:
import os
import urllib.request
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import subprocess
import sys

def install_package(package):
    """Installs a package using pip, trying 'python3' and then 'python'."""
    try:
        print("Attempting to install package using 'python3'...")
        subprocess.check_call(["python3", "-m", "pip", "install", package])
        print(f"Successfully installed {package} with 'python3'")
    except FileNotFoundError:
        print("'python3' not found. Trying 'python' instead...")
        try:
            subprocess.check_call(["python", "-m", "pip", "install", package])
            print(f"Successfully installed {package} with 'python'")
        except FileNotFoundError:
            print("Error: Neither 'python3' nor 'python' command was found in the environment's PATH.")
            raise
        except subprocess.CalledProcessError as e:
            print(f"Error installing {package} with 'python': {e}")
            raise
    except subprocess.CalledProcessError as e:
        print(f"Error installing {package} with 'python3': {e}")
        raise


def download_file(url, target_path):
    """Downloads a file from a URL and saves it locally."""
    try:
        with urllib.request.urlopen(url) as response, open(target_path, 'wb') as out_file:
            data = response.read()
            out_file.write(data)
        print(f"Successfully downloaded {os.path.basename(target_path)}")
    except Exception as e:
        print(f"Error downloading {url}: {e}")
        raise

def run_all_analyses_and_visualizations():
    """
    Sets up a DSSAT environment, runs multiple analysis scenarios,
    and generates all relevant visualizations.
    """
    # First, ensure DSSATTools is installed
    install_package('DSSATTools')
    
    # Now import from the newly installed package
    from DSSATTools import (
        crop,
        WeatherStation,
        SoilProfile,
        filex,
        DSSAT
    )
    from DSSATTools.filex import IrrigationEvent, FertilizerEvent

    # ===================================================================
    # SECTION 1: BASIC SETUP AND DATA ACQUISITION
    # ===================================================================
    print("\n--- SECTION 1: Initializing Workspace and Data ---")
    workspace = os.path.join(os.getcwd(), 'dssat_simulation_workspace')
    os.makedirs(workspace, exist_ok=True)
    print(f"DSSAT workspace created at: {workspace}")

    soil_path = os.path.join(workspace, 'SOIL.SOL')
    download_file('https://raw.githubusercontent.com/DSSAT/dssat-csm-data/develop/Soil/SOIL.SOL', soil_path)

    weather_dfs = []
    for year in ['80', '81']:
        filename = f'ITHY{year}01.WTH'
        url = f'https://raw.githubusercontent.com/DSSAT/dssat-csm-data/develop/Weather/{filename}'
        path = os.path.join(workspace, filename)
        download_file(url, path)
        df = pd.read_fwf(path, colspecs=[(0, 5), (6, 11), (12, 17), (18, 23), (24, 29)], 
                         names=['date', 'srad', 'tmax', 'tmin', 'rain'], skiprows=4)
        weather_dfs.append(df)

    df_weather = pd.concat(weather_dfs, ignore_index=True)
    df_weather.dropna(subset=['date'], inplace=True)
    df_weather['date'] = pd.to_datetime(df_weather['date'], format='%y%j')
    df_weather.drop_duplicates(subset='date', keep='first', inplace=True)
    df_weather.sort_values(by='date', inplace=True)

    # Create common DSSATTools objects
    weather_station = WeatherStation(insi='ITHY', lat=17.53, long=78.27, elev=0, tav=25.8, amp=11.8, table=df_weather)
    soil = SoilProfile.from_file('IBSG910085', soil_path)
    dssat_env = DSSAT(workspace)

    # Define common management objects
    initial_conditions = filex.InitialConditions(pcr='SG', icdat=datetime(1980, 7, 3), icres=1300)
    planting = filex.Planting(pdate=datetime(1980, 6, 17), ppop=18, plds='R', plrs=45, pldp=5)
    
    # ===================================================================
    # SECTION 2: TREATMENT COMPARISON (Rainfed vs. Irrigated)
    # ===================================================================
    print("\n--- SECTION 2: Running Treatment Comparison (Rainfed vs. Irrigated) ---")
    
    field = filex.Field(id_field='ITHY0001', wsta=weather_station, id_soil=soil)
    
    outputs_config = filex.SCOutputs(grout='Y', waout='Y')
    
    base_sim_controls_args = {
        'general': filex.SCGeneral(sdate=datetime(1980, 6, 17)),
        'options': filex.SCOptions(water='Y', nitro='Y', symbi='N'),
        'outputs': outputs_config
    }
    
    # Run Rainfed Treatment
    simulation_controls_rainfed = filex.SimulationControls(**base_sim_controls_args)
    simulation_controls_rainfed['management'] = filex.SCManagement(irrig='N', ferti='N', harvs='M')
    dssat_env.run_treatment(
        field=field,
        cultivar=crop.Sorghum('IB0026'),
        planting=planting,
        initial_conditions=initial_conditions,
        simulation_controls=simulation_controls_rainfed
    )
    rainfed_growth = dssat_env.output_tables.get('PlantGro').copy()

    # Run Irrigated Treatment
    irrigation = filex.Irrigation(table=[
        IrrigationEvent(idate=datetime(1980, 7, 15), irop='IR001', irval=20),
        IrrigationEvent(idate=datetime(1980, 8, 15), irop='IR001', irval=25),
    ])
    simulation_controls_irrigated = filex.SimulationControls(**base_sim_controls_args)
    simulation_controls_irrigated['management'] = filex.SCManagement(irrig='R', ferti='N', harvs='M')
    dssat_env.run_treatment(
        field=field,
        cultivar=crop.Sorghum('IB0026'),
        planting=planting,
        initial_conditions=initial_conditions,
        simulation_controls=simulation_controls_irrigated,
        irrigation=irrigation
    )
    irrigated_growth = dssat_env.output_tables.get('PlantGro').copy()
    irrigated_water = dssat_env.output_tables.get('SoilWat').copy()

    # Create Comparison Plot
    plt.figure(figsize=(10, 6))
    plt.plot(rainfed_growth.index, rainfed_growth['GWAD'], label='Rainfed')
    plt.plot(irrigated_growth.index, irrigated_growth['GWAD'], label='Irrigated', linestyle='--')
    plt.title('Treatment Comparison: Grain Growth')
    plt.xlabel('Date')
    plt.ylabel('Grain Weight (kg/ha)')
    plt.legend()
    plt.grid(True)
    comp_path = os.path.join(workspace, 'treatment_comparison_growth.png')
    plt.savefig(comp_path)
    print(f"Saved treatment comparison plot to: {comp_path}")
    plt.close()
    
    # ===================================================================
    # SECTION 3: PARAMETER SENSITIVITY (Varying 'p1' coefficient)
    # ===================================================================
    print("\n--- SECTION 3: Running Parameter Sensitivity Analysis (P1 Coefficient) ---")
    
    p1_values = [350, 380, 410, 440, 470]
    sensitivity_results = []
    
    cultivar_sens = crop.Sorghum('IB0026')
    simulation_controls_sens = filex.SimulationControls(**base_sim_controls_args)
    simulation_controls_sens['management'] = filex.SCManagement(irrig='N', ferti='N', harvs='M')
    
    for p1 in p1_values:
        cultivar_sens['p1'] = p1
        print(f"Running simulation for p1 = {p1}...")
        summary = dssat_env.run_treatment(
            field=field,
            cultivar=cultivar_sens,
            planting=planting,
            initial_conditions=initial_conditions,
            simulation_controls=simulation_controls_sens
        )
        summary['p1'] = p1
        sensitivity_results.append(summary)
        
    results_df = pd.DataFrame(sensitivity_results)

    # Create Sensitivity Plots
    plt.figure(figsize=(10, 6))
    plt.plot(results_df['p1'], results_df['harwt'], marker='o')
    plt.title('Sensitivity Analysis: Yield vs. P1 Coefficient')
    plt.xlabel('P1 Value (GDD)')
    plt.ylabel('Grain Weight at Harvest (kg/ha)')
    plt.grid(True)
    sens_yield_path = os.path.join(workspace, 'sensitivity_yield_vs_p1.png')
    plt.savefig(sens_yield_path)
    print(f"Saved sensitivity plot (Yield vs P1) to: {sens_yield_path}")
    plt.close()

    plt.figure(figsize=(10, 6))
    plt.plot(results_df['p1'], results_df['mat'], marker='o')
    plt.title('Sensitivity Analysis: Maturity vs. P1 Coefficient')
    plt.xlabel('P1 Value (GDD)')
    plt.ylabel('Days to Maturity (dap)')
    plt.grid(True)
    sens_mat_path = os.path.join(workspace, 'sensitivity_maturity_vs_p1.png')
    plt.savefig(sens_mat_path)
    print(f"Saved sensitivity plot (Maturity vs P1) to: {sens_mat_path}")
    plt.close()

    # ===================================================================
    # SECTION 4: DETAILED DAILY TIME-SERIES PLOTS
    # ===================================================================
    print("\n--- SECTION 4: Generating Detailed Daily Plots ---")

    # Re-use results from the irrigated run in Section 2
    plt.figure(figsize=(10, 6))
    plt.plot(irrigated_growth.index, irrigated_growth['LAID'])
    plt.title('Daily Leaf Area Index (Irrigated Treatment)')
    plt.xlabel('Date')
    plt.ylabel('Leaf Area Index (LAI)')
    plt.grid(True)
    lai_path = os.path.join(workspace, 'daily_lai.png')
    plt.savefig(lai_path)
    print(f"Saved LAI plot to: {lai_path}")
    plt.close()

    # FIXED: Corrected column name from 'IRR' to 'IRRIG'
    if irrigated_water is not None and not irrigated_water.empty and 'IRRIG' in irrigated_water.columns:
        irrigation_events = irrigated_water[irrigated_water['IRRIG'] > 0]
        plt.figure(figsize=(10, 6))
        # FIXED: Corrected column name from 'IRR' to 'IRRIG'
        plt.bar(irrigation_events.index, irrigation_events['IRRIG'], width=1.5)
        plt.title('Daily Irrigation Events')
        plt.xlabel('Date')
        plt.ylabel('Irrigation (mm)')
        plt.grid(axis='y')
        irrigation_path = os.path.join(workspace, 'daily_irrigation.png')
        plt.savefig(irrigation_path)
        print(f"Saved irrigation plot to: {irrigation_path}")
        plt.close()

if __name__ == "__main__":
    run_all_analyses_and_visualizations()

Attempting to install package using 'python3'...

--- SECTION 1: Initializing Workspace and Data ---
DSSAT workspace created at: d:\Final-Year-Project\dssat_simulation_workspace
Successfully downloaded SOIL.SOL
Successfully downloaded ITHY8001.WTH
Successfully downloaded ITHY8101.WTH
d:\Final-Year-Project\dssat_simulation_workspace created.

--- SECTION 2: Running Treatment Comparison (Rainfed vs. Irrigated) ---

RUN    TRT FLO MAT TOPWT HARWT  RAIN  TIRR   CET  PESW  TNUP  TNLF   TSON TSOC
           dap dap kg/ha kg/ha    mm    mm    mm    mm kg/ha kg/ha  kg/ha t/ha
  1 SG   1  67 109  1985   612   592     0   307   130    15     8  13377   95

RUN    TRT FLO MAT TOPWT HARWT  RAIN  TIRR   CET  PESW  TNUP  TNLF   TSON TSOC
           dap dap kg/ha kg/ha    mm    mm    mm    mm kg/ha kg/ha  kg/ha t/ha
  1 SG   1  67 109  1839   565   592    45   312   130    14     8  13377   95
Saved treatment comparison plot to: d:\Final-Year-Project\dssat_simulation_workspace\treatment_comparison_gr

In [16]:
import os
import urllib.request
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import subprocess
import sys

def install_package(package):
    """Installs a package using pip, trying 'python3' and then 'python'."""
    try:
        print("Attempting to install package using 'python3'...")
        subprocess.check_call(["python3", "-m", "pip", "install", package])
        print(f"Successfully installed {package} with 'python3'")
    except FileNotFoundError:
        print("'python3' not found. Trying 'python' instead...")
        try:
            subprocess.check_call(["python", "-m", "pip", "install", package])
            print(f"Successfully installed {package} with 'python'")
        except FileNotFoundError:
            print("Error: Neither 'python3' nor 'python' command was found in the environment's PATH.")
            raise
        except subprocess.CalledProcessError as e:
            print(f"Error installing {package} with 'python': {e}")
            raise
    except subprocess.CalledProcessError as e:
        print(f"Error installing {package} with 'python3': {e}")
        raise


def download_file(url, target_path):
    """Downloads a file from a URL and saves it locally."""
    try:
        with urllib.request.urlopen(url) as response, open(target_path, 'wb') as out_file:
            data = response.read()
            out_file.write(data)
        print(f"Successfully downloaded {os.path.basename(target_path)}")
    except Exception as e:
        print(f"Error downloading {url}: {e}")
        raise

def run_all_analyses_and_visualizations():
    """
    Sets up a DSSAT environment, runs multiple analysis scenarios,
    and generates all relevant visualizations.
    """
    # First, ensure DSSATTools is installed
    install_package('DSSATTools')
    
    # Now import from the newly installed package
    from DSSATTools import (
        crop,
        WeatherStation,
        SoilProfile,
        filex,
        DSSAT
    )
    from DSSATTools.filex import IrrigationEvent, FertilizerEvent

    # ===================================================================
    # SECTION 1: BASIC SETUP AND DATA ACQUISITION
    # ===================================================================
    print("\n--- SECTION 1: Initializing Workspace and Data ---")
    workspace = os.path.join(os.getcwd(), 'dssat_simulation_workspace')
    os.makedirs(workspace, exist_ok=True)
    print(f"DSSAT workspace created at: {workspace}")

    soil_path = os.path.join(workspace, 'SOIL.SOL')
    download_file('https://raw.githubusercontent.com/DSSAT/dssat-csm-data/develop/Soil/SOIL.SOL', soil_path)

    weather_dfs = []
    for year in ['80', '81']:
        filename = f'ITHY{year}01.WTH'
        url = f'https://raw.githubusercontent.com/DSSAT/dssat-csm-data/develop/Weather/{filename}'
        path = os.path.join(workspace, filename)
        download_file(url, path)
        df = pd.read_fwf(path, colspecs=[(0, 5), (6, 11), (12, 17), (18, 23), (24, 29)], 
                         names=['date', 'srad', 'tmax', 'tmin', 'rain'], skiprows=4)
        weather_dfs.append(df)

    df_weather = pd.concat(weather_dfs, ignore_index=True)
    df_weather.dropna(subset=['date'], inplace=True)
    df_weather['date'] = pd.to_datetime(df_weather['date'], format='%y%j')
    df_weather.drop_duplicates(subset='date', keep='first', inplace=True)
    df_weather.sort_values(by='date', inplace=True)

    # Create common DSSATTools objects
    weather_station = WeatherStation(insi='ITHY', lat=17.53, long=78.27, elev=0, tav=25.8, amp=11.8, table=df_weather)
    soil = SoilProfile.from_file('IBSG910085', soil_path)
    dssat_env = DSSAT(workspace)

    # Define common management objects
    initial_conditions = filex.InitialConditions(pcr='SG', icdat=datetime(1980, 7, 3), icres=1300)
    planting = filex.Planting(pdate=datetime(1980, 6, 17), ppop=18, plds='R', plrs=45, pldp=5)
    
    # ===================================================================
    # SECTION 2: TREATMENT COMPARISON (Rainfed vs. Irrigated)
    # ===================================================================
    print("\n--- SECTION 2: Running Treatment Comparison (Rainfed vs. Irrigated) ---")
    
    field = filex.Field(id_field='ITHY0001', wsta=weather_station, id_soil=soil)
    outputs_config = filex.SCOutputs(grout='Y', waout='Y')
    
    # Run Rainfed Treatment
    # FIXED: Define the complete SimulationControls object for this specific treatment
    simulation_controls_rainfed = filex.SimulationControls(
        general=filex.SCGeneral(sdate=datetime(1980, 6, 17)),
        options=filex.SCOptions(water='Y', nitro='Y', symbi='N'),
        management=filex.SCManagement(irrig='N', ferti='N', harvs='M'),
        outputs=outputs_config
    )
    dssat_env.run_treatment(
        field=field,
        cultivar=crop.Sorghum('IB0026'),
        planting=planting,
        initial_conditions=initial_conditions,
        simulation_controls=simulation_controls_rainfed
    )
    rainfed_growth = dssat_env.output_tables.get('PlantGro').copy()

    # Run Irrigated Treatment
    irrigation = filex.Irrigation(table=[
        IrrigationEvent(idate=datetime(1980, 7, 15), irop='IR001', irval=20),
        IrrigationEvent(idate=datetime(1980, 8, 15), irop='IR001', irval=25),
    ])
    # FIXED: Define the complete SimulationControls object for this specific treatment
    simulation_controls_irrigated = filex.SimulationControls(
        general=filex.SCGeneral(sdate=datetime(1980, 6, 17)),
        options=filex.SCOptions(water='Y', nitro='Y', symbi='N'),
        management=filex.SCManagement(irrig='R', ferti='N', harvs='M'),
        outputs=outputs_config
    )
    dssat_env.run_treatment(
        field=field,
        cultivar=crop.Sorghum('IB0026'),
        planting=planting,
        initial_conditions=initial_conditions,
        simulation_controls=simulation_controls_irrigated,
        irrigation=irrigation
    )
    irrigated_growth = dssat_env.output_tables.get('PlantGro').copy()
    irrigated_water = dssat_env.output_tables.get('SoilWat').copy()

    # Create Comparison Plot
    plt.figure(figsize=(10, 6))
    plt.plot(rainfed_growth.index, rainfed_growth['GWAD'], label='Rainfed')
    plt.plot(irrigated_growth.index, irrigated_growth['GWAD'], label='Irrigated', linestyle='--')
    plt.title('Treatment Comparison: Grain Growth')
    plt.xlabel('Date')
    plt.ylabel('Grain Weight (kg/ha)')
    plt.legend()
    plt.grid(True)
    comp_path = os.path.join(workspace, 'treatment_comparison_growth.png')
    plt.savefig(comp_path)
    print(f"Saved treatment comparison plot to: {comp_path}")
    plt.close()
    
    # ===================================================================
    # SECTION 3: PARAMETER SENSITIVITY (Varying 'p1' coefficient)
    # ===================================================================
    print("\n--- SECTION 3: Running Parameter Sensitivity Analysis (P1 Coefficient) ---")
    
    p1_values = [350, 380, 410, 440, 470]
    sensitivity_results = []
    
    cultivar_sens = crop.Sorghum('IB0026')
    # FIXED: Define the complete SimulationControls object for this analysis
    simulation_controls_sens = filex.SimulationControls(
        general=filex.SCGeneral(sdate=datetime(1980, 6, 17)),
        options=filex.SCOptions(water='Y', nitro='Y', symbi='N'),
        management=filex.SCManagement(irrig='N', ferti='N', harvs='M'),
        outputs=filex.SCOutputs(sumry='Y') # Only need summary output for this
    )
    
    for p1 in p1_values:
        cultivar_sens['p1'] = p1
        print(f"Running simulation for p1 = {p1}...")
        summary = dssat_env.run_treatment(
            field=field,
            cultivar=cultivar_sens,
            planting=planting,
            initial_conditions=initial_conditions,
            simulation_controls=simulation_controls_sens
        )
        summary['p1'] = p1
        sensitivity_results.append(summary)
        
    results_df = pd.DataFrame(sensitivity_results)

    # Create Sensitivity Plots
    plt.figure(figsize=(10, 6))
    plt.plot(results_df['p1'], results_df['harwt'], marker='o')
    plt.title('Sensitivity Analysis: Yield vs. P1 Coefficient')
    plt.xlabel('P1 Value (GDD)')
    plt.ylabel('Grain Weight at Harvest (kg/ha)')
    plt.grid(True)
    sens_yield_path = os.path.join(workspace, 'sensitivity_yield_vs_p1.png')
    plt.savefig(sens_yield_path)
    print(f"Saved sensitivity plot (Yield vs P1) to: {sens_yield_path}")
    plt.close()

    plt.figure(figsize=(10, 6))
    plt.plot(results_df['p1'], results_df['mat'], marker='o')
    plt.title('Sensitivity Analysis: Maturity vs. P1 Coefficient')
    plt.xlabel('P1 Value (GDD)')
    plt.ylabel('Days to Maturity (dap)')
    plt.grid(True)
    sens_mat_path = os.path.join(workspace, 'sensitivity_maturity_vs_p1.png')
    plt.savefig(sens_mat_path)
    print(f"Saved sensitivity plot (Maturity vs P1) to: {sens_mat_path}")
    plt.close()

    # ===================================================================
    # SECTION 4: DETAILED DAILY TIME-SERIES PLOTS
    # ===================================================================
    print("\n--- SECTION 4: Generating Detailed Daily Plots ---")

    # Re-use results from the irrigated run in Section 2
    plt.figure(figsize=(10, 6))
    plt.plot(irrigated_growth.index, irrigated_growth['LAID'])
    plt.title('Daily Leaf Area Index (Irrigated Treatment)')
    plt.xlabel('Date')
    plt.ylabel('Leaf Area Index (LAI)')
    plt.grid(True)
    lai_path = os.path.join(workspace, 'daily_lai.png')
    plt.savefig(lai_path)
    print(f"Saved LAI plot to: {lai_path}")
    plt.close()

    if irrigated_water is not None and not irrigated_water.empty and 'IRRIG' in irrigated_water.columns:
        irrigation_events = irrigated_water[irrigated_water['IRRIG'] > 0]
        plt.figure(figsize=(10, 6))
        plt.bar(irrigation_events.index, irrigation_events['IRRIG'], width=1.5)
        plt.title('Daily Irrigation Events')
        plt.xlabel('Date')
        plt.ylabel('Irrigation (mm)')
        plt.grid(axis='y')
        irrigation_path = os.path.join(workspace, 'daily_irrigation.png')
        plt.savefig(irrigation_path)
        print(f"Saved irrigation plot to: {irrigation_path}")
        plt.close()
    
    # Plot for Daily Yield (Grain Growth Rate)
    irrigated_growth['daily_yield'] = irrigated_growth['GWAD'].diff().fillna(0).clip(lower=0)
    plt.figure(figsize=(10, 6))
    plt.plot(irrigated_growth['DAP'], irrigated_growth['daily_yield'])
    plt.title('Daily Yield (Grain Growth Rate)')
    plt.xlabel('Days After Planting (DAP)')
    plt.ylabel('Daily Increase in Grain Weight (kg/ha/day)')
    plt.grid(True)
    daily_yield_path = os.path.join(workspace, 'daily_yield.png')
    plt.savefig(daily_yield_path)
    print(f"Saved daily yield plot to: {daily_yield_path}")
    plt.close()

if __name__ == "__main__":
    run_all_analyses_and_visualizations()

Attempting to install package using 'python3'...

--- SECTION 1: Initializing Workspace and Data ---
DSSAT workspace created at: d:\Final-Year-Project\dssat_simulation_workspace
Successfully downloaded SOIL.SOL
Successfully downloaded ITHY8001.WTH
Successfully downloaded ITHY8101.WTH
d:\Final-Year-Project\dssat_simulation_workspace created.

--- SECTION 2: Running Treatment Comparison (Rainfed vs. Irrigated) ---

RUN    TRT FLO MAT TOPWT HARWT  RAIN  TIRR   CET  PESW  TNUP  TNLF   TSON TSOC
           dap dap kg/ha kg/ha    mm    mm    mm    mm kg/ha kg/ha  kg/ha t/ha
  1 SG   1  67 109  1985   612   592     0   307   130    15     8  13377   95

RUN    TRT FLO MAT TOPWT HARWT  RAIN  TIRR   CET  PESW  TNUP  TNLF   TSON TSOC
           dap dap kg/ha kg/ha    mm    mm    mm    mm kg/ha kg/ha  kg/ha t/ha
  1 SG   1  67 109  1839   565   592    45   312   130    14     8  13377   95
Saved treatment comparison plot to: d:\Final-Year-Project\dssat_simulation_workspace\treatment_comparison_gr

In [19]:
import os
import urllib.request
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import subprocess
import sys

def install_package(package):
    """Installs a package using pip, trying 'python3' and then 'python'."""
    try:
        print("Attempting to install package using 'python3'...")
        subprocess.check_call(["python3", "-m", "pip", "install", package])
        print(f"Successfully installed {package} with 'python3'")
    except FileNotFoundError:
        print("'python3' not found. Trying 'python' instead...")
        try:
            subprocess.check_call(["python", "-m", "pip", "install", package])
            print(f"Successfully installed {package} with 'python'")
        except FileNotFoundError:
            print("Error: Neither 'python3' nor 'python' command was found in the environment's PATH.")
            raise
        except subprocess.CalledProcessError as e:
            print(f"Error installing {package} with 'python': {e}")
            raise
    except subprocess.CalledProcessError as e:
        print(f"Error installing {package} with 'python3': {e}")
        raise


def download_file(url, target_path):
    """Downloads a file from a URL and saves it locally."""
    try:
        with urllib.request.urlopen(url) as response, open(target_path, 'wb') as out_file:
            data = response.read()
            out_file.write(data)
        print(f"Successfully downloaded {os.path.basename(target_path)}")
    except Exception as e:
        print(f"Error downloading {url}: {e}")
        raise

def run_all_analyses_and_visualizations():
    """
    Sets up a DSSAT environment, runs multiple analysis scenarios,
    and generates all relevant visualizations.
    """
    # First, ensure DSSATTools is installed
    install_package('DSSATTools')
    
    # Now import from the newly installed package
    from DSSATTools import (
        crop,
        WeatherStation,
        SoilProfile,
        filex,
        DSSAT
    )
    from DSSATTools.filex import IrrigationEvent, FertilizerEvent

    # ===================================================================
    # SECTION 1: BASIC SETUP AND DATA ACQUISITION
    # ===================================================================
    print("\n--- SECTION 1: Initializing Workspace and Data ---")
    workspace = os.path.join(os.getcwd(), 'dssat_simulation_workspace')
    os.makedirs(workspace, exist_ok=True)
    print(f"DSSAT workspace created at: {workspace}")

    soil_path = os.path.join(workspace, 'SOIL.SOL')
    download_file('https://raw.githubusercontent.com/DSSAT/dssat-csm-data/develop/Soil/SOIL.SOL', soil_path)

    weather_dfs = []
    for year in ['80', '81']:
        filename = f'ITHY{year}01.WTH'
        url = f'https://raw.githubusercontent.com/DSSAT/dssat-csm-data/develop/Weather/{filename}'
        path = os.path.join(workspace, filename)
        download_file(url, path)
        df = pd.read_fwf(path, colspecs=[(0, 5), (6, 11), (12, 17), (18, 23), (24, 29)], 
                         names=['date', 'srad', 'tmax', 'tmin', 'rain'], skiprows=4)
        weather_dfs.append(df)

    df_weather = pd.concat(weather_dfs, ignore_index=True)
    df_weather.dropna(subset=['date'], inplace=True)
    df_weather['date'] = pd.to_datetime(df_weather['date'], format='%y%j')
    df_weather.drop_duplicates(subset='date', keep='first', inplace=True)
    df_weather.sort_values(by='date', inplace=True)

    weather_station = WeatherStation(insi='ITHY', lat=17.53, long=78.27, elev=0, tav=25.8, amp=11.8, table=df_weather)
    soil = SoilProfile.from_file('IBSG910085', soil_path)
    dssat_env = DSSAT(workspace)

    initial_conditions = filex.InitialConditions(pcr='SG', icdat=datetime(1980, 7, 3), icres=1300)
    planting = filex.Planting(pdate=datetime(1980, 6, 17), ppop=18, plds='R', plrs=45, pldp=5)
    
    # ===================================================================
    # SECTION 2: TREATMENT COMPARISON (Rainfed vs. Irrigated)
    # ===================================================================
    print("\n--- SECTION 2: Running Treatment Comparison (Rainfed vs. Irrigated) ---")
    
    field = filex.Field(id_field='ITHY0001', wsta=weather_station, id_soil=soil)
    outputs_config = filex.SCOutputs(grout='Y', waout='Y', sumry='Y')
    
    # Run Rainfed Treatment
    simulation_controls_rainfed = filex.SimulationControls(
        general=filex.SCGeneral(sdate=datetime(1980, 6, 17)),
        options=filex.SCOptions(water='Y', nitro='Y', symbi='N'),
        management=filex.SCManagement(irrig='N', ferti='N', harvs='M'),
        outputs=outputs_config
    )
    rainfed_summary = dssat_env.run_treatment(
        field=field,
        cultivar=crop.Sorghum('IB0026'),
        planting=planting,
        initial_conditions=initial_conditions,
        simulation_controls=simulation_controls_rainfed
    )
    rainfed_growth = dssat_env.output_tables.get('PlantGro').copy()

    # Run Irrigated Treatment
    irrigation = filex.Irrigation(table=[
        IrrigationEvent(idate=datetime(1980, 7, 15), irop='IR001', irval=20),
        IrrigationEvent(idate=datetime(1980, 8, 15), irop='IR001', irval=25),
    ])
    simulation_controls_irrigated = filex.SimulationControls(
        general=filex.SCGeneral(sdate=datetime(1980, 6, 17)),
        options=filex.SCOptions(water='Y', nitro='Y', symbi='N'),
        management=filex.SCManagement(irrig='R', ferti='N', harvs='M'),
        outputs=outputs_config
    )
    irrigated_summary = dssat_env.run_treatment(
        field=field,
        cultivar=crop.Sorghum('IB0026'),
        planting=planting,
        initial_conditions=initial_conditions,
        simulation_controls=simulation_controls_irrigated,
        irrigation=irrigation
    )
    irrigated_growth = dssat_env.output_tables.get('PlantGro').copy()
    irrigated_water = dssat_env.output_tables.get('SoilWat').copy()

    # Create Comparison Plot with Harvest Day markers
    plt.figure(figsize=(10, 6))
    plt.plot(rainfed_growth.index, rainfed_growth['GWAD'], label='Rainfed')
    plt.plot(irrigated_growth.index, irrigated_growth['GWAD'], label='Irrigated', linestyle='--')
    
    # Add vertical lines for harvest days
    # FIXED: Convert the DSSATTools DateType to a standard Python datetime object
    pdate_dssattools = planting['pdate']
    planting_date = datetime(pdate_dssattools.year, pdate_dssattools.month, pdate_dssattools.day)
    
    rainfed_mat_dap = rainfed_summary['mat']
    irrigated_mat_dap = irrigated_summary['mat']
    rainfed_harvest_date = planting_date + timedelta(days=rainfed_mat_dap)
    irrigated_harvest_date = planting_date + timedelta(days=irrigated_mat_dap)
    
    plt.axvline(x=rainfed_harvest_date, color='red', linestyle=':', label=f'Rainfed Harvest ({rainfed_mat_dap} DAP)')
    plt.axvline(x=irrigated_harvest_date, color='blue', linestyle=':', label=f'Irrigated Harvest ({irrigated_mat_dap} DAP)')
    
    plt.title('Treatment Comparison: Grain Growth')
    plt.xlabel('Date')
    plt.ylabel('Grain Weight (kg/ha)')
    plt.legend()
    plt.grid(True)
    comp_path = os.path.join(workspace, 'treatment_comparison_growth.png')
    plt.savefig(comp_path)
    print(f"Saved treatment comparison plot to: {comp_path}")
    plt.close()
    
    # ===================================================================
    # SECTION 3: PARAMETER SENSITIVITY (Varying 'p1' coefficient)
    # ===================================================================
    print("\n--- SECTION 3: Running Parameter Sensitivity Analysis (P1 Coefficient) ---")
    
    p1_values = [350, 380, 410, 440, 470]
    sensitivity_results = []
    
    cultivar_sens = crop.Sorghum('IB0026')
    simulation_controls_sens = filex.SimulationControls(
        general=filex.SCGeneral(sdate=datetime(1980, 6, 17)),
        options=filex.SCOptions(water='Y', nitro='Y', symbi='N'),
        management=filex.SCManagement(irrig='N', ferti='N', harvs='M'),
        outputs=filex.SCOutputs(sumry='Y') 
    )
    
    for p1 in p1_values:
        cultivar_sens['p1'] = p1
        print(f"Running simulation for p1 = {p1}...")
        summary = dssat_env.run_treatment(
            field=field,
            cultivar=cultivar_sens,
            planting=planting,
            initial_conditions=initial_conditions,
            simulation_controls=simulation_controls_sens
        )
        summary['p1'] = p1
        sensitivity_results.append(summary)
        
    results_df = pd.DataFrame(sensitivity_results)

    # Create Sensitivity Plots
    plt.figure(figsize=(10, 6))
    plt.plot(results_df['p1'], results_df['harwt'], marker='o')
    plt.title('Sensitivity Analysis: Yield vs. P1 Coefficient')
    plt.xlabel('P1 Value (GDD)')
    plt.ylabel('Grain Weight at Harvest (kg/ha)')
    plt.grid(True)
    sens_yield_path = os.path.join(workspace, 'sensitivity_yield_vs_p1.png')
    plt.savefig(sens_yield_path)
    print(f"Saved sensitivity plot (Yield vs P1) to: {sens_yield_path}")
    plt.close()

    plt.figure(figsize=(10, 6))
    plt.plot(results_df['p1'], results_df['mat'], marker='o')
    plt.title('Sensitivity Analysis: Maturity vs. P1 Coefficient')
    plt.xlabel('P1 Value (GDD)')
    plt.ylabel('Days to Maturity (dap)')
    plt.grid(True)
    sens_mat_path = os.path.join(workspace, 'sensitivity_maturity_vs_p1.png')
    plt.savefig(sens_mat_path)
    print(f"Saved sensitivity plot (Maturity vs P1) to: {sens_mat_path}")
    plt.close()

    # ===================================================================
    # SECTION 4: DETAILED DAILY TIME-SERIES PLOTS
    # ===================================================================
    print("\n--- SECTION 4: Generating Detailed Daily Plots ---")

    # Re-use results from the irrigated run in Section 2
    plt.figure(figsize=(10, 6))
    plt.plot(irrigated_growth.index, irrigated_growth['LAID'])
    plt.title('Daily Leaf Area Index (Irrigated Treatment)')
    plt.xlabel('Date')
    plt.ylabel('Leaf Area Index (LAI)')
    plt.grid(True)
    plt.axvline(x=irrigated_harvest_date, color='red', linestyle='--', label=f'Harvest Day ({irrigated_mat_dap} DAP)')
    plt.legend()
    lai_path = os.path.join(workspace, 'daily_lai.png')
    plt.savefig(lai_path)
    print(f"Saved LAI plot to: {lai_path}")
    plt.close()

    if irrigated_water is not None and not irrigated_water.empty and 'IRRIG' in irrigated_water.columns:
        irrigation_events = irrigated_water[irrigated_water['IRRIG'] > 0]
        plt.figure(figsize=(10, 6))
        plt.bar(irrigation_events.index, irrigation_events['IRRIG'], width=1.5)
        plt.title('Daily Irrigation Events')
        plt.xlabel('Date')
        plt.ylabel('Irrigation (mm)')
        plt.grid(axis='y')
        irrigation_path = os.path.join(workspace, 'daily_irrigation.png')
        plt.savefig(irrigation_path)
        print(f"Saved irrigation plot to: {irrigation_path}")
        plt.close()
    
    # Plot for Daily Yield (Grain Growth Rate)
    irrigated_growth['daily_yield'] = irrigated_growth['GWAD'].diff().fillna(0).clip(lower=0)
    plt.figure(figsize=(10, 6))
    plt.plot(irrigated_growth['DAP'], irrigated_growth['daily_yield'])
    plt.title('Daily Yield (Grain Growth Rate)')
    plt.xlabel('Days After Planting (DAP)')
    plt.ylabel('Daily Increase in Grain Weight (kg/ha/day)')
    plt.grid(True)
    plt.axvline(x=irrigated_mat_dap, color='red', linestyle='--', label='Harvest Day')
    plt.legend()
    daily_yield_path = os.path.join(workspace, 'daily_yield.png')
    plt.savefig(daily_yield_path)
    print(f"Saved daily yield plot to: {daily_yield_path}")
    plt.close()

if __name__ == "__main__":
    run_all_analyses_and_visualizations()

Attempting to install package using 'python3'...

--- SECTION 1: Initializing Workspace and Data ---
DSSAT workspace created at: d:\Final-Year-Project\dssat_simulation_workspace
Successfully downloaded SOIL.SOL
Successfully downloaded ITHY8001.WTH
Successfully downloaded ITHY8101.WTH
d:\Final-Year-Project\dssat_simulation_workspace created.

--- SECTION 2: Running Treatment Comparison (Rainfed vs. Irrigated) ---

RUN    TRT FLO MAT TOPWT HARWT  RAIN  TIRR   CET  PESW  TNUP  TNLF   TSON TSOC
           dap dap kg/ha kg/ha    mm    mm    mm    mm kg/ha kg/ha  kg/ha t/ha
  1 SG   1  67 109  1985   612   592     0   307   130    15     8  13377   95

RUN    TRT FLO MAT TOPWT HARWT  RAIN  TIRR   CET  PESW  TNUP  TNLF   TSON TSOC
           dap dap kg/ha kg/ha    mm    mm    mm    mm kg/ha kg/ha  kg/ha t/ha
  1 SG   1  67 109  1839   565   592    45   312   130    14     8  13377   95
Saved treatment comparison plot to: d:\Final-Year-Project\dssat_simulation_workspace\treatment_comparison_gr

In [20]:
import os
import urllib.request
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import subprocess
import sys

def install_package(package):
    """Installs a package using pip, trying 'python3' and then 'python'."""
    try:
        print("Attempting to install package using 'python3'...")
        subprocess.check_call(["python3", "-m", "pip", "install", package])
        print(f"Successfully installed {package} with 'python3'")
    except FileNotFoundError:
        print("'python3' not found. Trying 'python' instead...")
        try:
            subprocess.check_call(["python", "-m", "pip", "install", package])
            print(f"Successfully installed {package} with 'python'")
        except FileNotFoundError:
            print("Error: Neither 'python3' nor 'python' command was found in the environment's PATH.")
            raise
        except subprocess.CalledProcessError as e:
            print(f"Error installing {package} with 'python': {e}")
            raise
    except subprocess.CalledProcessError as e:
        print(f"Error installing {package} with 'python3': {e}")
        raise


def download_file(url, target_path):
    """Downloads a file from a URL and saves it locally."""
    try:
        with urllib.request.urlopen(url) as response, open(target_path, 'wb') as out_file:
            data = response.read()
            out_file.write(data)
        print(f"Successfully downloaded {os.path.basename(target_path)}")
    except Exception as e:
        print(f"Error downloading {url}: {e}")
        raise

def run_all_analyses_and_visualizations():
    """
    Sets up a DSSAT environment, runs multiple analysis scenarios,
    and generates all relevant visualizations.
    """
    # First, ensure DSSATTools is installed
    install_package('DSSATTools')
    
    # Now import from the newly installed package
    from DSSATTools import (
        crop,
        WeatherStation,
        SoilProfile,
        filex,
        DSSAT
    )
    from DSSATTools.filex import IrrigationEvent, FertilizerEvent

    # ===================================================================
    # SECTION 1: BASIC SETUP AND DATA ACQUISITION
    # ===================================================================
    print("\n--- SECTION 1: Initializing Workspace and Data ---")
    workspace = os.path.join(os.getcwd(), 'dssat_simulation_workspace')
    os.makedirs(workspace, exist_ok=True)
    print(f"DSSAT workspace created at: {workspace}")

    soil_path = os.path.join(workspace, 'SOIL.SOL')
    download_file('https://raw.githubusercontent.com/DSSAT/dssat-csm-data/develop/Soil/SOIL.SOL', soil_path)

    weather_dfs = []
    for year in ['80', '81']:
        filename = f'ITHY{year}01.WTH'
        url = f'https://raw.githubusercontent.com/DSSAT/dssat-csm-data/develop/Weather/{filename}'
        path = os.path.join(workspace, filename)
        download_file(url, path)
        df = pd.read_fwf(path, colspecs=[(0, 5), (6, 11), (12, 17), (18, 23), (24, 29)], 
                         names=['date', 'srad', 'tmax', 'tmin', 'rain'], skiprows=4)
        weather_dfs.append(df)

    df_weather = pd.concat(weather_dfs, ignore_index=True)
    df_weather.dropna(subset=['date'], inplace=True)
    df_weather['date'] = pd.to_datetime(df_weather['date'], format='%y%j')
    df_weather.drop_duplicates(subset='date', keep='first', inplace=True)
    df_weather.sort_values(by='date', inplace=True)

    weather_station = WeatherStation(insi='ITHY', lat=17.53, long=78.27, elev=0, tav=25.8, amp=11.8, table=df_weather)
    soil = SoilProfile.from_file('IBSG910085', soil_path)
    dssat_env = DSSAT(workspace)

    initial_conditions = filex.InitialConditions(pcr='SG', icdat=datetime(1980, 7, 3), icres=1300)
    planting = filex.Planting(pdate=datetime(1980, 6, 17), ppop=18, plds='R', plrs=45, pldp=5)
    
    # ===================================================================
    # SECTION 2: TREATMENT COMPARISON (Rainfed vs. Irrigated)
    # ===================================================================
    print("\n--- SECTION 2: Running Treatment Comparison (Rainfed vs. Irrigated) ---")
    
    field = filex.Field(id_field='ITHY0001', wsta=weather_station, id_soil=soil)
    outputs_config = filex.SCOutputs(grout='Y', waout='Y', sumry='Y')
    
    # Run Rainfed Treatment
    simulation_controls_rainfed = filex.SimulationControls(
        general=filex.SCGeneral(sdate=datetime(1980, 6, 17)),
        options=filex.SCOptions(water='Y', nitro='Y', symbi='N'),
        management=filex.SCManagement(irrig='N', ferti='N', harvs='M'),
        outputs=outputs_config
    )
    rainfed_summary = dssat_env.run_treatment(
        field=field,
        cultivar=crop.Sorghum('IB0026'),
        planting=planting,
        initial_conditions=initial_conditions,
        simulation_controls=simulation_controls_rainfed
    )
    rainfed_growth = dssat_env.output_tables.get('PlantGro').copy()

    # Run Irrigated Treatment
    irrigation = filex.Irrigation(table=[
        IrrigationEvent(idate=datetime(1980, 7, 15), irop='IR001', irval=20),
        IrrigationEvent(idate=datetime(1980, 8, 15), irop='IR001', irval=25),
    ])
    simulation_controls_irrigated = filex.SimulationControls(
        general=filex.SCGeneral(sdate=datetime(1980, 6, 17)),
        options=filex.SCOptions(water='Y', nitro='Y', symbi='N'),
        management=filex.SCManagement(irrig='R', ferti='N', harvs='M'),
        outputs=outputs_config
    )
    irrigated_summary = dssat_env.run_treatment(
        field=field,
        cultivar=crop.Sorghum('IB0026'),
        planting=planting,
        initial_conditions=initial_conditions,
        simulation_controls=simulation_controls_irrigated,
        irrigation=irrigation
    )
    irrigated_growth = dssat_env.output_tables.get('PlantGro').copy()
    irrigated_water = dssat_env.output_tables.get('SoilWat').copy()

    # Create Comparison Plot with Harvest Day markers
    plt.figure(figsize=(10, 6))
    plt.plot(rainfed_growth.index, rainfed_growth['GWAD'], label='Rainfed')
    plt.plot(irrigated_growth.index, irrigated_growth['GWAD'], label='Irrigated', linestyle='--')
    
    pdate_dssattools = planting['pdate']
    planting_date = datetime(pdate_dssattools.year, pdate_dssattools.month, pdate_dssattools.day)
    
    rainfed_mat_dap = rainfed_summary['mat']
    irrigated_mat_dap = irrigated_summary['mat']
    rainfed_harvest_date = planting_date + timedelta(days=rainfed_mat_dap)
    irrigated_harvest_date = planting_date + timedelta(days=irrigated_mat_dap)
    
    plt.axvline(x=rainfed_harvest_date, color='red', linestyle=':', label=f'Rainfed Harvest ({rainfed_mat_dap} DAP)')
    plt.axvline(x=irrigated_harvest_date, color='blue', linestyle=':', label=f'Irrigated Harvest ({irrigated_mat_dap} DAP)')
    
    plt.title('Treatment Comparison: Grain Growth')
    plt.xlabel('Date')
    plt.ylabel('Grain Weight (kg/ha)')
    plt.legend()
    plt.grid(True)
    comp_path = os.path.join(workspace, 'treatment_comparison_growth.png')
    plt.savefig(comp_path)
    print(f"Saved treatment comparison plot to: {comp_path}")
    plt.close()
    
    # ===================================================================
    # SECTION 3: PARAMETER SENSITIVITY (Varying 'p1' coefficient)
    # ===================================================================
    print("\n--- SECTION 3: Running Parameter Sensitivity Analysis (P1 Coefficient) ---")
    
    p1_values = [350, 380, 410, 440, 470]
    sensitivity_results = []
    
    cultivar_sens = crop.Sorghum('IB0026')
    simulation_controls_sens = filex.SimulationControls(
        general=filex.SCGeneral(sdate=datetime(1980, 6, 17)),
        options=filex.SCOptions(water='Y', nitro='Y', symbi='N'),
        management=filex.SCManagement(irrig='N', ferti='N', harvs='M'),
        outputs=filex.SCOutputs(sumry='Y') 
    )
    
    for p1 in p1_values:
        cultivar_sens['p1'] = p1
        print(f"Running simulation for p1 = {p1}...")
        summary = dssat_env.run_treatment(
            field=field,
            cultivar=cultivar_sens,
            planting=planting,
            initial_conditions=initial_conditions,
            simulation_controls=simulation_controls_sens
        )
        summary['p1'] = p1
        sensitivity_results.append(summary)
        
    results_df = pd.DataFrame(sensitivity_results)

    # Create Sensitivity Plots
    plt.figure(figsize=(10, 6))
    plt.plot(results_df['p1'], results_df['harwt'], marker='o')
    plt.title('Sensitivity Analysis: Yield vs. P1 Coefficient')
    plt.xlabel('P1 Value (GDD)')
    plt.ylabel('Grain Weight at Harvest (kg/ha)')
    plt.grid(True)
    sens_yield_path = os.path.join(workspace, 'sensitivity_yield_vs_p1.png')
    plt.savefig(sens_yield_path)
    print(f"Saved sensitivity plot (Yield vs P1) to: {sens_yield_path}")
    plt.close()

    plt.figure(figsize=(10, 6))
    plt.plot(results_df['p1'], results_df['mat'], marker='o')
    plt.title('Sensitivity Analysis: Maturity vs. P1 Coefficient')
    plt.xlabel('P1 Value (GDD)')
    plt.ylabel('Days to Maturity (dap)')
    plt.grid(True)
    sens_mat_path = os.path.join(workspace, 'sensitivity_maturity_vs_p1.png')
    plt.savefig(sens_mat_path)
    print(f"Saved sensitivity plot (Maturity vs P1) to: {sens_mat_path}")
    plt.close()

    # ===================================================================
    # SECTION 4: DETAILED DAILY TIME-SERIES PLOTS
    # ===================================================================
    print("\n--- SECTION 4: Generating Detailed Daily Plots ---")

    # Re-use results from the irrigated run in Section 2
    plt.figure(figsize=(10, 6))
    plt.plot(irrigated_growth.index, irrigated_growth['LAID'])
    plt.title('Daily Leaf Area Index (Irrigated Treatment)')
    plt.xlabel('Date')
    plt.ylabel('Leaf Area Index (LAI)')
    plt.grid(True)
    plt.axvline(x=irrigated_harvest_date, color='red', linestyle='--', label=f'Harvest Day ({irrigated_mat_dap} DAP)')
    plt.legend()
    lai_path = os.path.join(workspace, 'daily_lai.png')
    plt.savefig(lai_path)
    print(f"Saved LAI plot to: {lai_path}")
    plt.close()

    if irrigated_water is not None and not irrigated_water.empty and 'IRRIG' in irrigated_water.columns:
        irrigation_events = irrigated_water[irrigated_water['IRRIG'] > 0]
        plt.figure(figsize=(10, 6))
        plt.bar(irrigation_events.index, irrigation_events['IRRIG'], width=1.5)
        plt.title('Daily Irrigation Events')
        plt.xlabel('Date')
        plt.ylabel('Irrigation (mm)')
        plt.grid(axis='y')
        irrigation_path = os.path.join(workspace, 'daily_irrigation.png')
        plt.savefig(irrigation_path)
        print(f"Saved irrigation plot to: {irrigation_path}")
        plt.close()
    
    # Plot for Daily Yield (Grain Growth Rate)
    irrigated_growth['daily_yield'] = irrigated_growth['GWAD'].diff().fillna(0).clip(lower=0)
    plt.figure(figsize=(10, 6))
    plt.plot(irrigated_growth['DAP'], irrigated_growth['daily_yield'])
    plt.title('Daily Yield (Grain Growth Rate)')
    plt.xlabel('Days After Planting (DAP)')
    plt.ylabel('Daily Increase in Grain Weight (kg/ha/day)')
    plt.grid(True)
    plt.axvline(x=irrigated_mat_dap, color='red', linestyle='--', label='Harvest Day')
    plt.legend()
    daily_yield_path = os.path.join(workspace, 'daily_yield.png')
    plt.savefig(daily_yield_path)
    print(f"Saved daily yield plot to: {daily_yield_path}")
    plt.close()

    # NEW: Plot for Total Accumulated Yield
    # ----------------------------------------
    plt.figure(figsize=(10, 6))
    plt.plot(irrigated_growth['DAP'], irrigated_growth['GWAD'], label='Total Accumulated Grain')
    
    # Get the final yield value at harvest to mark on the plot
    final_yield = irrigated_growth.loc[irrigated_growth.index == irrigated_harvest_date, 'GWAD'].values[0]
    
    plt.axvline(x=irrigated_mat_dap, color='red', linestyle='--', label=f'Harvest Day ({irrigated_mat_dap} DAP)')
    plt.scatter(irrigated_mat_dap, final_yield, color='red', zorder=5, s=100, label=f'Final Yield: {int(final_yield)} kg/ha')
    
    plt.title('Total Accumulated Grain Yield Over Time')
    plt.xlabel('Days After Planting (DAP)')
    plt.ylabel('Total Grain Weight (kg/ha)')
    plt.legend()
    plt.grid(True)
    cumulative_yield_path = os.path.join(workspace, 'cumulative_yield.png')
    plt.savefig(cumulative_yield_path)
    print(f"Saved cumulative yield plot to: {cumulative_yield_path}")
    plt.close()


if __name__ == "__main__":
    run_all_analyses_and_visualizations()

Attempting to install package using 'python3'...

--- SECTION 1: Initializing Workspace and Data ---
DSSAT workspace created at: d:\Final-Year-Project\dssat_simulation_workspace
Successfully downloaded SOIL.SOL
Successfully downloaded ITHY8001.WTH
Successfully downloaded ITHY8101.WTH
d:\Final-Year-Project\dssat_simulation_workspace created.

--- SECTION 2: Running Treatment Comparison (Rainfed vs. Irrigated) ---

RUN    TRT FLO MAT TOPWT HARWT  RAIN  TIRR   CET  PESW  TNUP  TNLF   TSON TSOC
           dap dap kg/ha kg/ha    mm    mm    mm    mm kg/ha kg/ha  kg/ha t/ha
  1 SG   1  67 109  1985   612   592     0   307   130    15     8  13377   95

RUN    TRT FLO MAT TOPWT HARWT  RAIN  TIRR   CET  PESW  TNUP  TNLF   TSON TSOC
           dap dap kg/ha kg/ha    mm    mm    mm    mm kg/ha kg/ha  kg/ha t/ha
  1 SG   1  67 109  1839   565   592    45   312   130    14     8  13377   95
Saved treatment comparison plot to: d:\Final-Year-Project\dssat_simulation_workspace\treatment_comparison_gr

In [22]:
import os
import urllib.request
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import subprocess
import sys

def install_package(package):
    """Installs a package using pip, trying 'python3' and then 'python'."""
    try:
        print("Attempting to install package using 'python3'...")
        subprocess.check_call(["python3", "-m", "pip", "install", package])
        print(f"Successfully installed {package} with 'python3'")
    except FileNotFoundError:
        print("'python3' not found. Trying 'python' instead...")
        try:
            subprocess.check_call(["python", "-m", "pip", "install", package])
            print(f"Successfully installed {package} with 'python'")
        except FileNotFoundError:
            print("Error: Neither 'python3' nor 'python' command was found in the environment's PATH.")
            raise
        except subprocess.CalledProcessError as e:
            print(f"Error installing {package} with 'python': {e}")
            raise
    except subprocess.CalledProcessError as e:
        print(f"Error installing {package} with 'python3': {e}")
        raise


def download_file(url, target_path):
    """Downloads a file from a URL and saves it locally."""
    try:
        with urllib.request.urlopen(url) as response, open(target_path, 'wb') as out_file:
            data = response.read()
            out_file.write(data)
        print(f"Successfully downloaded {os.path.basename(target_path)}")
    except Exception as e:
        print(f"Error downloading {url}: {e}")
        raise

def run_all_analyses_and_visualizations():
    """
    Sets up a DSSAT environment, runs multiple analysis scenarios,
    and generates all relevant visualizations.
    """
    # First, ensure DSSATTools is installed
    install_package('DSSATTools')
    
    # Now import from the newly installed package
    from DSSATTools import (
        crop,
        WeatherStation,
        SoilProfile,
        filex,
        DSSAT
    )
    from DSSATTools.filex import IrrigationEvent, FertilizerEvent

    # ===================================================================
    # SECTION 1: BASIC SETUP AND DATA ACQUISITION
    # ===================================================================
    print("\n--- SECTION 1: Initializing Workspace and Data ---")
    workspace = os.path.join(os.getcwd(), 'dssat_simulation_workspace')
    os.makedirs(workspace, exist_ok=True)
    print(f"DSSAT workspace created at: {workspace}")

    soil_path = os.path.join(workspace, 'SOIL.SOL')
    download_file('https://raw.githubusercontent.com/DSSAT/dssat-csm-data/develop/Soil/SOIL.SOL', soil_path)

    weather_dfs = []
    for year in ['80', '81']:
        filename = f'ITHY{year}01.WTH'
        url = f'https://raw.githubusercontent.com/DSSAT/dssat-csm-data/develop/Weather/{filename}'
        path = os.path.join(workspace, filename)
        download_file(url, path)
        df = pd.read_fwf(path, colspecs=[(0, 5), (6, 11), (12, 17), (18, 23), (24, 29)], 
                         names=['date', 'srad', 'tmax', 'tmin', 'rain'], skiprows=4)
        weather_dfs.append(df)

    df_weather = pd.concat(weather_dfs, ignore_index=True)
    df_weather.dropna(subset=['date'], inplace=True)
    df_weather['date'] = pd.to_datetime(df_weather['date'], format='%y%j')
    df_weather.drop_duplicates(subset='date', keep='first', inplace=True)
    df_weather.sort_values(by='date', inplace=True)

    weather_station = WeatherStation(insi='ITHY', lat=17.53, long=78.27, elev=0, tav=25.8, amp=11.8, table=df_weather)
    soil = SoilProfile.from_file('IBSG910085', soil_path)
    dssat_env = DSSAT(workspace)

    initial_conditions = filex.InitialConditions(pcr='SG', icdat=datetime(1980, 7, 3), icres=1300)
    planting = filex.Planting(pdate=datetime(1980, 6, 17), ppop=18, plds='R', plrs=45, pldp=5)
    
    # ===================================================================
    # SECTION 2: TREATMENT COMPARISON (Rainfed vs. Irrigated)
    # ===================================================================
    print("\n--- SECTION 2: Running Treatment Comparison (Rainfed vs. Irrigated) ---")
    
    field = filex.Field(id_field='ITHY0001', wsta=weather_station, id_soil=soil)
    outputs_config = filex.SCOutputs(grout='Y', waout='Y', sumry='Y')
    
    # Run Rainfed Treatment
    simulation_controls_rainfed = filex.SimulationControls(
        general=filex.SCGeneral(sdate=datetime(1980, 6, 17)),
        options=filex.SCOptions(water='Y', nitro='Y', symbi='N'),
        management=filex.SCManagement(irrig='N', ferti='N', harvs='M'),
        outputs=outputs_config
    )
    rainfed_summary = dssat_env.run_treatment(
        field=field,
        cultivar=crop.Sorghum('IB0026'),
        planting=planting,
        initial_conditions=initial_conditions,
        simulation_controls=simulation_controls_rainfed
    )
    rainfed_growth = dssat_env.output_tables.get('PlantGro').copy()

    # Run Irrigated Treatment
    irrigation = filex.Irrigation(table=[
        IrrigationEvent(idate=datetime(1980, 7, 15), irop='IR001', irval=20),
        IrrigationEvent(idate=datetime(1980, 8, 15), irop='IR001', irval=25),
    ])
    simulation_controls_irrigated = filex.SimulationControls(
        general=filex.SCGeneral(sdate=datetime(1980, 6, 17)),
        options=filex.SCOptions(water='Y', nitro='Y', symbi='N'),
        management=filex.SCManagement(irrig='R', ferti='N', harvs='M'),
        outputs=outputs_config
    )
    irrigated_summary = dssat_env.run_treatment(
        field=field,
        cultivar=crop.Sorghum('IB0026'),
        planting=planting,
        initial_conditions=initial_conditions,
        simulation_controls=simulation_controls_irrigated,
        irrigation=irrigation
    )
    irrigated_growth = dssat_env.output_tables.get('PlantGro').copy()
    irrigated_water = dssat_env.output_tables.get('SoilWat').copy()

    # Create Comparison Plot with Harvest Day markers
    plt.figure(figsize=(10, 6))
    plt.plot(rainfed_growth.index, rainfed_growth['GWAD'], label='Rainfed')
    plt.plot(irrigated_growth.index, irrigated_growth['GWAD'], label='Irrigated', linestyle='--')
    
    pdate_dssattools = planting['pdate']
    planting_date = datetime(pdate_dssattools.year, pdate_dssattools.month, pdate_dssattools.day)
    
    rainfed_mat_dap = rainfed_summary['mat']
    irrigated_mat_dap = irrigated_summary['mat']
    rainfed_harvest_date = planting_date + timedelta(days=rainfed_mat_dap)
    irrigated_harvest_date = planting_date + timedelta(days=irrigated_mat_dap)
    
    plt.axvline(x=rainfed_harvest_date, color='red', linestyle=':', label=f'Rainfed Harvest ({rainfed_mat_dap} DAP)')
    plt.axvline(x=irrigated_harvest_date, color='blue', linestyle=':', label=f'Irrigated Harvest ({irrigated_mat_dap} DAP)')
    
    plt.title('Treatment Comparison: Grain Growth')
    plt.xlabel('Date')
    plt.ylabel('Grain Weight (kg/ha)')
    plt.legend()
    plt.grid(True)
    comp_path = os.path.join(workspace, 'treatment_comparison_growth.png')
    plt.savefig(comp_path)
    print(f"Saved treatment comparison plot to: {comp_path}")
    plt.close()
    
    # ===================================================================
    # SECTION 3: PARAMETER SENSITIVITY (Varying 'p1' coefficient)
    # ===================================================================
    print("\n--- SECTION 3: Running Parameter Sensitivity Analysis (P1 Coefficient) ---")
    
    p1_values = [350, 380, 410, 440, 470]
    sensitivity_results = []
    
    cultivar_sens = crop.Sorghum('IB0026')
    simulation_controls_sens = filex.SimulationControls(
        general=filex.SCGeneral(sdate=datetime(1980, 6, 17)),
        options=filex.SCOptions(water='Y', nitro='Y', symbi='N'),
        management=filex.SCManagement(irrig='N', ferti='N', harvs='M'),
        outputs=filex.SCOutputs(sumry='Y') 
    )
    
    for p1 in p1_values:
        cultivar_sens['p1'] = p1
        print(f"Running simulation for p1 = {p1}...")
        summary = dssat_env.run_treatment(
            field=field,
            cultivar=cultivar_sens,
            planting=planting,
            initial_conditions=initial_conditions,
            simulation_controls=simulation_controls_sens
        )
        summary['p1'] = p1
        sensitivity_results.append(summary)
        
    results_df = pd.DataFrame(sensitivity_results)

    # Create Sensitivity Plots
    plt.figure(figsize=(10, 6))
    plt.plot(results_df['p1'], results_df['harwt'], marker='o')
    plt.title('Sensitivity Analysis: Yield vs. P1 Coefficient')
    plt.xlabel('P1 Value (GDD)')
    plt.ylabel('Grain Weight at Harvest (kg/ha)')
    plt.grid(True)
    sens_yield_path = os.path.join(workspace, 'sensitivity_yield_vs_p1.png')
    plt.savefig(sens_yield_path)
    print(f"Saved sensitivity plot (Yield vs P1) to: {sens_yield_path}")
    plt.close()

    plt.figure(figsize=(10, 6))
    plt.plot(results_df['p1'], results_df['mat'], marker='o')
    plt.title('Sensitivity Analysis: Maturity vs. P1 Coefficient')
    plt.xlabel('P1 Value (GDD)')
    plt.ylabel('Days to Maturity (dap)')
    plt.grid(True)
    sens_mat_path = os.path.join(workspace, 'sensitivity_maturity_vs_p1.png')
    plt.savefig(sens_mat_path)
    print(f"Saved sensitivity plot (Maturity vs P1) to: {sens_mat_path}")
    plt.close()

    # ===================================================================
    # SECTION 4: DETAILED DAILY TIME-SERIES PLOTS
    # ===================================================================
    print("\n--- SECTION 4: Generating Detailed Daily Plots ---")

    # Re-use results from the irrigated run in Section 2
    plt.figure(figsize=(10, 6))
    plt.plot(irrigated_growth.index, irrigated_growth['LAID'])
    plt.title('Daily Leaf Area Index (Irrigated Treatment)')
    plt.xlabel('Date')
    plt.ylabel('Leaf Area Index (LAI)')
    plt.grid(True)
    plt.axvline(x=irrigated_harvest_date, color='red', linestyle='--', label=f'Harvest Day ({irrigated_mat_dap} DAP)')
    plt.legend()
    lai_path = os.path.join(workspace, 'daily_lai.png')
    plt.savefig(lai_path)
    print(f"Saved LAI plot to: {lai_path}")
    plt.close()

    if irrigated_water is not None and not irrigated_water.empty and 'IRRIG' in irrigated_water.columns:
        irrigation_events = irrigated_water[irrigated_water['IRRIG'] > 0]
        plt.figure(figsize=(10, 6))
        plt.bar(irrigation_events.index, irrigation_events['IRRIG'], width=1.5)
        plt.title('Daily Irrigation Events')
        plt.xlabel('Date')
        plt.ylabel('Irrigation (mm)')
        plt.grid(axis='y')
        irrigation_path = os.path.join(workspace, 'daily_irrigation.png')
        plt.savefig(irrigation_path)
        print(f"Saved irrigation plot to: {irrigation_path}")
        plt.close()
    
    # Plot for Daily Yield (Grain Growth Rate)
    irrigated_growth['daily_yield'] = irrigated_growth['GWAD'].diff().fillna(0).clip(lower=0)
    plt.figure(figsize=(10, 6))
    plt.plot(irrigated_growth['DAP'], irrigated_growth['daily_yield'])
    plt.title('Daily Yield (Grain Growth Rate)')
    plt.xlabel('Days After Planting (DAP)')
    plt.ylabel('Daily Increase in Grain Weight (kg/ha/day)')
    plt.grid(True)
    plt.axvline(x=irrigated_mat_dap, color='red', linestyle='--', label='Harvest Day')
    plt.legend()
    daily_yield_path = os.path.join(workspace, 'daily_yield.png')
    plt.savefig(daily_yield_path)
    print(f"Saved daily yield plot to: {daily_yield_path}")
    plt.close()

    # Plot for Total Accumulated Yield
    plt.figure(figsize=(10, 6))
    plt.plot(irrigated_growth['DAP'], irrigated_growth['GWAD'], label='Total Accumulated Grain')
    
    final_yield = irrigated_growth.loc[irrigated_growth.index == irrigated_harvest_date, 'GWAD'].values[0]
    
    plt.axvline(x=irrigated_mat_dap, color='red', linestyle='--', label=f'Harvest Day ({irrigated_mat_dap} DAP)')
    plt.scatter(irrigated_mat_dap, final_yield, color='red', zorder=5, s=100, label=f'Final Yield: {int(final_yield)} kg/ha')
    
    plt.title('Total Accumulated Grain Yield Over Time')
    plt.xlabel('Days After Planting (DAP)')
    plt.ylabel('Total Grain Weight (kg/ha)')
    plt.legend()
    plt.grid(True)
    cumulative_yield_path = os.path.join(workspace, 'cumulative_yield.png')
    plt.savefig(cumulative_yield_path)
    print(f"Saved cumulative yield plot to: {cumulative_yield_path}")
    plt.close()

    # Plot for Daily Water Input (Rain + Irrigation)
    if irrigated_water is not None and not irrigated_water.empty:
        # FIXED: Combine rain data from weather input with irrigation data from simulation output
        df_weather_indexed = df_weather.set_index('date')
        df_water_input = irrigated_water.join(df_weather_indexed['rain']).fillna(0)
        # Rename the column for consistency in plotting
        df_water_input.rename(columns={'rain': 'RAIN'}, inplace=True)
        
        plt.figure(figsize=(12, 6))
        # Plot rain as the base of the stacked bar chart
        plt.bar(df_water_input.index, df_water_input['RAIN'], label='Rain')
        # Plot irrigation on top of the rain
        if 'IRRIG' in df_water_input.columns:
            plt.bar(df_water_input.index, df_water_input['IRRIG'], bottom=df_water_input['RAIN'], label='Irrigation')

        plt.title('Daily Water Input (Rain + Irrigation)')
        plt.xlabel('Date')
        plt.ylabel('Water (mm/day)')
        plt.legend()
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.tight_layout()
        water_input_path = os.path.join(workspace, 'daily_water_input.png')
        plt.savefig(water_input_path)
        print(f"Saved daily water input plot to: {water_input_path}")
        plt.close()


if __name__ == "__main__":
    run_all_analyses_and_visualizations()

Attempting to install package using 'python3'...

--- SECTION 1: Initializing Workspace and Data ---
DSSAT workspace created at: d:\Final-Year-Project\dssat_simulation_workspace
Successfully downloaded SOIL.SOL
Successfully downloaded ITHY8001.WTH
Successfully downloaded ITHY8101.WTH
d:\Final-Year-Project\dssat_simulation_workspace created.

--- SECTION 2: Running Treatment Comparison (Rainfed vs. Irrigated) ---

RUN    TRT FLO MAT TOPWT HARWT  RAIN  TIRR   CET  PESW  TNUP  TNLF   TSON TSOC
           dap dap kg/ha kg/ha    mm    mm    mm    mm kg/ha kg/ha  kg/ha t/ha
  1 SG   1  67 109  1985   612   592     0   307   130    15     8  13377   95

RUN    TRT FLO MAT TOPWT HARWT  RAIN  TIRR   CET  PESW  TNUP  TNLF   TSON TSOC
           dap dap kg/ha kg/ha    mm    mm    mm    mm kg/ha kg/ha  kg/ha t/ha
  1 SG   1  67 109  1839   565   592    45   312   130    14     8  13377   95
Saved treatment comparison plot to: d:\Final-Year-Project\dssat_simulation_workspace\treatment_comparison_gr

In [23]:
import os
import urllib.request
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import subprocess
import sys

def install_package(package):
    """Installs a package using pip, trying 'python3' and then 'python'."""
    try:
        print("Attempting to install package using 'python3'...")
        subprocess.check_call(["python3", "-m", "pip", "install", package])
        print(f"Successfully installed {package} with 'python3'")
    except FileNotFoundError:
        print("'python3' not found. Trying 'python' instead...")
        try:
            subprocess.check_call(["python", "-m", "pip", "install", package])
            print(f"Successfully installed {package} with 'python'")
        except FileNotFoundError:
            print("Error: Neither 'python3' nor 'python' command was found in the environment's PATH.")
            raise
        except subprocess.CalledProcessError as e:
            print(f"Error installing {package} with 'python': {e}")
            raise
    except subprocess.CalledProcessError as e:
        print(f"Error installing {package} with 'python3': {e}")
        raise


def download_file(url, target_path):
    """Downloads a file from a URL and saves it locally."""
    try:
        with urllib.request.urlopen(url) as response, open(target_path, 'wb') as out_file:
            data = response.read()
            out_file.write(data)
        print(f"Successfully downloaded {os.path.basename(target_path)}")
    except Exception as e:
        print(f"Error downloading {url}: {e}")
        raise

def run_all_analyses_and_visualizations():
    """
    Sets up a DSSAT environment, runs multiple analysis scenarios,
    and generates all relevant visualizations.
    """
    # First, ensure DSSATTools is installed
    install_package('DSSATTools')
    
    # Now import from the newly installed package
    from DSSATTools import (
        crop,
        WeatherStation,
        SoilProfile,
        filex,
        DSSAT
    )
    from DSSATTools.filex import IrrigationEvent, FertilizerEvent

    # ===================================================================
    # SECTION 1: BASIC SETUP AND DATA ACQUISITION
    # ===================================================================
    print("\n--- SECTION 1: Initializing Workspace and Data ---")
    workspace = os.path.join(os.getcwd(), 'dssat_simulation_workspace')
    os.makedirs(workspace, exist_ok=True)
    print(f"DSSAT workspace created at: {workspace}")

    soil_path = os.path.join(workspace, 'SOIL.SOL')
    download_file('https://raw.githubusercontent.com/DSSAT/dssat-csm-data/develop/Soil/SOIL.SOL', soil_path)

    weather_dfs = []
    for year in ['80', '81']:
        filename = f'ITHY{year}01.WTH'
        url = f'https://raw.githubusercontent.com/DSSAT/dssat-csm-data/develop/Weather/{filename}'
        path = os.path.join(workspace, filename)
        download_file(url, path)
        df = pd.read_fwf(path, colspecs=[(0, 5), (6, 11), (12, 17), (18, 23), (24, 29)], 
                         names=['date', 'srad', 'tmax', 'tmin', 'rain'], skiprows=4)
        weather_dfs.append(df)

    df_weather = pd.concat(weather_dfs, ignore_index=True)
    df_weather.dropna(subset=['date'], inplace=True)
    df_weather['date'] = pd.to_datetime(df_weather['date'], format='%y%j')
    df_weather.drop_duplicates(subset='date', keep='first', inplace=True)
    df_weather.sort_values(by='date', inplace=True)

    weather_station = WeatherStation(insi='ITHY', lat=17.53, long=78.27, elev=0, tav=25.8, amp=11.8, table=df_weather)
    soil = SoilProfile.from_file('IBSG910085', soil_path)
    dssat_env = DSSAT(workspace)

    initial_conditions = filex.InitialConditions(pcr='SG', icdat=datetime(1980, 7, 3), icres=1300)
    planting = filex.Planting(pdate=datetime(1980, 6, 17), ppop=18, plds='R', plrs=45, pldp=5)
    
    # ===================================================================
    # SECTION 2: TREATMENT COMPARISON (Rainfed vs. Irrigated)
    # ===================================================================
    print("\n--- SECTION 2: Running Treatment Comparison (Rainfed vs. Irrigated) ---")
    
    field = filex.Field(id_field='ITHY0001', wsta=weather_station, id_soil=soil)
    outputs_config = filex.SCOutputs(grout='Y', waout='Y', sumry='Y')
    
    # Run Rainfed Treatment
    simulation_controls_rainfed = filex.SimulationControls(
        general=filex.SCGeneral(sdate=datetime(1980, 6, 17)),
        options=filex.SCOptions(water='Y', nitro='Y', symbi='N'),
        management=filex.SCManagement(irrig='N', ferti='N', harvs='M'),
        outputs=outputs_config
    )
    rainfed_summary = dssat_env.run_treatment(
        field=field,
        cultivar=crop.Sorghum('IB0026'),
        planting=planting,
        initial_conditions=initial_conditions,
        simulation_controls=simulation_controls_rainfed
    )
    rainfed_growth = dssat_env.output_tables.get('PlantGro').copy()

    # Run Irrigated Treatment
    irrigation = filex.Irrigation(table=[
        IrrigationEvent(idate=datetime(1980, 7, 15), irop='IR001', irval=20),
        IrrigationEvent(idate=datetime(1980, 8, 15), irop='IR001', irval=25),
    ])
    simulation_controls_irrigated = filex.SimulationControls(
        general=filex.SCGeneral(sdate=datetime(1980, 6, 17)),
        options=filex.SCOptions(water='Y', nitro='Y', symbi='N'),
        management=filex.SCManagement(irrig='R', ferti='N', harvs='M'),
        outputs=outputs_config
    )
    irrigated_summary = dssat_env.run_treatment(
        field=field,
        cultivar=crop.Sorghum('IB0026'),
        planting=planting,
        initial_conditions=initial_conditions,
        simulation_controls=simulation_controls_irrigated,
        irrigation=irrigation
    )
    irrigated_growth = dssat_env.output_tables.get('PlantGro').copy()
    irrigated_water = dssat_env.output_tables.get('SoilWat').copy()

    # Create Comparison Plot with Harvest Day markers
    plt.figure(figsize=(10, 6))
    plt.plot(rainfed_growth.index, rainfed_growth['GWAD'], label='Rainfed')
    plt.plot(irrigated_growth.index, irrigated_growth['GWAD'], label='Irrigated', linestyle='--')
    
    pdate_dssattools = planting['pdate']
    planting_date = datetime(pdate_dssattools.year, pdate_dssattools.month, pdate_dssattools.day)
    
    rainfed_mat_dap = rainfed_summary['mat']
    irrigated_mat_dap = irrigated_summary['mat']
    rainfed_harvest_date = planting_date + timedelta(days=rainfed_mat_dap)
    irrigated_harvest_date = planting_date + timedelta(days=irrigated_mat_dap)
    
    plt.axvline(x=rainfed_harvest_date, color='red', linestyle=':', label=f'Rainfed Harvest ({rainfed_mat_dap} DAP)')
    plt.axvline(x=irrigated_harvest_date, color='blue', linestyle=':', label=f'Irrigated Harvest ({irrigated_mat_dap} DAP)')
    
    plt.title('Treatment Comparison: Grain Growth')
    plt.xlabel('Date')
    plt.ylabel('Grain Weight (kg/ha)')
    plt.legend()
    plt.grid(True)
    comp_path = os.path.join(workspace, 'treatment_comparison_growth.png')
    plt.savefig(comp_path)
    print(f"Saved treatment comparison plot to: {comp_path}")
    plt.close()
    
    # ===================================================================
    # SECTION 3: PARAMETER SENSITIVITY (Varying 'p1' coefficient)
    # ===================================================================
    print("\n--- SECTION 3: Running Parameter Sensitivity Analysis (P1 Coefficient) ---")
    
    p1_values = [350, 380, 410, 440, 470]
    sensitivity_results = []
    
    cultivar_sens = crop.Sorghum('IB0026')
    simulation_controls_sens = filex.SimulationControls(
        general=filex.SCGeneral(sdate=datetime(1980, 6, 17)),
        options=filex.SCOptions(water='Y', nitro='Y', symbi='N'),
        management=filex.SCManagement(irrig='N', ferti='N', harvs='M'),
        outputs=filex.SCOutputs(sumry='Y') 
    )
    
    for p1 in p1_values:
        cultivar_sens['p1'] = p1
        print(f"Running simulation for p1 = {p1}...")
        summary = dssat_env.run_treatment(
            field=field,
            cultivar=cultivar_sens,
            planting=planting,
            initial_conditions=initial_conditions,
            simulation_controls=simulation_controls_sens
        )
        summary['p1'] = p1
        sensitivity_results.append(summary)
        
    results_df = pd.DataFrame(sensitivity_results)

    # Create Sensitivity Plots
    plt.figure(figsize=(10, 6))
    plt.plot(results_df['p1'], results_df['harwt'], marker='o')
    plt.title('Sensitivity Analysis: Yield vs. P1 Coefficient')
    plt.xlabel('P1 Value (GDD)')
    plt.ylabel('Grain Weight at Harvest (kg/ha)')
    plt.grid(True)
    sens_yield_path = os.path.join(workspace, 'sensitivity_yield_vs_p1.png')
    plt.savefig(sens_yield_path)
    print(f"Saved sensitivity plot (Yield vs P1) to: {sens_yield_path}")
    plt.close()

    plt.figure(figsize=(10, 6))
    plt.plot(results_df['p1'], results_df['mat'], marker='o')
    plt.title('Sensitivity Analysis: Maturity vs. P1 Coefficient')
    plt.xlabel('P1 Value (GDD)')
    plt.ylabel('Days to Maturity (dap)')
    plt.grid(True)
    sens_mat_path = os.path.join(workspace, 'sensitivity_maturity_vs_p1.png')
    plt.savefig(sens_mat_path)
    print(f"Saved sensitivity plot (Maturity vs P1) to: {sens_mat_path}")
    plt.close()

    # ===================================================================
    # SECTION 4: DETAILED DAILY TIME-SERIES PLOTS
    # ===================================================================
    print("\n--- SECTION 4: Generating Detailed Daily Plots ---")

    # Re-use results from the irrigated run in Section 2
    plt.figure(figsize=(10, 6))
    plt.plot(irrigated_growth.index, irrigated_growth['LAID'])
    plt.title('Daily Leaf Area Index (Irrigated Treatment)')
    plt.xlabel('Date')
    plt.ylabel('Leaf Area Index (LAI)')
    plt.grid(True)
    plt.axvline(x=irrigated_harvest_date, color='red', linestyle='--', label=f'Harvest Day ({irrigated_mat_dap} DAP)')
    plt.legend()
    lai_path = os.path.join(workspace, 'daily_lai.png')
    plt.savefig(lai_path)
    print(f"Saved LAI plot to: {lai_path}")
    plt.close()
    
    # Plot for Daily Yield (Grain Growth Rate)
    irrigated_growth['daily_yield'] = irrigated_growth['GWAD'].diff().fillna(0).clip(lower=0)
    plt.figure(figsize=(10, 6))
    plt.plot(irrigated_growth['DAP'], irrigated_growth['daily_yield'])
    plt.title('Daily Yield (Grain Growth Rate)')
    plt.xlabel('Days After Planting (DAP)')
    plt.ylabel('Daily Increase in Grain Weight (kg/ha/day)')
    plt.grid(True)
    plt.axvline(x=irrigated_mat_dap, color='red', linestyle='--', label='Harvest Day')
    plt.legend()
    daily_yield_path = os.path.join(workspace, 'daily_yield.png')
    plt.savefig(daily_yield_path)
    print(f"Saved daily yield plot to: {daily_yield_path}")
    plt.close()

    # Plot for Total Accumulated Yield
    plt.figure(figsize=(10, 6))
    plt.plot(irrigated_growth['DAP'], irrigated_growth['GWAD'], label='Total Accumulated Grain')
    
    final_yield = irrigated_growth.loc[irrigated_growth.index == irrigated_harvest_date, 'GWAD'].values[0]
    
    plt.axvline(x=irrigated_mat_dap, color='red', linestyle='--', label=f'Harvest Day ({irrigated_mat_dap} DAP)')
    plt.scatter(irrigated_mat_dap, final_yield, color='red', zorder=5, s=100, label=f'Final Yield: {int(final_yield)} kg/ha')
    
    plt.title('Total Accumulated Grain Yield Over Time')
    plt.xlabel('Days After Planting (DAP)')
    plt.ylabel('Total Grain Weight (kg/ha)')
    plt.legend()
    plt.grid(True)
    cumulative_yield_path = os.path.join(workspace, 'cumulative_yield.png')
    plt.savefig(cumulative_yield_path)
    print(f"Saved cumulative yield plot to: {cumulative_yield_path}")
    plt.close()

    # MODIFIED: Plot for Daily Water Input (Rain & Irrigation as Lines)
    # -----------------------------------------------------------------
    if irrigated_water is not None and not irrigated_water.empty:
        df_weather_indexed = df_weather.set_index('date')
        df_water_input = irrigated_water.join(df_weather_indexed['rain']).fillna(0)
        df_water_input.rename(columns={'rain': 'RAIN'}, inplace=True)
        
        plt.figure(figsize=(12, 6))
        # Plot rain and irrigation as separate lines
        plt.plot(df_water_input.index, df_water_input['RAIN'], label='Rain')
        if 'IRRIG' in df_water_input.columns:
            plt.plot(df_water_input.index, df_water_input['IRRIG'], label='Irrigation', linestyle='--')

        plt.title('Daily Water Input (Rain & Irrigation)')
        plt.xlabel('Date')
        plt.ylabel('Water (mm/day)')
        plt.legend()
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.tight_layout()
        water_input_path = os.path.join(workspace, 'daily_water_input.png')
        plt.savefig(water_input_path)
        print(f"Saved daily water input plot to: {water_input_path}")
        plt.close()


if __name__ == "__main__":
    run_all_analyses_and_visualizations()

Attempting to install package using 'python3'...

--- SECTION 1: Initializing Workspace and Data ---
DSSAT workspace created at: d:\Final-Year-Project\dssat_simulation_workspace
Successfully downloaded SOIL.SOL
Successfully downloaded ITHY8001.WTH
Successfully downloaded ITHY8101.WTH
d:\Final-Year-Project\dssat_simulation_workspace created.

--- SECTION 2: Running Treatment Comparison (Rainfed vs. Irrigated) ---

RUN    TRT FLO MAT TOPWT HARWT  RAIN  TIRR   CET  PESW  TNUP  TNLF   TSON TSOC
           dap dap kg/ha kg/ha    mm    mm    mm    mm kg/ha kg/ha  kg/ha t/ha
  1 SG   1  67 109  1985   612   592     0   307   130    15     8  13377   95

RUN    TRT FLO MAT TOPWT HARWT  RAIN  TIRR   CET  PESW  TNUP  TNLF   TSON TSOC
           dap dap kg/ha kg/ha    mm    mm    mm    mm kg/ha kg/ha  kg/ha t/ha
  1 SG   1  67 109  1839   565   592    45   312   130    14     8  13377   95
Saved treatment comparison plot to: d:\Final-Year-Project\dssat_simulation_workspace\treatment_comparison_gr

In [25]:
# import os
# import urllib.request
# import pandas as pd
# from datetime import datetime, timedelta
# import matplotlib.pyplot as plt
# import subprocess
# import sys

# def install_package(package):
#     """Installs a package using pip, trying 'python3' and then 'python'."""
#     try:
#         print("Attempting to install package using 'python3'...")
#         subprocess.check_call(["python3", "-m", "pip", "install", package])
#         print(f"Successfully installed {package} with 'python3'")
#     except FileNotFoundError:
#         print("'python3' not found. Trying 'python' instead...")
#         try:
#             subprocess.check_call(["python", "-m", "pip", "install", package])
#             print(f"Successfully installed {package} with 'python'")
#         except FileNotFoundError:
#             print("Error: Neither 'python3' nor 'python' command was found in the environment's PATH.")
#             raise
#         except subprocess.CalledProcessError as e:
#             print(f"Error installing {package} with 'python': {e}")
#             raise
#     except subprocess.CalledProcessError as e:
#         print(f"Error installing {package} with 'python3': {e}")
#         raise


# def download_file(url, target_path):
#     """Downloads a file from a URL and saves it locally."""
#     try:
#         with urllib.request.urlopen(url) as response, open(target_path, 'wb') as out_file:
#             data = response.read()
#             out_file.write(data)
#         print(f"Successfully downloaded {os.path.basename(target_path)}")
#     except Exception as e:
#         print(f"Error downloading {url}: {e}")
#         raise

# def run_all_analyses_and_visualizations():
#     """
#     Sets up a DSSAT environment, runs multiple analysis scenarios,
#     and generates all relevant visualizations.
#     """
#     # First, ensure DSSATTools is installed
#     install_package('DSSATTools')
    
#     # Now import from the newly installed package
#     from DSSATTools import (
#         crop,
#         WeatherStation,
#         SoilProfile,
#         filex,
#         DSSAT
#     )
#     from DSSATTools.filex import IrrigationEvent, FertilizerEvent

#     # ===================================================================
#     # SECTION 1: BASIC SETUP AND DATA ACQUISITION
#     # ===================================================================
#     print("\n--- SECTION 1: Initializing Workspace and Data ---")
#     workspace = os.path.join(os.getcwd(), 'dssat_simulation_workspace')
#     os.makedirs(workspace, exist_ok=True)
#     print(f"DSSAT workspace created at: {workspace}")

#     soil_path = os.path.join(workspace, 'SOIL.SOL')
#     download_file('https://raw.githubusercontent.com/DSSAT/dssat-csm-data/develop/Soil/SOIL.SOL', soil_path)

#     weather_dfs = []
#     for year in ['80', '81']:
#         filename = f'ITHY{year}01.WTH'
#         url = f'https://raw.githubusercontent.com/DSSAT/dssat-csm-data/develop/Weather/{filename}'
#         path = os.path.join(workspace, filename)
#         download_file(url, path)
#         df = pd.read_fwf(path, colspecs=[(0, 5), (6, 11), (12, 17), (18, 23), (24, 29)], 
#                          names=['date', 'srad', 'tmax', 'tmin', 'rain'], skiprows=4)
#         weather_dfs.append(df)

#     df_weather = pd.concat(weather_dfs, ignore_index=True)
#     df_weather.dropna(subset=['date'], inplace=True)
#     df_weather['date'] = pd.to_datetime(df_weather['date'], format='%y%j')
#     df_weather.drop_duplicates(subset='date', keep='first', inplace=True)
#     df_weather.sort_values(by='date', inplace=True)

#     weather_station = WeatherStation(insi='ITHY', lat=17.53, long=78.27, elev=0, tav=25.8, amp=11.8, table=df_weather)
#     soil = SoilProfile.from_file('IBSG910085', soil_path)
#     dssat_env = DSSAT(workspace)

#     initial_conditions = filex.InitialConditions(pcr='SG', icdat=datetime(1980, 7, 3), icres=1300)
#     planting = filex.Planting(pdate=datetime(1980, 6, 17), ppop=18, plds='R', plrs=45, pldp=5)
    
#     # ===================================================================
#     # SECTION 2: TREATMENT COMPARISON (Rainfed vs. Irrigated)
#     # ===================================================================
#     print("\n--- SECTION 2: Running Treatment Comparison (Rainfed vs. Irrigated) ---")
    
#     field = filex.Field(id_field='ITHY0001', wsta=weather_station, id_soil=soil)
#     outputs_config = filex.SCOutputs(grout='Y', waout='Y', sumry='Y')
    
#     # Run Rainfed Treatment
#     simulation_controls_rainfed = filex.SimulationControls(
#         general=filex.SCGeneral(sdate=datetime(1980, 6, 17)),
#         options=filex.SCOptions(water='Y', nitro='Y', symbi='N'),
#         management=filex.SCManagement(irrig='N', ferti='N', harvs='M'),
#         outputs=outputs_config
#     )
#     rainfed_summary = dssat_env.run_treatment(
#         field=field,
#         cultivar=crop.Sorghum('IB0026'),
#         planting=planting,
#         initial_conditions=initial_conditions,
#         simulation_controls=simulation_controls_rainfed
#     )
#     rainfed_growth = dssat_env.output_tables.get('PlantGro').copy()

#     # Run Irrigated Treatment
#     irrigation = filex.Irrigation(table=[
#         IrrigationEvent(idate=datetime(1980, 7, 15), irop='IR001', irval=20),
#         IrrigationEvent(idate=datetime(1980, 8, 15), irop='IR001', irval=25),
#     ])
#     simulation_controls_irrigated = filex.SimulationControls(
#         general=filex.SCGeneral(sdate=datetime(1980, 6, 17)),
#         options=filex.SCOptions(water='Y', nitro='Y', symbi='N'),
#         management=filex.SCManagement(irrig='R', ferti='N', harvs='M'),
#         outputs=outputs_config
#     )
#     irrigated_summary = dssat_env.run_treatment(
#         field=field,
#         cultivar=crop.Sorghum('IB0026'),
#         planting=planting,
#         initial_conditions=initial_conditions,
#         simulation_controls=simulation_controls_irrigated,
#         irrigation=irrigation
#     )
#     irrigated_growth = dssat_env.output_tables.get('PlantGro').copy()
#     irrigated_water = dssat_env.output_tables.get('SoilWat').copy()

#     # Create Comparison Plot with Harvest Day markers
#     plt.figure(figsize=(10, 6))
#     plt.plot(rainfed_growth.index, rainfed_growth['GWAD'], label='Rainfed')
#     plt.plot(irrigated_growth.index, irrigated_growth['GWAD'], label='Irrigated', linestyle='--')
    
#     pdate_dssattools = planting['pdate']
#     planting_date = datetime(pdate_dssattools.year, pdate_dssattools.month, pdate_dssattools.day)
    
#     rainfed_mat_dap = rainfed_summary['mat']
#     irrigated_mat_dap = irrigated_summary['mat']
#     rainfed_harvest_date = planting_date + timedelta(days=rainfed_mat_dap)
#     irrigated_harvest_date = planting_date + timedelta(days=irrigated_mat_dap)
    
#     plt.axvline(x=rainfed_harvest_date, color='red', linestyle=':', label=f'Rainfed Harvest ({rainfed_mat_dap} DAP)')
#     plt.axvline(x=irrigated_harvest_date, color='blue', linestyle=':', label=f'Irrigated Harvest ({irrigated_mat_dap} DAP)')
    
#     plt.title('Treatment Comparison: Grain Growth')
#     plt.xlabel('Date')
#     plt.ylabel('Grain Weight (kg/ha)')
#     plt.legend()
#     plt.grid(True)
#     comp_path = os.path.join(workspace, 'treatment_comparison_growth.png')
#     plt.savefig(comp_path)
#     print(f"Saved treatment comparison plot to: {comp_path}")
#     plt.close()
    
#     # ===================================================================
#     # SECTION 3: PARAMETER SENSITIVITY (Varying 'p1' coefficient)
#     # ===================================================================
#     print("\n--- SECTION 3: Running Parameter Sensitivity Analysis (P1 Coefficient) ---")
    
#     p1_values = [350, 380, 410, 440, 470]
#     sensitivity_results = []
    
#     cultivar_sens = crop.Sorghum('IB0026')
#     simulation_controls_sens = filex.SimulationControls(
#         general=filex.SCGeneral(sdate=datetime(1980, 6, 17)),
#         options=filex.SCOptions(water='Y', nitro='Y', symbi='N'),
#         management=filex.SCManagement(irrig='N', ferti='N', harvs='M'),
#         outputs=filex.SCOutputs(sumry='Y') 
#     )
    
#     for p1 in p1_values:
#         cultivar_sens['p1'] = p1
#         print(f"Running simulation for p1 = {p1}...")
#         summary = dssat_env.run_treatment(
#             field=field,
#             cultivar=cultivar_sens,
#             planting=planting,
#             initial_conditions=initial_conditions,
#             simulation_controls=simulation_controls_sens
#         )
#         summary['p1'] = p1
#         sensitivity_results.append(summary)
        
#     results_df = pd.DataFrame(sensitivity_results)

#     # Create Sensitivity Plots
#     plt.figure(figsize=(10, 6))
#     plt.plot(results_df['p1'], results_df['harwt'], marker='o')
#     plt.title('Sensitivity Analysis: Yield vs. P1 Coefficient')
#     plt.xlabel('P1 Value (GDD)')
#     plt.ylabel('Grain Weight at Harvest (kg/ha)')
#     plt.grid(True)
#     sens_yield_path = os.path.join(workspace, 'sensitivity_yield_vs_p1.png')
#     plt.savefig(sens_yield_path)
#     print(f"Saved sensitivity plot (Yield vs P1) to: {sens_yield_path}")
#     plt.close()

#     plt.figure(figsize=(10, 6))
#     plt.plot(results_df['p1'], results_df['mat'], marker='o')
#     plt.title('Sensitivity Analysis: Maturity vs. P1 Coefficient')
#     plt.xlabel('P1 Value (GDD)')
#     plt.ylabel('Days to Maturity (dap)')
#     plt.grid(True)
#     sens_mat_path = os.path.join(workspace, 'sensitivity_maturity_vs_p1.png')
#     plt.savefig(sens_mat_path)
#     print(f"Saved sensitivity plot (Maturity vs P1) to: {sens_mat_path}")
#     plt.close()

#     # ===================================================================
#     # SECTION 4: DETAILED DAILY TIME-SERIES PLOTS
#     # ===================================================================
#     print("\n--- SECTION 4: Generating Detailed Daily Plots ---")

#     # Re-use results from the irrigated run in Section 2
#     plt.figure(figsize=(10, 6))
#     plt.plot(irrigated_growth.index, irrigated_growth['LAID'])
#     plt.title('Daily Leaf Area Index (Irrigated Treatment)')
#     plt.xlabel('Date')
#     plt.ylabel('Leaf Area Index (LAI)')
#     plt.grid(True)
#     plt.axvline(x=irrigated_harvest_date, color='red', linestyle='--', label=f'Harvest Day ({irrigated_mat_dap} DAP)')
#     plt.legend()
#     lai_path = os.path.join(workspace, 'daily_lai.png')
#     plt.savefig(lai_path)
#     print(f"Saved LAI plot to: {lai_path}")
#     plt.close()
    
#     # Plot for Daily Yield (Grain Growth Rate)
#     irrigated_growth['daily_yield'] = irrigated_growth['GWAD'].diff().fillna(0).clip(lower=0)
#     plt.figure(figsize=(10, 6))
#     plt.plot(irrigated_growth['DAP'], irrigated_growth['daily_yield'])
#     plt.title('Daily Yield (Grain Growth Rate)')
#     plt.xlabel('Days After Planting (DAP)')
#     plt.ylabel('Daily Increase in Grain Weight (kg/ha/day)')
#     plt.grid(True)
#     plt.axvline(x=irrigated_mat_dap, color='red', linestyle='--', label='Harvest Day')
#     plt.legend()
#     daily_yield_path = os.path.join(workspace, 'daily_yield.png')
#     plt.savefig(daily_yield_path)
#     print(f"Saved daily yield plot to: {daily_yield_path}")
#     plt.close()

#     # Plot for Total Accumulated Yield
#     plt.figure(figsize=(10, 6))
#     plt.plot(irrigated_growth['DAP'], irrigated_growth['GWAD'], label='Total Accumulated Grain')
    
#     final_yield = irrigated_growth.loc[irrigated_growth.index == irrigated_harvest_date, 'GWAD'].values[0]
    
#     plt.axvline(x=irrigated_mat_dap, color='red', linestyle='--', label=f'Harvest Day ({irrigated_mat_dap} DAP)')
#     plt.scatter(irrigated_mat_dap, final_yield, color='red', zorder=5, s=100, label=f'Final Yield: {int(final_yield)} kg/ha')
    
#     plt.title('Total Accumulated Grain Yield Over Time')
#     plt.xlabel('Days After Planting (DAP)')
#     plt.ylabel('Total Grain Weight (kg/ha)')
#     plt.legend()
#     plt.grid(True)
#     cumulative_yield_path = os.path.join(workspace, 'cumulative_yield.png')
#     plt.savefig(cumulative_yield_path)
#     print(f"Saved cumulative yield plot to: {cumulative_yield_path}")
#     plt.close()

#     # MODIFIED: Plot for Daily Water Input (Rain as Line, Irrigation as Bars)
#     # -----------------------------------------------------------------------
#     if irrigated_water is not None and not irrigated_water.empty:
#         df_weather_indexed = df_weather.set_index('date')
#         df_water_input = irrigated_water.join(df_weather_indexed['rain']).fillna(0)
#         df_water_input.rename(columns={'rain': 'RAIN'}, inplace=True)
        
#         plt.figure(figsize=(12, 6))
#         # Plot rain as a blue line
#         plt.plot(df_water_input.index, df_water_input['RAIN'], label='Rain', color='blue')
#         # Plot irrigation as purple bars
#         if 'IRRIG' in df_water_input.columns:
#             plt.bar(df_water_input.index, df_water_input['IRRIG'], label='Irrigation', color='purple', width=1.0)

#         plt.title('Daily Water Input (Rain & Irrigation)')
#         plt.xlabel('Date')
#         plt.ylabel('Water (mm/day)')
#         plt.legend()
#         plt.grid(axis='y', linestyle='--', alpha=0.7)
#         plt.tight_layout()
#         water_input_path = os.path.join(workspace, 'daily_water_input.png')
#         plt.savefig(water_input_path)
#         print(f"Saved daily water input plot to: {water_input_path}")
#         plt.close()


# if __name__ == "__main__":
#     run_all_analyses_and_visualizations()

In [29]:
import os
import urllib.request
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import subprocess
import sys

def install_package(package):
    """Installs a package using pip, trying 'python3' and then 'python'."""
    try:
        print("Attempting to install package using 'python3'...")
        subprocess.check_call(["python3", "-m", "pip", "install", package])
        print(f"Successfully installed {package} with 'python3'")
    except FileNotFoundError:
        print("'python3' not found. Trying 'python' instead...")
        try:
            subprocess.check_call(["python", "-m", "pip", "install", package])
            print(f"Successfully installed {package} with 'python'")
        except FileNotFoundError:
            print("Error: Neither 'python3' nor 'python' command was found in the environment's PATH.")
            raise
        except subprocess.CalledProcessError as e:
            print(f"Error installing {package} with 'python': {e}")
            raise
    except subprocess.CalledProcessError as e:
        print(f"Error installing {package} with 'python3': {e}")
        raise


def download_file(url, target_path):
    """Downloads a file from a URL and saves it locally."""
    try:
        with urllib.request.urlopen(url) as response, open(target_path, 'wb') as out_file:
            data = response.read()
            out_file.write(data)
        print(f"Successfully downloaded {os.path.basename(target_path)}")
    except Exception as e:
        print(f"Error downloading {url}: {e}")
        raise

def run_all_analyses_and_visualizations():
    """
    Sets up a DSSAT environment, runs multiple analysis scenarios,
    and generates all relevant visualizations.
    """
    # First, ensure DSSATTools is installed
    install_package('DSSATTools')
    
    # Now import from the newly installed package
    from DSSATTools import (
        crop,
        WeatherStation,
        SoilProfile,
        filex,
        DSSAT
    )
    from DSSATTools.filex import IrrigationEvent, FertilizerEvent

    # ===================================================================
    # SECTION 1: BASIC SETUP AND DATA ACQUISITION
    # ===================================================================
    print("\n--- SECTION 1: Initializing Workspace and Data ---")
    workspace = os.path.join(os.getcwd(), 'dssat_simulation_workspace')
    os.makedirs(workspace, exist_ok=True)
    print(f"DSSAT workspace created at: {workspace}")

    soil_path = os.path.join(workspace, 'SOIL.SOL')
    download_file('https://raw.githubusercontent.com/DSSAT/dssat-csm-data/develop/Soil/SOIL.SOL', soil_path)

    weather_dfs = []
    for year in ['80', '81']:
        filename = f'ITHY{year}01.WTH'
        url = f'https://raw.githubusercontent.com/DSSAT/dssat-csm-data/develop/Weather/{filename}'
        path = os.path.join(workspace, filename)
        download_file(url, path)
        df = pd.read_fwf(path, colspecs=[(0, 5), (6, 11), (12, 17), (18, 23), (24, 29)], 
                         names=['date', 'srad', 'tmax', 'tmin', 'rain'], skiprows=4)
        weather_dfs.append(df)

    df_weather = pd.concat(weather_dfs, ignore_index=True)
    df_weather.dropna(subset=['date'], inplace=True)
    df_weather['date'] = pd.to_datetime(df_weather['date'], format='%y%j')
    df_weather.drop_duplicates(subset='date', keep='first', inplace=True)
    df_weather.sort_values(by='date', inplace=True)

    weather_station = WeatherStation(insi='ITHY', lat=17.53, long=78.27, elev=0, tav=25.8, amp=11.8, table=df_weather)
    soil = SoilProfile.from_file('IBSG910085', soil_path)
    dssat_env = DSSAT(workspace)

    initial_conditions = filex.InitialConditions(pcr='SG', icdat=datetime(1980, 7, 3), icres=1300)
    planting = filex.Planting(pdate=datetime(1980, 6, 17), ppop=18, plds='R', plrs=45, pldp=5)
    
    # ===================================================================
    # SECTION 2: TREATMENT COMPARISON (Rainfed vs. Irrigated)
    # ===================================================================
    print("\n--- SECTION 2: Running Treatment Comparison (Rainfed vs. Irrigated) ---")
    
    field = filex.Field(id_field='ITHY0001', wsta=weather_station, id_soil=soil)
    outputs_config = filex.SCOutputs(grout='Y', waout='Y', sumry='Y')
    
    # Run Rainfed Treatment
    simulation_controls_rainfed = filex.SimulationControls(
        general=filex.SCGeneral(sdate=datetime(1980, 6, 17)),
        options=filex.SCOptions(water='Y', nitro='Y', symbi='N'),
        management=filex.SCManagement(irrig='N', ferti='N', harvs='M'),
        outputs=outputs_config
    )
    rainfed_summary = dssat_env.run_treatment(
        field=field,
        cultivar=crop.Sorghum('IB0026'),
        planting=planting,
        initial_conditions=initial_conditions,
        simulation_controls=simulation_controls_rainfed
    )
    rainfed_growth = dssat_env.output_tables.get('PlantGro').copy()

    # Run Irrigated Treatment
    # FIXED: Added efir=1 (irrigation efficiency) to ensure irrigation is applied.
    irrigation = filex.Irrigation(
        efir=1, 
        table=[
            IrrigationEvent(idate=datetime(1980, 7, 15), irop='IR001', irval=20),
            IrrigationEvent(idate=datetime(1980, 8, 15), irop='IR001', irval=25),
        ]
    )
    simulation_controls_irrigated = filex.SimulationControls(
        general=filex.SCGeneral(sdate=datetime(1980, 6, 17)),
        options=filex.SCOptions(water='Y', nitro='Y', symbi='N'),
        management=filex.SCManagement(irrig='R', ferti='N', harvs='M'),
        outputs=outputs_config
    )
    irrigated_summary = dssat_env.run_treatment(
        field=field,
        cultivar=crop.Sorghum('IB0026'),
        planting=planting,
        initial_conditions=initial_conditions,
        simulation_controls=simulation_controls_irrigated,
        irrigation=irrigation
    )
    irrigated_growth = dssat_env.output_tables.get('PlantGro').copy()
    irrigated_water = dssat_env.output_tables.get('SoilWat').copy()

    # Create Comparison Plot with Harvest Day markers
    plt.figure(figsize=(10, 6))
    plt.plot(rainfed_growth.index, rainfed_growth['GWAD'], label='Rainfed')
    plt.plot(irrigated_growth.index, irrigated_growth['GWAD'], label='Irrigated', linestyle='--')
    
    pdate_dssattools = planting['pdate']
    planting_date = datetime(pdate_dssattools.year, pdate_dssattools.month, pdate_dssattools.day)
    
    rainfed_mat_dap = rainfed_summary['mat']
    irrigated_mat_dap = irrigated_summary['mat']
    rainfed_harvest_date = planting_date + timedelta(days=rainfed_mat_dap)
    irrigated_harvest_date = planting_date + timedelta(days=irrigated_mat_dap)
    
    plt.axvline(x=rainfed_harvest_date, color='red', linestyle=':', label=f'Rainfed Harvest ({rainfed_mat_dap} DAP)')
    plt.axvline(x=irrigated_harvest_date, color='blue', linestyle=':', label=f'Irrigated Harvest ({irrigated_mat_dap} DAP)')
    
    plt.title('Treatment Comparison: Grain Growth')
    plt.xlabel('Date')
    plt.ylabel('Grain Weight (kg/ha)')
    plt.legend()
    plt.grid(True)
    comp_path = os.path.join(workspace, 'treatment_comparison_growth.png')
    plt.savefig(comp_path)
    print(f"Saved treatment comparison plot to: {comp_path}")
    plt.close()
    
    # ===================================================================
    # SECTION 3: PARAMETER SENSITIVITY (Varying 'p1' coefficient)
    # ===================================================================
    print("\n--- SECTION 3: Running Parameter Sensitivity Analysis (P1 Coefficient) ---")
    
    p1_values = [350, 380, 410, 440, 470]
    sensitivity_results = []
    
    cultivar_sens = crop.Sorghum('IB0026')
    simulation_controls_sens = filex.SimulationControls(
        general=filex.SCGeneral(sdate=datetime(1980, 6, 17)),
        options=filex.SCOptions(water='Y', nitro='Y', symbi='N'),
        management=filex.SCManagement(irrig='N', ferti='N', harvs='M'),
        outputs=filex.SCOutputs(sumry='Y') 
    )
    
    for p1 in p1_values:
        cultivar_sens['p1'] = p1
        print(f"Running simulation for p1 = {p1}...")
        summary = dssat_env.run_treatment(
            field=field,
            cultivar=cultivar_sens,
            planting=planting,
            initial_conditions=initial_conditions,
            simulation_controls=simulation_controls_sens
        )
        summary['p1'] = p1
        sensitivity_results.append(summary)
        
    results_df = pd.DataFrame(sensitivity_results)

    # Create Sensitivity Plots
    plt.figure(figsize=(10, 6))
    plt.plot(results_df['p1'], results_df['harwt'], marker='o')
    plt.title('Sensitivity Analysis: Yield vs. P1 Coefficient')
    plt.xlabel('P1 Value (GDD)')
    plt.ylabel('Grain Weight at Harvest (kg/ha)')
    plt.grid(True)
    sens_yield_path = os.path.join(workspace, 'sensitivity_yield_vs_p1.png')
    plt.savefig(sens_yield_path)
    print(f"Saved sensitivity plot (Yield vs P1) to: {sens_yield_path}")
    plt.close()

    plt.figure(figsize=(10, 6))
    plt.plot(results_df['p1'], results_df['mat'], marker='o')
    plt.title('Sensitivity Analysis: Maturity vs. P1 Coefficient')
    plt.xlabel('P1 Value (GDD)')
    plt.ylabel('Days to Maturity (dap)')
    plt.grid(True)
    sens_mat_path = os.path.join(workspace, 'sensitivity_maturity_vs_p1.png')
    plt.savefig(sens_mat_path)
    print(f"Saved sensitivity plot (Maturity vs P1) to: {sens_mat_path}")
    plt.close()

    # ===================================================================
    # SECTION 4: DETAILED DAILY TIME-SERIES PLOTS
    # ===================================================================
    print("\n--- SECTION 4: Generating Detailed Daily Plots ---")

    # Re-use results from the irrigated run in Section 2
    plt.figure(figsize=(10, 6))
    plt.plot(irrigated_growth.index, irrigated_growth['LAID'])
    plt.title('Daily Leaf Area Index (Irrigated Treatment)')
    plt.xlabel('Date')
    plt.ylabel('Leaf Area Index (LAI)')
    plt.grid(True)
    plt.axvline(x=irrigated_harvest_date, color='red', linestyle='--', label=f'Harvest Day ({irrigated_mat_dap} DAP)')
    plt.legend()
    lai_path = os.path.join(workspace, 'daily_lai.png')
    plt.savefig(lai_path)
    print(f"Saved LAI plot to: {lai_path}")
    plt.close()
    
    # Plot for Daily Yield (Grain Growth Rate)
    irrigated_growth['daily_yield'] = irrigated_growth['GWAD'].diff().fillna(0).clip(lower=0)
    plt.figure(figsize=(10, 6))
    plt.plot(irrigated_growth['DAP'], irrigated_growth['daily_yield'])
    plt.title('Daily Yield (Grain Growth Rate)')
    plt.xlabel('Days After Planting (DAP)')
    plt.ylabel('Daily Increase in Grain Weight (kg/ha/day)')
    plt.grid(True)
    plt.axvline(x=irrigated_mat_dap, color='red', linestyle='--', label='Harvest Day')
    plt.legend()
    daily_yield_path = os.path.join(workspace, 'daily_yield.png')
    plt.savefig(daily_yield_path)
    print(f"Saved daily yield plot to: {daily_yield_path}")
    plt.close()

    # Plot for Total Accumulated Yield
    plt.figure(figsize=(10, 6))
    plt.plot(irrigated_growth['DAP'], irrigated_growth['GWAD'], label='Total Accumulated Grain')
    
    final_yield = irrigated_growth.loc[irrigated_growth.index == irrigated_harvest_date, 'GWAD'].values[0]
    
    plt.axvline(x=irrigated_mat_dap, color='red', linestyle='--', label=f'Harvest Day ({irrigated_mat_dap} DAP)')
    plt.scatter(irrigated_mat_dap, final_yield, color='red', zorder=5, s=100, label=f'Final Yield: {int(final_yield)} kg/ha')
    
    plt.title('Total Accumulated Grain Yield Over Time')
    plt.xlabel('Days After Planting (DAP)')
    plt.ylabel('Total Grain Weight (kg/ha)')
    plt.legend()
    plt.grid(True)
    cumulative_yield_path = os.path.join(workspace, 'cumulative_yield.png')
    plt.savefig(cumulative_yield_path)
    print(f"Saved cumulative yield plot to: {cumulative_yield_path}")
    plt.close()

    # Plot for Daily Water Input (Rain & Irrigation)
    if irrigated_water is not None and not irrigated_water.empty:
        df_weather_indexed = df_weather.set_index('date')
        df_water_input = irrigated_water.join(df_weather_indexed['rain']).fillna(0)
        df_water_input.rename(columns={'rain': 'RAIN'}, inplace=True)
        
        plt.figure(figsize=(12, 6))
        plt.plot(df_water_input.index, df_water_input['RAIN'], label='Rain', color='blue')
        
        if 'IRRIG' in df_water_input.columns:
            plt.bar(df_water_input.index, df_water_input['IRRIG'], label='Irrigation', color='purple', width=1.0)
        else:
            print("\nWARNING: 'IRRIG' column not found in SoilWat.OUT. Irrigation will not be plotted.")
            plt.bar([], [], label='Irrigation (Not Found)', color='purple')

        plt.title('Daily Water Input (Rain & Irrigation)')
        plt.xlabel('Date')
        plt.ylabel('Water (mm/day)')
        plt.legend()
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.tight_layout()
        water_input_path = os.path.join(workspace, 'daily_water_input.png')
        plt.savefig(water_input_path)
        print(f"Saved daily water input plot to: {water_input_path}")
        plt.close()


if __name__ == "__main__":
    run_all_analyses_and_visualizations()
    

Attempting to install package using 'python3'...

--- SECTION 1: Initializing Workspace and Data ---
DSSAT workspace created at: d:\Final-Year-Project\dssat_simulation_workspace
Successfully downloaded SOIL.SOL
Successfully downloaded ITHY8001.WTH
Successfully downloaded ITHY8101.WTH
d:\Final-Year-Project\dssat_simulation_workspace created.

--- SECTION 2: Running Treatment Comparison (Rainfed vs. Irrigated) ---

RUN    TRT FLO MAT TOPWT HARWT  RAIN  TIRR   CET  PESW  TNUP  TNLF   TSON TSOC
           dap dap kg/ha kg/ha    mm    mm    mm    mm kg/ha kg/ha  kg/ha t/ha
  1 SG   1  67 109  1985   612   592     0   307   130    15     8  13377   95

RUN    TRT FLO MAT TOPWT HARWT  RAIN  TIRR   CET  PESW  TNUP  TNLF   TSON TSOC
           dap dap kg/ha kg/ha    mm    mm    mm    mm kg/ha kg/ha  kg/ha t/ha
  1 SG   1  67 109  1839   565   592    45   312   130    14     8  13377   95
Saved treatment comparison plot to: d:\Final-Year-Project\dssat_simulation_workspace\treatment_comparison_gr

In [30]:
import os
import urllib.request
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import subprocess
import sys

def install_package(package):
    """Installs a package using pip, trying 'python3' and then 'python'."""
    try:
        print("Attempting to install package using 'python3'...")
        subprocess.check_call(["python3", "-m", "pip", "install", package])
        print(f"Successfully installed {package} with 'python3'")
    except FileNotFoundError:
        print("'python3' not found. Trying 'python' instead...")
        try:
            subprocess.check_call(["python", "-m", "pip", "install", package])
            print(f"Successfully installed {package} with 'python'")
        except FileNotFoundError:
            print("Error: Neither 'python3' nor 'python' command was found in the environment's PATH.")
            raise
        except subprocess.CalledProcessError as e:
            print(f"Error installing {package} with 'python': {e}")
            raise
    except subprocess.CalledProcessError as e:
        print(f"Error installing {package} with 'python3': {e}")
        raise


def download_file(url, target_path):
    """Downloads a file from a URL and saves it locally."""
    try:
        with urllib.request.urlopen(url) as response, open(target_path, 'wb') as out_file:
            data = response.read()
            out_file.write(data)
        print(f"Successfully downloaded {os.path.basename(target_path)}")
    except Exception as e:
        print(f"Error downloading {url}: {e}")
        raise

def run_all_analyses_and_visualizations():
    """
    Sets up a DSSAT environment, runs multiple analysis scenarios,
    and generates all relevant visualizations.
    """
    # First, ensure DSSATTools is installed
    install_package('DSSATTools')
    
    # Now import from the newly installed package
    from DSSATTools import (
        crop,
        WeatherStation,
        SoilProfile,
        filex,
        DSSAT
    )
    from DSSATTools.filex import IrrigationEvent, FertilizerEvent

    # ===================================================================
    # SECTION 1: BASIC SETUP AND DATA ACQUISITION
    # ===================================================================
    print("\n--- SECTION 1: Initializing Workspace and Data ---")
    workspace = os.path.join(os.getcwd(), 'dssat_simulation_workspace')
    os.makedirs(workspace, exist_ok=True)
    print(f"DSSAT workspace created at: {workspace}")

    soil_path = os.path.join(workspace, 'SOIL.SOL')
    download_file('https://raw.githubusercontent.com/DSSAT/dssat-csm-data/develop/Soil/SOIL.SOL', soil_path)

    weather_dfs = []
    for year in ['80', '81']:
        filename = f'ITHY{year}01.WTH'
        url = f'https://raw.githubusercontent.com/DSSAT/dssat-csm-data/develop/Weather/{filename}'
        path = os.path.join(workspace, filename)
        download_file(url, path)
        df = pd.read_fwf(path, colspecs=[(0, 5), (6, 11), (12, 17), (18, 23), (24, 29)], 
                         names=['date', 'srad', 'tmax', 'tmin', 'rain'], skiprows=4)
        weather_dfs.append(df)

    df_weather = pd.concat(weather_dfs, ignore_index=True)
    df_weather.dropna(subset=['date'], inplace=True)
    df_weather['date'] = pd.to_datetime(df_weather['date'], format='%y%j')
    df_weather.drop_duplicates(subset='date', keep='first', inplace=True)
    df_weather.sort_values(by='date', inplace=True)

    weather_station = WeatherStation(insi='ITHY', lat=17.53, long=78.27, elev=0, tav=25.8, amp=11.8, table=df_weather)
    soil = SoilProfile.from_file('IBSG910085', soil_path)
    dssat_env = DSSAT(workspace)

    initial_conditions = filex.InitialConditions(pcr='SG', icdat=datetime(1980, 7, 3), icres=1300)
    planting = filex.Planting(pdate=datetime(1980, 6, 17), ppop=18, plds='R', plrs=45, pldp=5)
    
    # ===================================================================
    # SECTION 2: TREATMENT COMPARISON (Rainfed vs. Irrigated)
    # ===================================================================
    print("\n--- SECTION 2: Running Treatment Comparison (Rainfed vs. Irrigated) ---")
    
    field = filex.Field(id_field='ITHY0001', wsta=weather_station, id_soil=soil)
    
    # FINAL FIX: Added vbose='Y' to get all detailed columns in the daily output files.
    outputs_config = filex.SCOutputs(grout='Y', waout='Y', sumry='Y', vbose='Y')
    
    # Run Rainfed Treatment
    simulation_controls_rainfed = filex.SimulationControls(
        general=filex.SCGeneral(sdate=datetime(1980, 6, 17)),
        options=filex.SCOptions(water='Y', nitro='Y', symbi='N'),
        management=filex.SCManagement(irrig='N', ferti='N', harvs='M'),
        outputs=outputs_config
    )
    rainfed_summary = dssat_env.run_treatment(
        field=field,
        cultivar=crop.Sorghum('IB0026'),
        planting=planting,
        initial_conditions=initial_conditions,
        simulation_controls=simulation_controls_rainfed
    )
    rainfed_growth = dssat_env.output_tables.get('PlantGro').copy()

    # Run Irrigated Treatment
    irrigation = filex.Irrigation(
        efir=1, 
        table=[
            IrrigationEvent(idate=datetime(1980, 7, 15), irop='IR001', irval=20),
            IrrigationEvent(idate=datetime(1980, 8, 15), irop='IR001', irval=25),
        ]
    )
    simulation_controls_irrigated = filex.SimulationControls(
        general=filex.SCGeneral(sdate=datetime(1980, 6, 17)),
        options=filex.SCOptions(water='Y', nitro='Y', symbi='N'),
        management=filex.SCManagement(irrig='R', ferti='N', harvs='M'),
        outputs=outputs_config
    )
    irrigated_summary = dssat_env.run_treatment(
        field=field,
        cultivar=crop.Sorghum('IB0026'),
        planting=planting,
        initial_conditions=initial_conditions,
        simulation_controls=simulation_controls_irrigated,
        irrigation=irrigation
    )
    irrigated_growth = dssat_env.output_tables.get('PlantGro').copy()
    irrigated_water = dssat_env.output_tables.get('SoilWat').copy()

    # Create Comparison Plot with Harvest Day markers
    plt.figure(figsize=(10, 6))
    plt.plot(rainfed_growth.index, rainfed_growth['GWAD'], label='Rainfed')
    plt.plot(irrigated_growth.index, irrigated_growth['GWAD'], label='Irrigated', linestyle='--')
    
    pdate_dssattools = planting['pdate']
    planting_date = datetime(pdate_dssattools.year, pdate_dssattools.month, pdate_dssattools.day)
    
    rainfed_mat_dap = rainfed_summary['mat']
    irrigated_mat_dap = irrigated_summary['mat']
    rainfed_harvest_date = planting_date + timedelta(days=rainfed_mat_dap)
    irrigated_harvest_date = planting_date + timedelta(days=irrigated_mat_dap)
    
    plt.axvline(x=rainfed_harvest_date, color='red', linestyle=':', label=f'Rainfed Harvest ({rainfed_mat_dap} DAP)')
    plt.axvline(x=irrigated_harvest_date, color='blue', linestyle=':', label=f'Irrigated Harvest ({irrigated_mat_dap} DAP)')
    
    plt.title('Treatment Comparison: Grain Growth')
    plt.xlabel('Date')
    plt.ylabel('Grain Weight (kg/ha)')
    plt.legend()
    plt.grid(True)
    comp_path = os.path.join(workspace, 'treatment_comparison_growth.png')
    plt.savefig(comp_path)
    print(f"Saved treatment comparison plot to: {comp_path}")
    plt.close()
    
    # ===================================================================
    # SECTION 3: PARAMETER SENSITIVITY (Varying 'p1' coefficient)
    # ===================================================================
    print("\n--- SECTION 3: Running Parameter Sensitivity Analysis (P1 Coefficient) ---")
    
    p1_values = [350, 380, 410, 440, 470]
    sensitivity_results = []
    
    cultivar_sens = crop.Sorghum('IB0026')
    simulation_controls_sens = filex.SimulationControls(
        general=filex.SCGeneral(sdate=datetime(1980, 6, 17)),
        options=filex.SCOptions(water='Y', nitro='Y', symbi='N'),
        management=filex.SCManagement(irrig='N', ferti='N', harvs='M'),
        outputs=filex.SCOutputs(sumry='Y') 
    )
    
    for p1 in p1_values:
        cultivar_sens['p1'] = p1
        print(f"Running simulation for p1 = {p1}...")
        summary = dssat_env.run_treatment(
            field=field,
            cultivar=cultivar_sens,
            planting=planting,
            initial_conditions=initial_conditions,
            simulation_controls=simulation_controls_sens
        )
        summary['p1'] = p1
        sensitivity_results.append(summary)
        
    results_df = pd.DataFrame(sensitivity_results)

    # Create Sensitivity Plots
    plt.figure(figsize=(10, 6))
    plt.plot(results_df['p1'], results_df['harwt'], marker='o')
    plt.title('Sensitivity Analysis: Yield vs. P1 Coefficient')
    plt.xlabel('P1 Value (GDD)')
    plt.ylabel('Grain Weight at Harvest (kg/ha)')
    plt.grid(True)
    sens_yield_path = os.path.join(workspace, 'sensitivity_yield_vs_p1.png')
    plt.savefig(sens_yield_path)
    print(f"Saved sensitivity plot (Yield vs P1) to: {sens_yield_path}")
    plt.close()

    plt.figure(figsize=(10, 6))
    plt.plot(results_df['p1'], results_df['mat'], marker='o')
    plt.title('Sensitivity Analysis: Maturity vs. P1 Coefficient')
    plt.xlabel('P1 Value (GDD)')
    plt.ylabel('Days to Maturity (dap)')
    plt.grid(True)
    sens_mat_path = os.path.join(workspace, 'sensitivity_maturity_vs_p1.png')
    plt.savefig(sens_mat_path)
    print(f"Saved sensitivity plot (Maturity vs P1) to: {sens_mat_path}")
    plt.close()

    # ===================================================================
    # SECTION 4: DETAILED DAILY TIME-SERIES PLOTS
    # ===================================================================
    print("\n--- SECTION 4: Generating Detailed Daily Plots ---")

    # Re-use results from the irrigated run in Section 2
    plt.figure(figsize=(10, 6))
    plt.plot(irrigated_growth.index, irrigated_growth['LAID'])
    plt.title('Daily Leaf Area Index (Irrigated Treatment)')
    plt.xlabel('Date')
    plt.ylabel('Leaf Area Index (LAI)')
    plt.grid(True)
    plt.axvline(x=irrigated_harvest_date, color='red', linestyle='--', label=f'Harvest Day ({irrigated_mat_dap} DAP)')
    plt.legend()
    lai_path = os.path.join(workspace, 'daily_lai.png')
    plt.savefig(lai_path)
    print(f"Saved LAI plot to: {lai_path}")
    plt.close()
    
    # Plot for Daily Yield (Grain Growth Rate)
    irrigated_growth['daily_yield'] = irrigated_growth['GWAD'].diff().fillna(0).clip(lower=0)
    plt.figure(figsize=(10, 6))
    plt.plot(irrigated_growth['DAP'], irrigated_growth['daily_yield'])
    plt.title('Daily Yield (Grain Growth Rate)')
    plt.xlabel('Days After Planting (DAP)')
    plt.ylabel('Daily Increase in Grain Weight (kg/ha/day)')
    plt.grid(True)
    plt.axvline(x=irrigated_mat_dap, color='red', linestyle='--', label='Harvest Day')
    plt.legend()
    daily_yield_path = os.path.join(workspace, 'daily_yield.png')
    plt.savefig(daily_yield_path)
    print(f"Saved daily yield plot to: {daily_yield_path}")
    plt.close()

    # Plot for Total Accumulated Yield
    plt.figure(figsize=(10, 6))
    plt.plot(irrigated_growth['DAP'], irrigated_growth['GWAD'], label='Total Accumulated Grain')
    
    final_yield = irrigated_growth.loc[irrigated_growth.index == irrigated_harvest_date, 'GWAD'].values[0]
    
    plt.axvline(x=irrigated_mat_dap, color='red', linestyle='--', label=f'Harvest Day ({irrigated_mat_dap} DAP)')
    plt.scatter(irrigated_mat_dap, final_yield, color='red', zorder=5, s=100, label=f'Final Yield: {int(final_yield)} kg/ha')
    
    plt.title('Total Accumulated Grain Yield Over Time')
    plt.xlabel('Days After Planting (DAP)')
    plt.ylabel('Total Grain Weight (kg/ha)')
    plt.legend()
    plt.grid(True)
    cumulative_yield_path = os.path.join(workspace, 'cumulative_yield.png')
    plt.savefig(cumulative_yield_path)
    print(f"Saved cumulative yield plot to: {cumulative_yield_path}")
    plt.close()

    # Plot for Daily Water Input (Rain & Irrigation)
    if irrigated_water is not None and not irrigated_water.empty:
        df_weather_indexed = df_weather.set_index('date')
        df_water_input = irrigated_water.join(df_weather_indexed['rain']).fillna(0)
        df_water_input.rename(columns={'rain': 'RAIN'}, inplace=True)
        
        plt.figure(figsize=(12, 6))
        plt.plot(df_water_input.index, df_water_input['RAIN'], label='Rain', color='blue')
        
        if 'IRRIG' in df_water_input.columns:
            plt.bar(df_water_input.index, df_water_input['IRRIG'], label='Irrigation', color='purple', width=1.0)
        else:
            print("\nWARNING: 'IRRIG' column not found in SoilWat.OUT. Irrigation will not be plotted.")
            plt.bar([], [], label='Irrigation (Not Found)', color='purple')

        plt.title('Daily Water Input (Rain & Irrigation)')
        plt.xlabel('Date')
        plt.ylabel('Water (mm/day)')
        plt.legend()
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.tight_layout()
        water_input_path = os.path.join(workspace, 'daily_water_input.png')
        plt.savefig(water_input_path)
        print(f"Saved daily water input plot to: {water_input_path}")
        plt.close()


if __name__ == "__main__":
    run_all_analyses_and_visualizations()

Attempting to install package using 'python3'...

--- SECTION 1: Initializing Workspace and Data ---
DSSAT workspace created at: d:\Final-Year-Project\dssat_simulation_workspace
Successfully downloaded SOIL.SOL
Successfully downloaded ITHY8001.WTH
Successfully downloaded ITHY8101.WTH
d:\Final-Year-Project\dssat_simulation_workspace created.

--- SECTION 2: Running Treatment Comparison (Rainfed vs. Irrigated) ---

RUN    TRT FLO MAT TOPWT HARWT  RAIN  TIRR   CET  PESW  TNUP  TNLF   TSON TSOC
           dap dap kg/ha kg/ha    mm    mm    mm    mm kg/ha kg/ha  kg/ha t/ha
  1 SG   1  67 109  1985   612   592     0   307   130    15     8  13377   95

RUN    TRT FLO MAT TOPWT HARWT  RAIN  TIRR   CET  PESW  TNUP  TNLF   TSON TSOC
           dap dap kg/ha kg/ha    mm    mm    mm    mm kg/ha kg/ha  kg/ha t/ha
  1 SG   1  67 109  1839   565   592    45   312   130    14     8  13377   95
Saved treatment comparison plot to: d:\Final-Year-Project\dssat_simulation_workspace\treatment_comparison_gr

In [42]:
import os
import urllib.request
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import subprocess
import sys

def install_package(package):
    """Installs a package using pip, trying 'python3' and then 'python'."""
    try:
        print("Attempting to install package using 'python3'...")
        subprocess.check_call(["python3", "-m", "pip", "install", package])
        print(f"Successfully installed {package} with 'python3'")
    except FileNotFoundError:
        print("'python3' not found. Trying 'python' instead...")
        try:
            subprocess.check_call(["python", "-m", "pip", "install", package])
            print(f"Successfully installed {package} with 'python'")
        except FileNotFoundError:
            print("Error: Neither 'python3' nor 'python' command was found in the environment's PATH.")
            raise
        except subprocess.CalledProcessError as e:
            print(f"Error installing {package} with 'python': {e}")
            raise
    except subprocess.CalledProcessError as e:
        print(f"Error installing {package} with 'python3': {e}")
        raise


def download_file(url, target_path):
    """Downloads a file from a URL and saves it locally."""
    try:
        with urllib.request.urlopen(url) as response, open(target_path, 'wb') as out_file:
            data = response.read()
            out_file.write(data)
        print(f"Successfully downloaded {os.path.basename(target_path)}")
    except Exception as e:
        print(f"Error downloading {url}: {e}")
        raise

def run_all_analyses_and_visualizations():
    """
    Sets up a DSSAT environment, runs multiple analysis scenarios,
    and generates all relevant visualizations.
    """
    # First, ensure DSSATTools is installed
    install_package('DSSATTools')
    
    # Now import from the newly installed package
    from DSSATTools import (
        crop,
        WeatherStation,
        SoilProfile,
        filex,
        DSSAT
    )
    from DSSATTools.filex import IrrigationEvent, FertilizerEvent

    # ===================================================================
    # SECTION 1: BASIC SETUP AND DATA ACQUISITION
    # ===================================================================
    print("\n--- SECTION 1: Initializing Workspace and Data ---")
    workspace = os.path.join(os.getcwd(), 'dssat_simulation_workspace')
    os.makedirs(workspace, exist_ok=True)
    print(f"DSSAT workspace created at: {workspace}")

    soil_path = os.path.join(workspace, 'SOIL.SOL')
    download_file('https://raw.githubusercontent.com/DSSAT/dssat-csm-data/develop/Soil/SOIL.SOL', soil_path)

    weather_dfs = []
    for year in ['80', '81']:
        filename = f'ITHY{year}01.WTH'
        url = f'https://raw.githubusercontent.com/DSSAT/dssat-csm-data/develop/Weather/{filename}'
        path = os.path.join(workspace, filename)
        download_file(url, path)
        df = pd.read_fwf(path, colspecs=[(0, 5), (6, 11), (12, 17), (18, 23), (24, 29)], 
                         names=['date', 'srad', 'tmax', 'tmin', 'rain'], skiprows=4)
        weather_dfs.append(df)

    df_weather = pd.concat(weather_dfs, ignore_index=True)
    df_weather.dropna(subset=['date'], inplace=True)
    df_weather['date'] = pd.to_datetime(df_weather['date'], format='%y%j')
    df_weather.drop_duplicates(subset='date', keep='first', inplace=True)
    df_weather.sort_values(by='date', inplace=True)

    weather_station = WeatherStation(insi='ITHY', lat=17.53, long=78.27, elev=0, tav=25.8, amp=11.8, table=df_weather)
    soil = SoilProfile.from_file('IBSG910085', soil_path)
    dssat_env = DSSAT(workspace)

    initial_conditions = filex.InitialConditions(pcr='SG', icdat=datetime(1980, 7, 3), icres=1300)
    planting = filex.Planting(pdate=datetime(1980, 6, 17), ppop=18, plds='R', plrs=45, pldp=5)
    
    # ===================================================================
    # SECTION 2: TREATMENT COMPARISON (Rainfed vs. Irrigated)
    # ===================================================================
    print("\n--- SECTION 2: Running Treatment Comparison (Rainfed vs. Irrigated) ---")
    
    field = filex.Field(id_field='ITHY0001', wsta=weather_station, id_soil=soil)
    outputs_config = filex.SCOutputs(grout='Y', waout='Y', sumry='Y', vbose='Y')
    
    # Run Rainfed Treatment
    simulation_controls_rainfed = filex.SimulationControls(
        general=filex.SCGeneral(sdate=datetime(1980, 6, 17)),
        options=filex.SCOptions(water='Y', nitro='Y', symbi='N'),
        management=filex.SCManagement(irrig='N', ferti='N', harvs='M'),
        outputs=outputs_config
    )
    rainfed_summary = dssat_env.run_treatment(
        field=field,
        cultivar=crop.Sorghum('IB0026'),
        planting=planting,
        initial_conditions=initial_conditions,
        simulation_controls=simulation_controls_rainfed
    )
    rainfed_growth = dssat_env.output_tables.get('PlantGro').copy()

    # Create and read irrigation schedule from a CSV file
    irrigation_csv_path = os.path.join(workspace, 'irrigation_schedule.csv')
    irrigation_data = """date,amount
1980-07-15,20
1980-08-15,25
1980-09-01,15
"""
    with open(irrigation_csv_path, 'w') as f:
        f.write(irrigation_data)
    print(f"Created sample irrigation schedule at: {irrigation_csv_path}")
    
    schedule_df = pd.read_csv(irrigation_csv_path, parse_dates=['date'])
    
    irrigation_events = [
        IrrigationEvent(idate=row['date'], irval=row['amount'], irop='IR001')
        for index, row in schedule_df.iterrows()
    ]

    irrigation = filex.Irrigation(efir=1, table=irrigation_events)

    # Run Irrigated Treatment using the schedule from the file
    simulation_controls_irrigated = filex.SimulationControls(
        general=filex.SCGeneral(sdate=datetime(1980, 6, 17)),
        options=filex.SCOptions(water='Y', nitro='Y', symbi='N'),
        management=filex.SCManagement(irrig='R', ferti='N', harvs='M'),
        outputs=outputs_config
    )
    irrigated_summary = dssat_env.run_treatment(
        field=field,
        cultivar=crop.Sorghum('IB0026'),
        planting=planting,
        initial_conditions=initial_conditions,
        simulation_controls=simulation_controls_irrigated,
        irrigation=irrigation
    )
    irrigated_growth = dssat_env.output_tables.get('PlantGro').copy()
    irrigated_water = dssat_env.output_tables.get('SoilWat').copy()

    # Create Comparison Plot with Harvest Day markers
    plt.figure(figsize=(10, 6))
    plt.plot(rainfed_growth.index, rainfed_growth['GWAD'], label='Rainfed')
    plt.plot(irrigated_growth.index, irrigated_growth['GWAD'], label='Irrigated', linestyle='--')
    
    pdate_dssattools = planting['pdate']
    planting_date = datetime(pdate_dssattools.year, pdate_dssattools.month, pdate_dssattools.day)
    
    rainfed_mat_dap = rainfed_summary['mat']
    irrigated_mat_dap = irrigated_summary['mat']
    rainfed_harvest_date = planting_date + timedelta(days=rainfed_mat_dap)
    irrigated_harvest_date = planting_date + timedelta(days=irrigated_mat_dap)
    
    plt.axvline(x=rainfed_harvest_date, color='red', linestyle=':', label=f'Rainfed Harvest ({rainfed_mat_dap} DAP)')
    plt.axvline(x=irrigated_harvest_date, color='blue', linestyle=':', label=f'Irrigated Harvest ({irrigated_mat_dap} DAP)')
    
    plt.title('Treatment Comparison: Grain Growth')
    plt.xlabel('Date')
    plt.ylabel('Grain Weight (kg/ha)')
    plt.legend()
    plt.grid(True)
    comp_path = os.path.join(workspace, 'treatment_comparison_growth.png')
    plt.savefig(comp_path)
    print(f"Saved treatment comparison plot to: {comp_path}")
    plt.close()
    
    # ===================================================================
    # SECTION 3: PARAMETER SENSITIVITY (Varying 'p1' coefficient)
    # ===================================================================
    print("\n--- SECTION 3: Running Parameter Sensitivity Analysis (P1 Coefficient) ---")
    
    p1_values = [350, 380, 410, 440, 470]
    sensitivity_results = []
    
    cultivar_sens = crop.Sorghum('IB0026')
    simulation_controls_sens = filex.SimulationControls(
        general=filex.SCGeneral(sdate=datetime(1980, 6, 17)),
        options=filex.SCOptions(water='Y', nitro='Y', symbi='N'),
        management=filex.SCManagement(irrig='N', ferti='N', harvs='M'),
        outputs=filex.SCOutputs(sumry='Y') 
    )
    
    for p1 in p1_values:
        cultivar_sens['p1'] = p1
        print(f"Running simulation for p1 = {p1}...")
        summary = dssat_env.run_treatment(
            field=field,
            cultivar=cultivar_sens,
            planting=planting,
            initial_conditions=initial_conditions,
            simulation_controls=simulation_controls_sens
        )
        summary['p1'] = p1
        sensitivity_results.append(summary)
        
    results_df = pd.DataFrame(sensitivity_results)

    # Create Sensitivity Plots
    plt.figure(figsize=(10, 6))
    plt.plot(results_df['p1'], results_df['harwt'], marker='o')
    plt.title('Sensitivity Analysis: Yield vs. P1 Coefficient')
    plt.xlabel('P1 Value (GDD)')
    plt.ylabel('Grain Weight at Harvest (kg/ha)')
    plt.grid(True)
    sens_yield_path = os.path.join(workspace, 'sensitivity_yield_vs_p1.png')
    plt.savefig(sens_yield_path)
    print(f"Saved sensitivity plot (Yield vs P1) to: {sens_yield_path}")
    plt.close()

    plt.figure(figsize=(10, 6))
    plt.plot(results_df['p1'], results_df['mat'], marker='o')
    plt.title('Sensitivity Analysis: Maturity vs. P1 Coefficient')
    plt.xlabel('P1 Value (GDD)')
    plt.ylabel('Days to Maturity (dap)')
    plt.grid(True)
    sens_mat_path = os.path.join(workspace, 'sensitivity_maturity_vs_p1.png')
    plt.savefig(sens_mat_path)
    print(f"Saved sensitivity plot (Maturity vs P1) to: {sens_mat_path}")
    plt.close()

    # ===================================================================
    # SECTION 4: DETAILED DAILY TIME-SERIES PLOTS
    # ===================================================================
    print("\n--- SECTION 4: Generating Detailed Daily Plots ---")

    # Re-use results from the irrigated run in Section 2
    plt.figure(figsize=(10, 6))
    plt.plot(irrigated_growth.index, irrigated_growth['LAID'])
    plt.title('Daily Leaf Area Index (Irrigated Treatment)')
    plt.xlabel('Date')
    plt.ylabel('Leaf Area Index (LAI)')
    plt.grid(True)
    plt.axvline(x=irrigated_harvest_date, color='red', linestyle='--', label=f'Harvest Day ({irrigated_mat_dap} DAP)')
    plt.legend()
    lai_path = os.path.join(workspace, 'daily_lai.png')
    plt.savefig(lai_path)
    print(f"Saved LAI plot to: {lai_path}")
    plt.close()
    
    # Plot for Daily Yield (Grain Growth Rate)
    irrigated_growth['daily_yield'] = irrigated_growth['GWAD'].diff().fillna(0).clip(lower=0)
    plt.figure(figsize=(10, 6))
    plt.plot(irrigated_growth['DAP'], irrigated_growth['daily_yield'])
    plt.title('Daily Yield (Grain Growth Rate)')
    plt.xlabel('Days After Planting (DAP)')
    plt.ylabel('Daily Increase in Grain Weight (kg/ha/day)')
    plt.grid(True)
    plt.axvline(x=irrigated_mat_dap, color='red', linestyle='--', label='Harvest Day')
    plt.legend()
    daily_yield_path = os.path.join(workspace, 'daily_yield.png')
    plt.savefig(daily_yield_path)
    print(f"Saved daily yield plot to: {daily_yield_path}")
    plt.close()

    # Plot for Total Accumulated Yield
    plt.figure(figsize=(10, 6))
    plt.plot(irrigated_growth['DAP'], irrigated_growth['GWAD'], label='Total Accumulated Grain')
    
    final_yield = irrigated_growth.loc[irrigated_growth.index == irrigated_harvest_date, 'GWAD'].values[0]
    
    plt.axvline(x=irrigated_mat_dap, color='red', linestyle='--', label=f'Harvest Day ({irrigated_mat_dap} DAP)')
    plt.scatter(irrigated_mat_dap, final_yield, color='red', zorder=5, s=100, label=f'Final Yield: {int(final_yield)} kg/ha')
    
    plt.title('Total Accumulated Grain Yield Over Time')
    plt.xlabel('Days After Planting (DAP)')
    plt.ylabel('Total Grain Weight (kg/ha)')
    plt.legend()
    plt.grid(True)
    cumulative_yield_path = os.path.join(workspace, 'cumulative_yield.png')
    plt.savefig(cumulative_yield_path)
    print(f"Saved cumulative yield plot to: {cumulative_yield_path}")
    plt.close()

    # MODIFIED: Plot for Daily Water Input using both weather data and schedule data
    # -----------------------------------------------------------------------------
    # We use the full weather data for the rain, and the schedule DataFrame for irrigation
    df_weather_indexed = df_weather.set_index('date')
    
    plt.figure(figsize=(12, 6))
    # Plot rain as a blue line from the weather data
    plt.plot(df_weather_indexed.index, df_weather_indexed['rain'], label='Rain', color='blue')
    
    # Plot irrigation as purple bars from the schedule file we created
    plt.bar(schedule_df['date'], schedule_df['amount'], label='Irrigation', color='purple', width=1.0)

    plt.title('Daily Water Input (Rain & Irrigation)')
    plt.xlabel('Date')
    plt.ylabel('Water (mm/day)')
    # Limit x-axis to the growing season for clarity
    plt.xlim(planting_date, irrigated_harvest_date + timedelta(days=5))
    plt.legend()
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    water_input_path = os.path.join(workspace, 'daily_water_input.png')
    plt.savefig(water_input_path)
    print(f"Saved daily water input plot to: {water_input_path}")
    plt.close()


if __name__ == "__main__":
    run_all_analyses_and_visualizations()

Attempting to install package using 'python3'...

--- SECTION 1: Initializing Workspace and Data ---
DSSAT workspace created at: d:\Final-Year-Project\dssat_simulation_workspace
Successfully downloaded SOIL.SOL
Successfully downloaded ITHY8001.WTH
Successfully downloaded ITHY8101.WTH
d:\Final-Year-Project\dssat_simulation_workspace created.

--- SECTION 2: Running Treatment Comparison (Rainfed vs. Irrigated) ---

RUN    TRT FLO MAT TOPWT HARWT  RAIN  TIRR   CET  PESW  TNUP  TNLF   TSON TSOC
           dap dap kg/ha kg/ha    mm    mm    mm    mm kg/ha kg/ha  kg/ha t/ha
  1 SG   1  67 109  1985   612   592     0   307   130    15     8  13377   95
Created sample irrigation schedule at: d:\Final-Year-Project\dssat_simulation_workspace\irrigation_schedule.csv

RUN    TRT FLO MAT TOPWT HARWT  RAIN  TIRR   CET  PESW  TNUP  TNLF   TSON TSOC
           dap dap kg/ha kg/ha    mm    mm    mm    mm kg/ha kg/ha  kg/ha t/ha
  1 SG   1  67 109  1824   553   592    60   313   131    14     8  13377  

In [36]:
import os
import urllib.request
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import subprocess
import sys

def install_package(package):
    """Installs a package using pip, trying 'python3' and then 'python'."""
    try:
        print("Attempting to install package using 'python3'...")
        subprocess.check_call(["python3", "-m", "pip", "install", package])
        print(f"Successfully installed {package} with 'python3'")
    except FileNotFoundError:
        print("'python3' not found. Trying 'python' instead...")
        try:
            subprocess.check_call(["python", "-m", "pip", "install", package])
            print(f"Successfully installed {package} with 'python'")
        except FileNotFoundError:
            print("Error: Neither 'python3' nor 'python' command was found in the environment's PATH.")
            raise
        except subprocess.CalledProcessError as e:
            print(f"Error installing {package} with 'python': {e}")
            raise
    except subprocess.CalledProcessError as e:
        print(f"Error installing {package} with 'python3': {e}")
        raise


def download_file(url, target_path):
    """Downloads a file from a URL and saves it locally."""
    try:
        with urllib.request.urlopen(url) as response, open(target_path, 'wb') as out_file:
            data = response.read()
            out_file.write(data)
        print(f"Successfully downloaded {os.path.basename(target_path)}")
    except Exception as e:
        print(f"Error downloading {url}: {e}")
        raise

def run_all_analyses_and_visualizations():
    """
    Sets up a DSSAT environment, runs multiple analysis scenarios,
    and generates all relevant visualizations.
    """
    # First, ensure DSSATTools is installed
    install_package('DSSATTools')
    
    # Now import from the newly installed package
    from DSSATTools import (
        crop,
        WeatherStation,
        SoilProfile,
        filex,
        DSSAT
    )
    from DSSATTools.filex import IrrigationEvent, FertilizerEvent

    # ===================================================================
    # SECTION 1: BASIC SETUP AND DATA ACQUISITION
    # ===================================================================
    print("\n--- SECTION 1: Initializing Workspace and Data ---")
    workspace = os.path.join(os.getcwd(), 'dssat_simulation_workspace')
    os.makedirs(workspace, exist_ok=True)
    print(f"DSSAT workspace created at: {workspace}")

    soil_path = os.path.join(workspace, 'SOIL.SOL')
    download_file('https://raw.githubusercontent.com/DSSAT/dssat-csm-data/develop/Soil/SOIL.SOL', soil_path)

    weather_dfs = []
    for year in ['80', '81']:
        filename = f'ITHY{year}01.WTH'
        url = f'https://raw.githubusercontent.com/DSSAT/dssat-csm-data/develop/Weather/{filename}'
        path = os.path.join(workspace, filename)
        download_file(url, path)
        df = pd.read_fwf(path, colspecs=[(0, 5), (6, 11), (12, 17), (18, 23), (24, 29)], 
                         names=['date', 'srad', 'tmax', 'tmin', 'rain'], skiprows=4)
        weather_dfs.append(df)

    df_weather = pd.concat(weather_dfs, ignore_index=True)
    df_weather.dropna(subset=['date'], inplace=True)
    df_weather['date'] = pd.to_datetime(df_weather['date'], format='%y%j')
    df_weather.drop_duplicates(subset='date', keep='first', inplace=True)
    df_weather.sort_values(by='date', inplace=True)

    weather_station = WeatherStation(insi='ITHY', lat=17.53, long=78.27, elev=0, tav=25.8, amp=11.8, table=df_weather)
    soil = SoilProfile.from_file('IBSG910085', soil_path)
    dssat_env = DSSAT(workspace)

    initial_conditions = filex.InitialConditions(pcr='SG', icdat=datetime(1980, 7, 3), icres=1300)
    planting = filex.Planting(pdate=datetime(1980, 6, 17), ppop=18, plds='R', plrs=45, pldp=5)
    
    # ===================================================================
    # SECTION 2: TREATMENT COMPARISON (Rainfed vs. Irrigated)
    # ===================================================================
    print("\n--- SECTION 2: Running Treatment Comparison (Rainfed vs. Irrigated) ---")
    
    field = filex.Field(id_field='ITHY0001', wsta=weather_station, id_soil=soil)
    outputs_config = filex.SCOutputs(grout='Y', waout='Y', sumry='Y', vbose='Y')
    
    # Run Rainfed Treatment
    simulation_controls_rainfed = filex.SimulationControls(
        general=filex.SCGeneral(sdate=datetime(1980, 6, 17)),
        options=filex.SCOptions(water='Y', nitro='Y', symbi='N'),
        management=filex.SCManagement(irrig='N', ferti='N', harvs='M'),
        outputs=outputs_config
    )
    rainfed_summary = dssat_env.run_treatment(
        field=field,
        cultivar=crop.Sorghum('IB0026'),
        planting=planting,
        initial_conditions=initial_conditions,
        simulation_controls=simulation_controls_rainfed
    )
    rainfed_growth = dssat_env.output_tables.get('PlantGro').copy()

    # Create and read irrigation schedule from a CSV file
    irrigation_csv_path = os.path.join(workspace, 'irrigation_schedule.csv')
    irrigation_data = """date,amount
1980-07-15,20
1980-08-15,25
1980-09-01,15
"""
    with open(irrigation_csv_path, 'w') as f:
        f.write(irrigation_data)
    print(f"Created sample irrigation schedule at: {irrigation_csv_path}")
    
    schedule_df = pd.read_csv(irrigation_csv_path, parse_dates=['date'])
    
    irrigation_events = [
        IrrigationEvent(idate=row['date'], irval=row['amount'], irop='IR001')
        for index, row in schedule_df.iterrows()
    ]

    irrigation = filex.Irrigation(efir=1, table=irrigation_events)

    # Run Irrigated Treatment using the schedule from the file
    simulation_controls_irrigated = filex.SimulationControls(
        general=filex.SCGeneral(sdate=datetime(1980, 6, 17)),
        options=filex.SCOptions(water='Y', nitro='Y', symbi='N'),
        management=filex.SCManagement(irrig='R', ferti='N', harvs='M'),
        outputs=outputs_config
    )
    irrigated_summary = dssat_env.run_treatment(
        field=field,
        cultivar=crop.Sorghum('IB0026'),
        planting=planting,
        initial_conditions=initial_conditions,
        simulation_controls=simulation_controls_irrigated,
        irrigation=irrigation
    )
    irrigated_growth = dssat_env.output_tables.get('PlantGro').copy()
    irrigated_water = dssat_env.output_tables.get('SoilWat').copy()

    # Create Comparison Plot with Harvest Day markers
    plt.figure(figsize=(10, 6))
    plt.plot(rainfed_growth.index, rainfed_growth['GWAD'], label='Rainfed')
    plt.plot(irrigated_growth.index, irrigated_growth['GWAD'], label='Irrigated', linestyle='--')
    
    pdate_dssattools = planting['pdate']
    planting_date = datetime(pdate_dssattools.year, pdate_dssattools.month, pdate_dssattools.day)
    
    rainfed_mat_dap = rainfed_summary['mat']
    irrigated_mat_dap = irrigated_summary['mat']
    rainfed_harvest_date = planting_date + timedelta(days=rainfed_mat_dap)
    irrigated_harvest_date = planting_date + timedelta(days=irrigated_mat_dap)
    
    plt.axvline(x=rainfed_harvest_date, color='red', linestyle=':', label=f'Rainfed Harvest ({rainfed_mat_dap} DAP)')
    plt.axvline(x=irrigated_harvest_date, color='blue', linestyle=':', label=f'Irrigated Harvest ({irrigated_mat_dap} DAP)')
    
    plt.title('Treatment Comparison: Grain Growth')
    plt.xlabel('Date')
    plt.ylabel('Grain Weight (kg/ha)')
    plt.legend()
    plt.grid(True)
    comp_path = os.path.join(workspace, 'treatment_comparison_growth.png')
    plt.savefig(comp_path)
    print(f"Saved treatment comparison plot to: {comp_path}")
    plt.close()
    
    # ===================================================================
    # SECTION 3: PARAMETER SENSITIVITY (Varying 'p1' coefficient)
    # ===================================================================
    print("\n--- SECTION 3: Running Parameter Sensitivity Analysis (P1 Coefficient) ---")
    
    p1_values = [350, 380, 410, 440, 470]
    sensitivity_results = []
    
    cultivar_sens = crop.Sorghum('IB0026')
    simulation_controls_sens = filex.SimulationControls(
        general=filex.SCGeneral(sdate=datetime(1980, 6, 17)),
        options=filex.SCOptions(water='Y', nitro='Y', symbi='N'),
        management=filex.SCManagement(irrig='N', ferti='N', harvs='M'),
        outputs=filex.SCOutputs(sumry='Y') 
    )
    
    for p1 in p1_values:
        cultivar_sens['p1'] = p1
        print(f"Running simulation for p1 = {p1}...")
        summary = dssat_env.run_treatment(
            field=field,
            cultivar=cultivar_sens,
            planting=planting,
            initial_conditions=initial_conditions,
            simulation_controls=simulation_controls_sens
        )
        summary['p1'] = p1
        sensitivity_results.append(summary)
        
    results_df = pd.DataFrame(sensitivity_results)

    # Create Sensitivity Plots
    plt.figure(figsize=(10, 6))
    plt.plot(results_df['p1'], results_df['harwt'], marker='o')
    plt.title('Sensitivity Analysis: Yield vs. P1 Coefficient')
    plt.xlabel('P1 Value (GDD)')
    plt.ylabel('Grain Weight at Harvest (kg/ha)')
    plt.grid(True)
    sens_yield_path = os.path.join(workspace, 'sensitivity_yield_vs_p1.png')
    plt.savefig(sens_yield_path)
    print(f"Saved sensitivity plot (Yield vs P1) to: {sens_yield_path}")
    plt.close()

    plt.figure(figsize=(10, 6))
    plt.plot(results_df['p1'], results_df['mat'], marker='o')
    plt.title('Sensitivity Analysis: Maturity vs. P1 Coefficient')
    plt.xlabel('P1 Value (GDD)')
    plt.ylabel('Days to Maturity (dap)')
    plt.grid(True)
    sens_mat_path = os.path.join(workspace, 'sensitivity_maturity_vs_p1.png')
    plt.savefig(sens_mat_path)
    print(f"Saved sensitivity plot (Maturity vs P1) to: {sens_mat_path}")
    plt.close()

    # ===================================================================
    # SECTION 4: DETAILED DAILY TIME-SERIES PLOTS
    # ===================================================================
    print("\n--- SECTION 4: Generating Detailed Daily Plots ---")

    # Re-use results from the irrigated run in Section 2
    plt.figure(figsize=(10, 6))
    plt.plot(irrigated_growth.index, irrigated_growth['LAID'])
    plt.title('Daily Leaf Area Index (Irrigated Treatment)')
    plt.xlabel('Date')
    plt.ylabel('Leaf Area Index (LAI)')
    plt.grid(True)
    plt.axvline(x=irrigated_harvest_date, color='red', linestyle='--', label=f'Harvest Day ({irrigated_mat_dap} DAP)')
    plt.legend()
    lai_path = os.path.join(workspace, 'daily_lai.png')
    plt.savefig(lai_path)
    print(f"Saved LAI plot to: {lai_path}")
    plt.close()
    
    # Plot for Daily Yield (Grain Growth Rate)
    irrigated_growth['daily_yield'] = irrigated_growth['GWAD'].diff().fillna(0).clip(lower=0)
    plt.figure(figsize=(10, 6))
    plt.plot(irrigated_growth['DAP'], irrigated_growth['daily_yield'])
    plt.title('Daily Yield (Grain Growth Rate)')
    plt.xlabel('Days After Planting (DAP)')
    plt.ylabel('Daily Increase in Grain Weight (kg/ha/day)')
    plt.grid(True)
    plt.axvline(x=irrigated_mat_dap, color='red', linestyle='--', label='Harvest Day')
    plt.legend()
    daily_yield_path = os.path.join(workspace, 'daily_yield.png')
    plt.savefig(daily_yield_path)
    print(f"Saved daily yield plot to: {daily_yield_path}")
    plt.close()

    # Plot for Total Accumulated Yield
    plt.figure(figsize=(10, 6))
    plt.plot(irrigated_growth['DAP'], irrigated_growth['GWAD'], label='Total Accumulated Grain')
    
    final_yield = irrigated_growth.loc[irrigated_growth.index == irrigated_harvest_date, 'GWAD'].values[0]
    
    plt.axvline(x=irrigated_mat_dap, color='red', linestyle='--', label=f'Harvest Day ({irrigated_mat_dap} DAP)')
    plt.scatter(irrigated_mat_dap, final_yield, color='red', zorder=5, s=100, label=f'Final Yield: {int(final_yield)} kg/ha')
    
    plt.title('Total Accumulated Grain Yield Over Time')
    plt.xlabel('Days After Planting (DAP)')
    plt.ylabel('Total Grain Weight (kg/ha)')
    plt.legend()
    plt.grid(True)
    cumulative_yield_path = os.path.join(workspace, 'cumulative_yield.png')
    plt.savefig(cumulative_yield_path)
    print(f"Saved cumulative yield plot to: {cumulative_yield_path}")
    plt.close()

    # MODIFIED: Plot for Daily Water Input (Rain & Irrigation as Lines with negative y-axis)
    # --------------------------------------------------------------------------------------
    df_weather_indexed = df_weather.set_index('date')
    
    # Create a DataFrame for plotting that covers the whole simulation period
    plot_dates = pd.date_range(start=planting_date, end=irrigated_harvest_date + timedelta(days=5))
    df_water_plot = pd.DataFrame(index=plot_dates)

    # Join rain data
    df_water_plot = df_water_plot.join(df_weather_indexed['rain'])
    df_water_plot.rename(columns={'rain': 'RAIN'}, inplace=True)
    df_water_plot['RAIN'].fillna(0, inplace=True)
    
    # Join irrigation data from the schedule
    irrigation_series = schedule_df.set_index('date')['amount']
    df_water_plot = df_water_plot.join(irrigation_series)
    df_water_plot.rename(columns={'amount': 'IRRIGATION'}, inplace=True)
    df_water_plot['IRRIGATION'].fillna(0, inplace=True)

    plt.figure(figsize=(12, 6))
    # Plot rain and irrigation as separate lines
    plt.plot(df_water_plot.index, df_water_plot['RAIN'], label='Rain', color='blue')
    plt.plot(df_water_plot.index, df_water_plot['IRRIGATION'], label='Irrigation', color='purple', linestyle='--')

    plt.title('Daily Water Input (Rain & Irrigation)')
    plt.xlabel('Date')
    plt.ylabel('Water (mm/day)')
    
    # Set Y-axis to show a negative portion
    max_water = max(df_water_plot['RAIN'].max(), df_water_plot['IRRIGATION'].max())
    plt.ylim(-5, max_water * 1.1)
    
    plt.legend()
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    water_input_path = os.path.join(workspace, 'daily_water_input.png')
    plt.savefig(water_input_path)
    print(f"Saved daily water input plot to: {water_input_path}")
    plt.close()


if __name__ == "__main__":
    run_all_analyses_and_visualizations()

Attempting to install package using 'python3'...

--- SECTION 1: Initializing Workspace and Data ---
DSSAT workspace created at: d:\Final-Year-Project\dssat_simulation_workspace
Successfully downloaded SOIL.SOL
Successfully downloaded ITHY8001.WTH
Successfully downloaded ITHY8101.WTH
d:\Final-Year-Project\dssat_simulation_workspace created.

--- SECTION 2: Running Treatment Comparison (Rainfed vs. Irrigated) ---

RUN    TRT FLO MAT TOPWT HARWT  RAIN  TIRR   CET  PESW  TNUP  TNLF   TSON TSOC
           dap dap kg/ha kg/ha    mm    mm    mm    mm kg/ha kg/ha  kg/ha t/ha
  1 SG   1  67 109  1985   612   592     0   307   130    15     8  13377   95
Created sample irrigation schedule at: d:\Final-Year-Project\dssat_simulation_workspace\irrigation_schedule.csv

RUN    TRT FLO MAT TOPWT HARWT  RAIN  TIRR   CET  PESW  TNUP  TNLF   TSON TSOC
           dap dap kg/ha kg/ha    mm    mm    mm    mm kg/ha kg/ha  kg/ha t/ha
  1 SG   1  67 109  1824   553   592    60   313   131    14     8  13377  

C:\Users\gurup\AppData\Local\Temp\ipykernel_18396\2492923697.py:300: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_water_plot['RAIN'].fillna(0, inplace=True)
C:\Users\gurup\AppData\Local\Temp\ipykernel_18396\2492923697.py:306: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For ex

In [ ]:
import os
import urllib.request
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import subprocess
import sys

def install_package(package):
    """Installs a package using pip, trying 'python3' and then 'python'."""
    try:
        print("Attempting to install package using 'python3'...")
        subprocess.check_call(["python3", "-m", "pip", "install", package])
        print(f"Successfully installed {package} with 'python3'")
    except FileNotFoundError:
        print("'python3' not found. Trying 'python' instead...")
        try:
            subprocess.check_call(["python", "-m", "pip", "install", package])
            print(f"Successfully installed {package} with 'python'")
        except FileNotFoundError:
            print("Error: Neither 'python3' nor 'python' command was found in the environment's PATH.")
            raise
        except subprocess.CalledProcessError as e:
            print(f"Error installing {package} with 'python': {e}")
            raise
    except subprocess.CalledProcessError as e:
        print(f"Error installing {package} with 'python3': {e}")
        raise


def download_file(url, target_path):
    """Downloads a file from a URL and saves it locally."""
    try:
        with urllib.request.urlopen(url) as response, open(target_path, 'wb') as out_file:
            data = response.read()
            out_file.write(data)
        print(f"Successfully downloaded {os.path.basename(target_path)}")
    except Exception as e:
        print(f"Error downloading {url}: {e}")
        raise

def run_all_analyses_and_visualizations():
    """
    Sets up a DSSAT environment, runs multiple analysis scenarios,
    and generates all relevant visualizations.
    """
    # First, ensure DSSATTools is installed
    install_package('DSSATTools')
    
    # Now import from the newly installed package
    from DSSATTools import (
        crop,
        WeatherStation,
        SoilProfile,
        filex,
        DSSAT
    )
    from DSSATTools.filex import IrrigationEvent, FertilizerEvent

    # ===================================================================
    # SECTION 1: BASIC SETUP AND DATA ACQUISITION
    # ===================================================================
    print("\n--- SECTION 1: Initializing Workspace and Data ---")
    workspace = os.path.join(os.getcwd(), 'dssat_simulation_workspace')
    os.makedirs(workspace, exist_ok=True)
    print(f"DSSAT workspace created at: {workspace}")

    soil_path = os.path.join(workspace, 'SOIL.SOL')
    download_file('https://raw.githubusercontent.com/DSSAT/dssat-csm-data/develop/Soil/SOIL.SOL', soil_path)

    weather_dfs = []
    for year in ['80', '81']:
        filename = f'ITHY{year}01.WTH'
        url = f'https://raw.githubusercontent.com/DSSAT/dssat-csm-data/develop/Weather/{filename}'
        path = os.path.join(workspace, filename)
        download_file(url, path)
        df = pd.read_fwf(path, colspecs=[(0, 5), (6, 11), (12, 17), (18, 23), (24, 29)], 
                        names=['date', 'srad', 'tmax', 'tmin', 'rain'], skiprows=4)
        weather_dfs.append(df)

    df_weather = pd.concat(weather_dfs, ignore_index=True)
    df_weather.dropna(subset=['date'], inplace=True)
    df_weather['date'] = pd.to_datetime(df_weather['date'], format='%y%j')
    df_weather.drop_duplicates(subset='date', keep='first', inplace=True)
    df_weather.sort_values(by='date', inplace=True)

    weather_station = WeatherStation(insi='ITHY', lat=17.53, long=78.27, elev=0, tav=25.8, amp=11.8, table=df_weather)
    soil = SoilProfile.from_file('IBSG910085', soil_path)
    dssat_env = DSSAT(workspace)

    initial_conditions = filex.InitialConditions(pcr='SG', icdat=datetime(1980, 7, 3), icres=1300)
    planting = filex.Planting(pdate=datetime(1980, 6, 17), ppop=18, plds='R', plrs=45, pldp=5)
    
    # ===================================================================
    # SECTION 2: TREATMENT COMPARISON (Rainfed vs. Irrigated)
    # ===================================================================
    print("\n--- SECTION 2: Running Treatment Comparison (Rainfed vs. Irrigated) ---")
    
    field = filex.Field(id_field='ITHY0001', wsta=weather_station, id_soil=soil)
    outputs_config = filex.SCOutputs(grout='Y', waout='Y', sumry='Y', vbose='Y')
    
    # Run Rainfed Treatment
    simulation_controls_rainfed = filex.SimulationControls(
        general=filex.SCGeneral(sdate=datetime(1980, 6, 17)),
        options=filex.SCOptions(water='Y', nitro='Y', symbi='N'),
        management=filex.SCManagement(irrig='N', ferti='N', harvs='M'),
        outputs=outputs_config
    )
    rainfed_summary = dssat_env.run_treatment(
        field=field,
        cultivar=crop.Sorghum('IB0026'),
        planting=planting,
        initial_conditions=initial_conditions,
        simulation_controls=simulation_controls_rainfed
    )
    rainfed_growth = dssat_env.output_tables.get('PlantGro').copy()

    # Create and read irrigation schedule from a CSV file
    irrigation_csv_path = os.path.join(workspace, 'irrigation_schedule.csv')
    irrigation_data = """date,amount
1980-07-15,20
1980-08-15,25
1980-09-01,15
"""
    with open(irrigation_csv_path, 'w') as f:
        f.write(irrigation_data)
    print(f"Created sample irrigation schedule at: {irrigation_csv_path}")
    
    schedule_df = pd.read_csv(irrigation_csv_path, parse_dates=['date'])
    
    irrigation_events = [
        IrrigationEvent(idate=row['date'], irval=row['amount'], irop='IR001')
        for index, row in schedule_df.iterrows()
    ]

    irrigation = filex.Irrigation(efir=1, table=irrigation_events)

    # Run Irrigated Treatment using the schedule from the file
    simulation_controls_irrigated = filex.SimulationControls(
        general=filex.SCGeneral(sdate=datetime(1980, 6, 17)),
        options=filex.SCOptions(water='Y', nitro='Y', symbi='N'),
        management=filex.SCManagement(irrig='R', ferti='N', harvs='M'),
        outputs=outputs_config
    )
    irrigated_summary = dssat_env.run_treatment(
        field=field,
        cultivar=crop.Sorghum('IB0026'),
        planting=planting,
        initial_conditions=initial_conditions,
        simulation_controls=simulation_controls_irrigated,
        irrigation=irrigation
    )
    irrigated_growth = dssat_env.output_tables.get('PlantGro').copy()
    irrigated_water = dssat_env.output_tables.get('SoilWat').copy()

    # Create Comparison Plot with Harvest Day markers
    plt.figure(figsize=(10, 6))
    plt.plot(rainfed_growth.index, rainfed_growth['GWAD'], label='Rainfed')
    plt.plot(irrigated_growth.index, irrigated_growth['GWAD'], label='Irrigated', linestyle='--')
    
    pdate_dssattools = planting['pdate']
    planting_date = datetime(pdate_dssattools.year, pdate_dssattools.month, pdate_dssattools.day)
    
    rainfed_mat_dap = rainfed_summary['mat']
    irrigated_mat_dap = irrigated_summary['mat']
    rainfed_harvest_date = planting_date + timedelta(days=rainfed_mat_dap)
    irrigated_harvest_date = planting_date + timedelta(days=irrigated_mat_dap)
    
    plt.axvline(x=rainfed_harvest_date, color='red', linestyle=':', label=f'Rainfed Harvest ({rainfed_mat_dap} DAP)')
    plt.axvline(x=irrigated_harvest_date, color='blue', linestyle=':', label=f'Irrigated Harvest ({irrigated_mat_dap} DAP)')
    
    plt.title('Treatment Comparison: Grain Growth')
    plt.xlabel('Date')
    plt.ylabel('Grain Weight (kg/ha)')
    plt.legend()
    plt.grid(True)
    comp_path = os.path.join(workspace, 'treatment_comparison_growth.png')
    plt.savefig(comp_path)
    print(f"Saved treatment comparison plot to: {comp_path}")
    plt.close()
    
    # ===================================================================
    # SECTION 3: PARAMETER SENSITIVITY (Varying 'p1' coefficient)
    # ===================================================================
    print("\n--- SECTION 3: Running Parameter Sensitivity Analysis (P1 Coefficient) ---")
    
    p1_values = [350, 380, 410, 440, 470]
    sensitivity_results = []
    
    cultivar_sens = crop.Sorghum('IB0026')
    simulation_controls_sens = filex.SimulationControls(
        general=filex.SCGeneral(sdate=datetime(1980, 6, 17)),
        options=filex.SCOptions(water='Y', nitro='Y', symbi='N'),
        management=filex.SCManagement(irrig='N', ferti='N', harvs='M'),
        outputs=filex.SCOutputs(sumry='Y') 
    )
    
    for p1 in p1_values:
        cultivar_sens['p1'] = p1
        print(f"Running simulation for p1 = {p1}...")
        summary = dssat_env.run_treatment(
            field=field,
            cultivar=cultivar_sens,
            planting=planting,
            initial_conditions=initial_conditions,
            simulation_controls=simulation_controls_sens
        )
        summary['p1'] = p1
        sensitivity_results.append(summary)
        
    results_df = pd.DataFrame(sensitivity_results)

    # Create Sensitivity Plots
    plt.figure(figsize=(10, 6))
    plt.plot(results_df['p1'], results_df['harwt'], marker='o')
    plt.title('Sensitivity Analysis: Yield vs. P1 Coefficient')
    plt.xlabel('P1 Value (GDD)')
    plt.ylabel('Grain Weight at Harvest (kg/ha)')
    plt.grid(True)
    sens_yield_path = os.path.join(workspace, 'sensitivity_yield_vs_p1.png')
    plt.savefig(sens_yield_path)
    print(f"Saved sensitivity plot (Yield vs P1) to: {sens_yield_path}")
    plt.close()

    plt.figure(figsize=(10, 6))
    plt.plot(results_df['p1'], results_df['mat'], marker='o')
    plt.title('Sensitivity Analysis: Maturity vs. P1 Coefficient')
    plt.xlabel('P1 Value (GDD)')
    plt.ylabel('Days to Maturity (dap)')
    plt.grid(True)
    sens_mat_path = os.path.join(workspace, 'sensitivity_maturity_vs_p1.png')
    plt.savefig(sens_mat_path)
    print(f"Saved sensitivity plot (Maturity vs P1) to: {sens_mat_path}")
    plt.close()

    # ===================================================================
    # SECTION 4: DETAILED DAILY TIME-SERIES PLOTS
    # ===================================================================
    print("\n--- SECTION 4: Generating Detailed Daily Plots ---")

    # Re-use results from the irrigated run in Section 2
    plt.figure(figsize=(10, 6))
    plt.plot(irrigated_growth.index, irrigated_growth['LAID'])
    plt.title('Daily Leaf Area Index (Irrigated Treatment)')
    plt.xlabel('Date')
    plt.ylabel('Leaf Area Index (LAI)')
    plt.grid(True)
    plt.axvline(x=irrigated_harvest_date, color='red', linestyle='--', label=f'Harvest Day ({irrigated_mat_dap} DAP)')
    plt.legend()
    lai_path = os.path.join(workspace, 'daily_lai.png')
    plt.savefig(lai_path)
    print(f"Saved LAI plot to: {lai_path}")
    plt.close()
    
    # Plot for Daily Yield (Grain Growth Rate)
    irrigated_growth['daily_yield'] = irrigated_growth['GWAD'].diff().fillna(0).clip(lower=0)
    plt.figure(figsize=(10, 6))
    plt.plot(irrigated_growth['DAP'], irrigated_growth['daily_yield'])
    plt.title('Daily Yield (Grain Growth Rate)')
    plt.xlabel('Days After Planting (DAP)')
    plt.ylabel('Daily Increase in Grain Weight (kg/ha/day)')
    plt.grid(True)
    plt.axvline(x=irrigated_mat_dap, color='red', linestyle='--', label='Harvest Day')
    plt.legend()
    daily_yield_path = os.path.join(workspace, 'daily_yield.png')
    plt.savefig(daily_yield_path)
    print(f"Saved daily yield plot to: {daily_yield_path}")
    plt.close()

    # Plot for Total Accumulated Yield
    plt.figure(figsize=(10, 6))
    plt.plot(irrigated_growth['DAP'], irrigated_growth['GWAD'], label='Total Accumulated Grain')
    
    final_yield = irrigated_growth.loc[irrigated_growth.index == irrigated_harvest_date, 'GWAD'].values[0]
    
    plt.axvline(x=irrigated_mat_dap, color='red', linestyle='--', label=f'Harvest Day ({irrigated_mat_dap} DAP)')
    plt.scatter(irrigated_mat_dap, final_yield, color='red', zorder=5, s=100, label=f'Final Yield: {int(final_yield)} kg/ha')
    
    plt.title('Total Accumulated Grain Yield Over Time')
    plt.xlabel('Days After Planting (DAP)')
    plt.ylabel('Total Grain Weight (kg/ha)')
    plt.legend()
    plt.grid(True)
    cumulative_yield_path = os.path.join(workspace, 'cumulative_yield.png')
    plt.savefig(cumulative_yield_path)
    print(f"Saved cumulative yield plot to: {cumulative_yield_path}")
    plt.close()

    # Plot for Daily Water Input (Rain & Irrigation)
    df_weather_indexed = df_weather.set_index('date')
    plot_dates = pd.date_range(start=planting_date, end=irrigated_harvest_date + timedelta(days=5))
    df_water_plot = pd.DataFrame(index=plot_dates)

    df_water_plot = df_water_plot.join(df_weather_indexed['rain'])
    df_water_plot.rename(columns={'rain': 'RAIN'}, inplace=True)
    
    # FIXED: Replaced `inplace=True` to remove the FutureWarning
    df_water_plot['RAIN'] = df_water_plot['RAIN'].fillna(0)
    
    irrigation_series = schedule_df.set_index('date')['amount']
    df_water_plot = df_water_plot.join(irrigation_series)
    df_water_plot.rename(columns={'amount': 'IRRIGATION'}, inplace=True)
    
    # FIXED: Replaced `inplace=True` to remove the FutureWarning
    df_water_plot['IRRIGATION'] = df_water_plot['IRRIGATION'].fillna(0)

    plt.figure(figsize=(12, 6))
    plt.plot(df_water_plot.index, df_water_plot['RAIN'], label='Rain', color='blue')
    plt.plot(df_water_plot.index, df_water_plot['IRRIGATION'], label='Irrigation', color='purple', linestyle='--')

    plt.title('Daily Water Input (Rain & Irrigation)')
    plt.xlabel('Date')
    plt.ylabel('Water (mm/day)')
    
    max_water = max(df_water_plot['RAIN'].max(), df_water_plot['IRRIGATION'].max())
    plt.ylim(-5, max_water * 1.1)
    
    plt.legend()
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    water_input_path = os.path.join(workspace, 'daily_water_input.png')
    plt.savefig(water_input_path)
    print(f"Saved daily water input plot to: {water_input_path}")
    plt.close()


if __name__ == "__main__":
    run_all_analyses_and_visualizations()

Attempting to install package using 'python3'...

--- SECTION 1: Initializing Workspace and Data ---
DSSAT workspace created at: d:\Final-Year-Project\dssat_simulation_workspace
Successfully downloaded SOIL.SOL
Successfully downloaded ITHY8001.WTH
Successfully downloaded ITHY8101.WTH
d:\Final-Year-Project\dssat_simulation_workspace created.

--- SECTION 2: Running Treatment Comparison (Rainfed vs. Irrigated) ---

RUN    TRT FLO MAT TOPWT HARWT  RAIN  TIRR   CET  PESW  TNUP  TNLF   TSON TSOC
           dap dap kg/ha kg/ha    mm    mm    mm    mm kg/ha kg/ha  kg/ha t/ha
  1 SG   1  67 109  1985   612   592     0   307   130    15     8  13377   95
Created sample irrigation schedule at: d:\Final-Year-Project\dssat_simulation_workspace\irrigation_schedule.csv

RUN    TRT FLO MAT TOPWT HARWT  RAIN  TIRR   CET  PESW  TNUP  TNLF   TSON TSOC
           dap dap kg/ha kg/ha    mm    mm    mm    mm kg/ha kg/ha  kg/ha t/ha
  1 SG   1  67 109  1824   553   592    60   313   131    14     8  13377  

In [45]:
import os
import urllib.request
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import subprocess
import sys

def install_package(package):
    """Installs a package using pip, trying 'python3' and then 'python'."""
    try:
        print("Attempting to install package using 'python3'...")
        subprocess.check_call(["python3", "-m", "pip", "install", package])
        print(f"Successfully installed {package} with 'python3'")
    except FileNotFoundError:
        print("'python3' not found. Trying 'python' instead...")
        try:
            subprocess.check_call(["python", "-m", "pip", "install", package])
            print(f"Successfully installed {package} with 'python'")
        except FileNotFoundError:
            print("Error: Neither 'python3' nor 'python' command was found in the environment's PATH.")
            raise
        except subprocess.CalledProcessError as e:
            print(f"Error installing {package} with 'python': {e}")
            raise
    except subprocess.CalledProcessError as e:
        print(f"Error installing {package} with 'python3': {e}")
        raise


def download_file(url, target_path):
    """Downloads a file from a URL and saves it locally."""
    try:
        with urllib.request.urlopen(url) as response, open(target_path, 'wb') as out_file:
            data = response.read()
            out_file.write(data)
        print(f"Successfully downloaded {os.path.basename(target_path)}")
    except Exception as e:
        print(f"Error downloading {url}: {e}")
        raise

def run_all_analyses_and_visualizations():
    """
    Sets up a DSSAT environment, runs multiple analysis scenarios,
    and generates all relevant visualizations.
    """
    # First, ensure DSSATTools is installed
    install_package('DSSATTools')
    
    # Now import from the newly installed package
    from DSSATTools import (
        crop,
        WeatherStation,
        SoilProfile,
        filex,
        DSSAT
    )
    from DSSATTools.filex import IrrigationEvent, FertilizerEvent

    # ===================================================================
    # SECTION 1: BASIC SETUP AND DATA ACQUISITION
    # ===================================================================
    print("\n--- SECTION 1: Initializing Workspace and Data ---")
    workspace = os.path.join(os.getcwd(), 'dssat_simulation_workspace')
    os.makedirs(workspace, exist_ok=True)
    print(f"DSSAT workspace created at: {workspace}")

    soil_path = os.path.join(workspace, 'SOIL.SOL')
    download_file('https://raw.githubusercontent.com/DSSAT/dssat-csm-data/develop/Soil/SOIL.SOL', soil_path)

    weather_dfs = []
    for year in ['80', '81']:
        filename = f'ITHY{year}01.WTH'
        url = f'https://raw.githubusercontent.com/DSSAT/dssat-csm-data/develop/Weather/{filename}'
        path = os.path.join(workspace, filename)
        download_file(url, path)
        df = pd.read_fwf(path, colspecs=[(0, 5), (6, 11), (12, 17), (18, 23), (24, 29)], 
                         names=['date', 'srad', 'tmax', 'tmin', 'rain'], skiprows=4)
        weather_dfs.append(df)

    df_weather = pd.concat(weather_dfs, ignore_index=True)
    df_weather.dropna(subset=['date'], inplace=True)
    df_weather['date'] = pd.to_datetime(df_weather['date'], format='%y%j')
    df_weather.drop_duplicates(subset='date', keep='first', inplace=True)
    df_weather.sort_values(by='date', inplace=True)

    weather_station = WeatherStation(insi='ITHY', lat=17.53, long=78.27, elev=0, tav=25.8, amp=11.8, table=df_weather)
    soil = SoilProfile.from_file('IBSG910085', soil_path)
    dssat_env = DSSAT(workspace)

    initial_conditions = filex.InitialConditions(pcr='SG', icdat=datetime(1980, 7, 3), icres=1300)
    planting = filex.Planting(pdate=datetime(1980, 6, 17), ppop=18, plds='R', plrs=45, pldp=5)
    
    # ===================================================================
    # SECTION 2: COMBINED RAINFED AND IRRIGATED TREATMENT
    # ===================================================================
    print("\n--- SECTION 2: Running Combined Rainfed & Irrigated Simulation ---")
    
    field = filex.Field(id_field='ITHY0001', wsta=weather_station, id_soil=soil)
    outputs_config = filex.SCOutputs(grout='Y', waout='Y', sumry='Y', vbose='Y')
    
    # Create and read irrigation schedule from a CSV file
    irrigation_csv_path = os.path.join(workspace, 'irrigation_schedule.csv')
    irrigation_data = """date,amount
1980-07-15,20
1980-08-15,25
1980-09-01,15
"""
    with open(irrigation_csv_path, 'w') as f:
        f.write(irrigation_data)
    print(f"Created sample irrigation schedule at: {irrigation_csv_path}")
    
    schedule_df = pd.read_csv(irrigation_csv_path, parse_dates=['date'])
    
    irrigation_events = [
        IrrigationEvent(idate=row['date'], irval=row['amount'], irop='IR001')
        for index, row in schedule_df.iterrows()
    ]

    irrigation = filex.Irrigation(efir=1, table=irrigation_events)

    # Run the combined treatment
    simulation_controls = filex.SimulationControls(
        general=filex.SCGeneral(sdate=datetime(1980, 6, 17)),
        options=filex.SCOptions(water='Y', nitro='Y', symbi='N'),
        management=filex.SCManagement(irrig='R', ferti='N', harvs='M'),
        outputs=outputs_config
    )
    summary_results = dssat_env.run_treatment(
        field=field,
        cultivar=crop.Sorghum('IB0026'),
        planting=planting,
        initial_conditions=initial_conditions,
        simulation_controls=simulation_controls,
        irrigation=irrigation
    )
    growth_data = dssat_env.output_tables.get('PlantGro').copy()

    pdate_dssattools = planting['pdate']
    planting_date = datetime(pdate_dssattools.year, pdate_dssattools.month, pdate_dssattools.day)
    
    harvest_dap = summary_results['mat']
    harvest_date = planting_date + timedelta(days=harvest_dap)
    
    # ===================================================================
    # SECTION 3: PARAMETER SENSITIVITY (Varying 'p1' coefficient)
    # ===================================================================
    print("\n--- SECTION 3: Running Parameter Sensitivity Analysis (P1 Coefficient) ---")
    
    p1_values = [350, 380, 410, 440, 470]
    sensitivity_results = []
    
    cultivar_sens = crop.Sorghum('IB0026')
    simulation_controls_sens = filex.SimulationControls(
        general=filex.SCGeneral(sdate=datetime(1980, 6, 17)),
        options=filex.SCOptions(water='Y', nitro='Y', symbi='N'),
        management=filex.SCManagement(irrig='R', ferti='N', harvs='M'),
        outputs=filex.SCOutputs(sumry='Y') 
    )
    
    for p1 in p1_values:
        cultivar_sens['p1'] = p1
        print(f"Running simulation for p1 = {p1}...")
        summary = dssat_env.run_treatment(
            field=field,
            cultivar=cultivar_sens,
            planting=planting,
            initial_conditions=initial_conditions,
            simulation_controls=simulation_controls_sens,
            irrigation=irrigation
        )
        summary['p1'] = p1
        sensitivity_results.append(summary)
        
    results_df = pd.DataFrame(sensitivity_results)

    # Create Sensitivity Plots
    plt.figure(figsize=(10, 6))
    plt.plot(results_df['p1'], results_df['harwt'], marker='o')
    plt.title('Sensitivity Analysis: Yield vs. P1 Coefficient')
    plt.xlabel('P1 Value (GDD)')
    plt.ylabel('Grain Weight at Harvest (kg/ha)')
    plt.grid(True)
    sens_yield_path = os.path.join(workspace, 'sensitivity_yield_vs_p1.png')
    plt.savefig(sens_yield_path)
    print(f"Saved sensitivity plot (Yield vs P1) to: {sens_yield_path}")
    plt.close()

    plt.figure(figsize=(10, 6))
    plt.plot(results_df['p1'], results_df['mat'], marker='o')
    plt.title('Sensitivity Analysis: Maturity vs. P1 Coefficient')
    plt.xlabel('P1 Value (GDD)')
    plt.ylabel('Days to Maturity (dap)')
    plt.grid(True)
    sens_mat_path = os.path.join(workspace, 'sensitivity_maturity_vs_p1.png')
    plt.savefig(sens_mat_path)
    print(f"Saved sensitivity plot (Maturity vs P1) to: {sens_mat_path}")
    plt.close()

    # ===================================================================
    # SECTION 4: DETAILED DAILY TIME-SERIES PLOTS
    # ===================================================================
    print("\n--- SECTION 4: Generating Detailed Daily Plots ---")

    # Plot for Daily Leaf Area Index (LAI)
    plt.figure(figsize=(10, 6))
    plt.plot(growth_data.index, growth_data['LAID'])
    plt.title('Daily Leaf Area Index (Combined Treatment)')
    plt.xlabel('Date')
    plt.ylabel('Leaf Area Index (LAI)')
    plt.grid(True)
    plt.axvline(x=harvest_date, color='red', linestyle='--', label=f'Harvest Day ({harvest_dap} DAP)')
    plt.legend()
    lai_path = os.path.join(workspace, 'daily_lai.png')
    plt.savefig(lai_path)
    print(f"Saved LAI plot to: {lai_path}")
    plt.close()
    
    # Plot for Daily Yield (Grain Growth Rate)
    growth_data['daily_yield'] = growth_data['GWAD'].diff().fillna(0).clip(lower=0)
    plt.figure(figsize=(10, 6))
    plt.plot(growth_data['DAP'], growth_data['daily_yield'])
    plt.title('Daily Yield (Grain Growth Rate)')
    plt.xlabel('Days After Planting (DAP)')
    plt.ylabel('Daily Increase in Grain Weight (kg/ha/day)')
    plt.grid(True)
    plt.axvline(x=harvest_dap, color='red', linestyle='--', label='Harvest Day')
    plt.legend()
    daily_yield_path = os.path.join(workspace, 'daily_yield.png')
    plt.savefig(daily_yield_path)
    print(f"Saved daily yield plot to: {daily_yield_path}")
    plt.close()

    # Plot for Total Accumulated Yield
    plt.figure(figsize=(10, 6))
    plt.plot(growth_data['DAP'], growth_data['GWAD'], label='Total Accumulated Grain')
    
    final_yield = growth_data.loc[growth_data.index == harvest_date, 'GWAD'].values[0]
    
    plt.axvline(x=harvest_dap, color='red', linestyle='--', label=f'Harvest Day ({harvest_dap} DAP)')
    plt.scatter(harvest_dap, final_yield, color='red', zorder=5, s=100, label=f'Final Yield: {int(final_yield)} kg/ha')
    
    plt.title('Total Accumulated Grain Yield Over Time')
    plt.xlabel('Days After Planting (DAP)')
    plt.ylabel('Total Grain Weight (kg/ha)')
    plt.legend()
    plt.grid(True)
    cumulative_yield_path = os.path.join(workspace, 'cumulative_yield.png')
    plt.savefig(cumulative_yield_path)
    print(f"Saved cumulative yield plot to: {cumulative_yield_path}")
    plt.close()

    # MODIFIED: Plot for Daily Water Input (Rain & Irrigation as Lines with negative y-axis)
    # --------------------------------------------------------------------------------------
    df_weather_indexed = df_weather.set_index('date')
    
    # Create a DataFrame for plotting that covers the whole simulation period
    plot_dates = pd.date_range(start=planting_date, end=harvest_date + timedelta(days=5))
    df_water_plot = pd.DataFrame(index=plot_dates)

    # Join rain data
    df_water_plot = df_water_plot.join(df_weather_indexed['rain'])
    df_water_plot.rename(columns={'rain': 'RAIN'}, inplace=True)
    
    # Corrected lines to avoid FutureWarning
    df_water_plot['RAIN'] = df_water_plot['RAIN'].fillna(0)
    
    # Join irrigation data from the schedule
    irrigation_series = schedule_df.set_index('date')['amount']
    df_water_plot = df_water_plot.join(irrigation_series)
    df_water_plot.rename(columns={'amount': 'IRRIGATION'}, inplace=True)

    # Corrected lines to avoid FutureWarning
    df_water_plot['IRRIGATION'] = df_water_plot['IRRIGATION'].fillna(0)

    plt.figure(figsize=(12, 6))
    # Plot rain and irrigation as separate lines
    plt.plot(df_water_plot.index, df_water_plot['RAIN'], label='Rain', color='blue')
    plt.plot(df_water_plot.index, df_water_plot['IRRIGATION'], label='Irrigation', color='purple', linestyle='--')

    plt.title('Daily Water Input (Rain & Irrigation)')
    plt.xlabel('Date')
    plt.ylabel('Water (mm/day)')
    
    # Set Y-axis to show a negative portion
    max_water = max(df_water_plot['RAIN'].max(), df_water_plot['IRRIGATION'].max())
    plt.ylim(-5, max_water * 1.1)
    
    plt.legend()
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    water_input_path = os.path.join(workspace, 'daily_water_input.png')
    plt.savefig(water_input_path)
    print(f"Saved daily water input plot to: {water_input_path}")
    plt.close()


if __name__ == "__main__":
    run_all_analyses_and_visualizations()

Attempting to install package using 'python3'...

--- SECTION 1: Initializing Workspace and Data ---
DSSAT workspace created at: d:\Final-Year-Project\dssat_simulation_workspace
Successfully downloaded SOIL.SOL
Successfully downloaded ITHY8001.WTH
Successfully downloaded ITHY8101.WTH
d:\Final-Year-Project\dssat_simulation_workspace created.

--- SECTION 2: Running Combined Rainfed & Irrigated Simulation ---
Created sample irrigation schedule at: d:\Final-Year-Project\dssat_simulation_workspace\irrigation_schedule.csv

RUN    TRT FLO MAT TOPWT HARWT  RAIN  TIRR   CET  PESW  TNUP  TNLF   TSON TSOC
           dap dap kg/ha kg/ha    mm    mm    mm    mm kg/ha kg/ha  kg/ha t/ha
  1 SG   1  67 109  1824   553   592    60   313   131    14     8  13377   95

--- SECTION 3: Running Parameter Sensitivity Analysis (P1 Coefficient) ---
Running simulation for p1 = 350...

RUN    TRT FLO MAT TOPWT HARWT  RAIN  TIRR   CET  PESW  TNUP  TNLF   TSON TSOC
           dap dap kg/ha kg/ha    mm    mm    mm